In [17]:
import random

# Failed v17

import inspect

def print_variables(*args):
    frame = inspect.currentframe()
    frame = inspect.getouterframes(frame)[1]
    string = inspect.getframeinfo(frame[0]).code_context[0].strip()
    # print('inspect.getframeinfo(frame[0]).code_context',inspect.getframeinfo(frame[0]).code_context)
    # print('string:',string)
    args_base = string[string.find('(') + 1:-1].replace('\n','').split(',')
   
    names = []
    for i in args_base:
        if i.find('=') != -1:
            names.append(i.split('=')[1].strip())
       
        else:
            names.append(i)

    variable_dict = {name:arg for name,arg in zip(names,args)}
    
    ind = names[0]
    print(f'\ntata{ind} start')
    for k,v in variable_dict.items():
        if k==ind:
            continue
        print(f'{k}: {v}')

    print(f'tata{ind} end\n')
    
    
def print_variables(*args):
    pass
    
pp = print_variables
p = lambda x: print(f'pointer {x}')


from collections import deque,Counter

class Node:
    def __init__(self, val):
        self.val = val
        self.well_cost = None
        self.pipes = dict()
        self.is_outer = True
        self.info = {
            'connected_min_well_node':{},
            'connected_min_well_cost':{},
            'connected_pipe_cost':{},
            'connected_pipe_set':{}
        }
        
    def check_outer(self):
        if len(self.pipes)>1:
            self.is_outer = False
        else:
            self.is_outer = True
            
    def add_pipe(self, other_node,cost):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        self.pipes[other_node_val] = cost
        self.check_outer()
    
    def remove_pipe(self, other_node):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        del self.pipes[other_node_val]
        self.check_outer()
        
    def __eq__(self, other):
        return self.val==other.val
    
    def __gt__(self,other):
        return self.val>other.val
        
    def __lt__(self, other):
        return self.val<other.val
    
    def __hash__(self):
        return hash(self.val)
    
        

class Solution:
    
    def minCostToSupplyWater_helper(self, n, wells, pipes,testing=False):
        
        BASE_NODE = 2 #77#65#-1#46# 64 # 33 # 25 # 326
        
        
        def packupNode(n, well_cost, pipes):
            # Initial Node
            

            if type(well_cost)==dict:
                node_dict = {i:Node(i) for i in well_cost.keys()}
                
                for k,v in well_cost.items():
                    node_dict[k].well_cost = v
                
            elif type(well_cost)==list:
                node_dict = {i:Node(i) for i in range(1,n+1)}
                for i in range(len(well_cost)):
                    node_dict[i+1].well_cost = well_cost[i]
            else:
                raise

            # Add Edges  
            for edge in pipes:
                node_a = node_dict[edge[0]]
                node_b = node_dict[edge[1]]
                cost = edge[2]
                
                if node_a.val in node_b.pipes:
                    if  node_b.pipes[node_a.val]>cost:
                        node_b.add_pipe(node_a.val,cost)
                        node_a.add_pipe(node_b.val,cost)
                else:
                    node_b.add_pipe(node_a.val,cost)
                    node_a.add_pipe(node_b.val,cost)     
                    
            return node_dict
    

        def getGroupCost(group_set, node_dict):

#             well_status = {}
            well_status = {x.val:0 for  x in group_set}
            pipe_status = {}


            connected_node_set = set()
            connected_pipe_set = set()
            
            # Round1 Edge to Fork (or edge)
            def get_pipe_key(node1,node2):
                if (type(node1)==str) or (type(node1)==int):
                    key = tuple(sorted([node1, node2]))
                else:
                    key = tuple(sorted([node1.val, node2.val]))
                return key
            
            
            
            def update_pipe_status(node1,node2,status):
                key = get_pipe_key(node1,node2)
                pipe_status[key] = status 
                
                
                
            def disconnect(node1,node2):
                # pp(' Disconnect',node1.val,node2.val)
                if (type(node1) in (str,int)):
                    node1 = node_dict[node1]
                if (type(node2) in (str,int)):
                    node2 = node_dict[node2]
                node1.remove_pipe(node2)
                node2.remove_pipe(node1)
                
            def clean_edge(node):
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if (cost> neighbor_node.well_cost) and (cost>node.well_cost):
                        disconnect(node,neighbor_node)
                    

            def check_t1(node):# Center Smallest
                if node.val==BASE_NODE:
                    pp(1.01,well_status)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                # pp(' t1',node.val)
                min_node = node.val
                well_cost = node.well_cost
                for neighbor,cost in node.pipes.items():
                    if cost < well_cost:
                        return False,None,None
                return True,min_node,well_cost
                
                
            def check_t2(node): # All neighbor connected, choose the min pipe
                # pp(' t2',node.val)
                if node.val==BASE_NODE:
                    pp(2.01)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                min_node = node.val
                min_cost = node.well_cost
                for neighbor,cost in node.pipes.items():
                    if not neighbor in connected_node_set:
                        return False,None,None
                    if cost<min_cost:
                        min_cost = cost
                        min_node = neighbor
                        
                return True, min_node, min_cost
                    
                
#             def check_t3(node): # Short version
#                 min_node = node.val
#                 min_cost = node.well_cost
#                 for neighbor,cost in node.pipes.items():
#                     if not neighbor in connected_node_set:
#                         neighbor_node = node_dict[neighbor]
#                         for n,_ in neighbor_node.pipes.items():
#                             if n==node.val:
#                                 continue
#                             else:
#                                 if not n in connected_node_set:
#                                     return False,None,None
#                         if (cost+neighbor_node.well_cost)<min_cost:
#                             min_cost = cost+neighbor_node.well_cost
#                             min_node = neighbor_node.val
                            
#                     else:
#                         if cost<min_cost:
#                             min_cost = cost
#                             min_node = neighbor
                
#                 return True,min_node,min_cost
                
                
            def get_connected_group(node, connected_set):
                stack = deque()
                group_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                
                while(stack):
                    cur = stack.pop()
                    group_set.add(cur)
                    for pipe, _ in cur.pipes.items():
                        adj_node = node_dict[pipe]
                        if (not adj_node in group_set) and (not adj_node in connected_set):
                            stack.append(adj_node)   
                            if adj_node.well_cost<min_cost:
                                min_node = adj_node.val
                                min_cost = adj_node.well_cost
                return min_node,min_cost
                
                
            # Unknown
            def check_t4(node): 
                min_node,min_cost = get_connected_group(node, connected_node_set)
                return True, min_node,min_cost
                
            # Not sure
            def check_t5(node):
                pass
                
            # 
            def check_t6(node):  # is outer or connected 
                # pp(6)
                if node.val==BASE_NODE:
                    pp(6.01)
                    p = pp
                    import pdb
                    bp = pdb.set_trace
                else:
                    def p(*args):
                        pass
                    def bp():
                        pass
                
                
                min_node = node.val
                min_node_cost = node.well_cost
                min_pipe_cost = 0
                min_cost = (min_node_cost+min_pipe_cost)
                
                with_additional_node = True
                with_additional_pipe = False
                for neighbor,cost in node.pipes.items():
                    
                    p(6.010,neighbor)
                    
                    neighbor_node = node_dict[neighbor]

                    if (not neighbor in connected_node_set) and neighbor_node.is_outer:
                        if neighbor_node.well_cost<cost:
                            with_additional_node = True
                            with_additional_pipe = False
                            min_node = neighbor
                            return True,min_node,neighbor_node.well_cost, with_additional_node,with_additional_pipe
                    
                    
                    p(6.010,node.val,connected_node_set,neighbor,cost,neighbor_node.val,neighbor_node.well_cost,neighbor_node.is_outer,min_cost,neighbor_node.pipes.keys(),well_status,pipe_status)
                    
                    min_cost = (min_node_cost+min_pipe_cost)
                    
                    
                    if neighbor in connected_node_set:
                        p(6.0101)
                        if min_node_cost==0: # pre is pipe
                            if cost<min_pipe_cost:
                                min_pipe_cost = cost
                                min_node_cost = 0
                                min_node = neighbor
                                with_additional_node = False
                                with_additional_pipe = True
                        else:
                            if cost<min_node_cost:
                                min_pipe_cost = cost
                                min_node_cost = 0
                                min_node = neighbor
                                with_additional_node = False
                                with_additional_pipe = True
                    
                    elif neighbor_node.is_outer:
                        p(6.0102)
                        
                        if min_node_cost==0: # pre is pipe
                            if neighbor_node.well_cost <min_pipe_cost:
                                min_pipe_cost = cost
                                min_node_cost = neighbor_node.well_cost
                                min_node = neighbor
                                with_additional_node = True
                                with_additional_pipe = True
                        else:
#                             bp()
                            if neighbor_node.well_cost <min_node_cost:
                                min_pipe_cost = cost
                                min_node_cost = neighbor_node.well_cost
                                min_node = neighbor
                                with_additional_node = True
                                with_additional_pipe = True
                            
                    elif cost>min_cost:
                        continue
                    else:
                        p(6.0103)
                        return False, None, None, None, None 
                        
                p(6.011,node.val,neighbor,cost,neighbor_node.val,neighbor_node.well_cost,min_node,well_status,pipe_status)
                
                return True, min_node, min_node_cost+min_pipe_cost, with_additional_node,with_additional_pipe
                
                
            def check_t7(node):  # Disconnected Group Min
                # pp(7)
                if node.val==BASE_NODE:
                    pp(7.01)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                stack = deque()
                group_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                
                while(stack):
                    cur = stack.pop()
                    if cur.val in connected_node_set:
                        return False, None, None
                    group_set.add(cur)               
                    if cur.well_cost<min_cost:
                        min_node = cur.val
                        min_cost = cur.well_cost
                    
                    for pipe, _ in cur.pipes.items():
                        adj_node = node_dict[pipe]
                        if adj_node.val in connected_node_set:
                            return False, None, None
                        elif not adj_node in group_set:
                            stack.append(adj_node)
                p(7.01,'Success')
                return True, min_node,min_cost
                
              
            
#             def check_t8(node):  # is outer min_node in surounding connected edges or surounding not connected edges 
#                                  # but with larger cost 
#                 if node.val==BASE_NODE:
#                     pp(8.01)
#                     p = pp
#                 else:
#                     def p(*args):
#                         pass
                
                
#                 min_node = node.val
#                 min_node_cost = node.well_cost
#                 min_pipe_cost = 0
                
#                 min_cost = min_node_cost+min_pipe_cost
                
#                 with_additional_node = False
#                 not_connected_node_list = []
#                 for neighbor,cost in node.pipes.items():
#                     neighbor_node = node_dict[neighbor]
                    
#                     min_cost = min_node_cost+min_pipe_cost
                    
#                     if neighbor in connected_node_set:
#                         if cost<min_cost:
#                             min_cost = cost
#                             min_node = neighbor
#                             with_additional_node=False
                        
#                     elif neighbor_node.is_outer:
#                         if (neighbor_node.well_cost+cost)<min_cost:
#                             min_cost = neighbor_node.well_cost+cost
#                             min_node = neighbor
#                             with_additional_node=True
                            
#                     else: # not connected not outer
#                         not_connected_node_list.append((neighbor, cost))
                        
#                 for neighbor,cost in not_connected_node_list:
#                     if cost<min_cost:
#                         return False, None, None, None
 
#                 p(8.01,'Success')
#                 return True, min_node,min_cost,with_additional_node


                    
            def check_t9(node):    # Min neighbor node
                if node.val==BASE_NODE:
                    pp(9.01)
                    p = pp
                else:
                    def p(*args):
                        pass

                min_node = node.val
                min_well_cost = node.well_cost
                min_pipe_cost = 0
                
                p(9.010,well_status)
                with_additional_node = False
                for neighbor,cost in node.pipes.items():
                    neighbor_node = node_dict[neighbor]
                    p(9.011,neighbor,neighbor_node.is_outer,cost,min_node,min_well_cost,min_pipe_cost,with_additional_node)
                    
                    if neighbor in connected_node_set:
                        if min_well_cost==0: 
                            if cost<min_pipe_cost:
                                min_pipe_cost = cost
                                min_node = neighbor
                                min_well_cost = 0
                                with_additional_node = False

                        else:
                            if cost<min_well_cost:
                                min_pipe_cost = cost
                                min_node = neighbor
                                min_well_cost = 0
                                with_additional_node = False
                        # pass
                    
                    else:
                        for nn,cc in neighbor_node.pipes.items():
                            if nn==node.val:
                                continue 
                                
                            nn_node = node_dict[nn]
                            if cc<neighbor_node.well_cost:
                                p(9.011,node.val,neighbor,nn,cc,neighbor_node.well_cost)
                                return False, None, None, None
                            elif (neighbor_node.well_cost>min_well_cost) and (cost<min_pipe_cost):
                                return False, None, None, None
                            

#                         bp()
                        if min_well_cost==0: 
        
                            if neighbor_node.is_outer and (neighbor_node.well_cost<min_pipe_cost) and (cost<min_pipe_cost):
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                            
                            elif (neighbor_node.well_cost+cost) < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                            
                            elif neighbor_node.well_cost < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                        else:
                            
                            if cost > (min_pipe_cost+min_well_cost):
                                continue
                                     
                            
                            elif (neighbor_node.well_cost+cost) < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                                
                            elif neighbor_node.well_cost < min_well_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                        # pass
                    
                p(9.012,node.val,min_node,min_well_cost,min_pipe_cost,with_additional_node)
                p(9.01,'Success')
                return True,min_node,min_well_cost+min_pipe_cost,with_additional_node
            
            
            
            def check_t10(node):  # Min Node or Already Connected Pipe in a Group
                # pp(10)
                if node.val==BASE_NODE:
                    pp(10.01)
                    p = pp
                else:
                    def p(*args):
                        pass

                stack = deque()
                checked_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                pre_node = None
                
                with_additional_node=True
                while(stack):
                    cur = stack.pop()
                    checked_set.add(cur.val)               
                    if cur.well_cost<min_cost:
                        min_node = cur.val
                        min_cost = cur.well_cost
                        with_additional_node=True
                        pre_node = None
                    
                    for adj, cc in cur.pipes.items():
                        adj_node = node_dict[adj]
                        if adj in connected_node_set:
                            checked_set.add(adj)
                            if cc<min_cost:
                                min_node = adj
                                min_cost = cc
                                with_additional_node = False
                                pre_node = cur.val
    
                        elif not adj in checked_set:
                            stack.append(adj_node)   
                
                
                p(10.011,node.val,min_node,min_cost,with_additional_node,pre_node)
                p(10.01,'Success')
                return True,min_node,min_cost,with_additional_node,pre_node
                
            
            def clean_edge_II(node): # Pruning
                min_node = node.val
                min_cost = node.well_cost
                changed_ind=False
                
                with_additional_node=False
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if neighbor in connected_node_set:
                        if cost<min_cost:
                            min_cost = cost
                            min_node = neighbor
                            with_additional_node = False
                    # elif neighbor_node.is_outer:
                    elif (neighbor_node.well_cost+cost)<min_cost:
                            min_cost = neighbor_node.well_cost+cost
                            min_node = neighbor
                            with_additional_node = True
                            
                    else:
                        pass
                    
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if neighbor in connected_node_set:
                        if cost>min_cost:
                            disconnect(node,neighbor)
                            changed_ind=True
            
                return changed_ind
            
            
            def check_min_cneter(node):
                
                for pipe,cost in node.pipes.items():
                    if cost<node.well_cost:
                        return False
                return True
            
            
            
            for node in group_set:
                clean_edge(node)
                
            # print()
            # for x in group_set:
            #     print('node:',x.val)
            #     # print('info:',x.info)
            #     print('pipes:',x.pipes.keys())
            #     print('is_outer:',x.is_outer)
            #     print()
            
            # Check 0 
            for node in list(group_set):
                if check_min_cneter(node):
                    group_set.remove(node)
                    well_status[node.val]=1
                    connected_node_set.add(node.val)
            
            
                
            stack = deque(group_set)
            
            
            import copy
            pre = copy.copy(stack)
            pre_pipe_status = copy.copy(pipe_status)
            pre_well_status = copy.copy(well_status)
            while(stack):
                for i in range(len(stack)):
                    if not stack:
                        break
#                     print('stack:',[x.val for x in stack])
                        
                    cur = stack.popleft()
                    
                    pp('#####################################################',cur.val,well_status)
                    
                    # if cur.val in connected_node_set:
                    #     continue
                    
                    ind, min_node, min_cost = check_t1(cur)
                    if ind:
                        connected_node_set.add(cur.val)
                        well_status[cur.val] = 1
                        continue

                    ind, min_node, min_cost = check_t2(cur)
                    if ind:
                        connected_node_set.add(cur.val)
                        if cur.val!=min_node:
                            pipe_key = get_pipe_key(min_node,cur.val)
                            connected_pipe_set.add(pipe_key)
                            pipe_status[pipe_key] = 1
                        continue  

                    # ind, min_node, min_cost = check_t3(cur)
                    # if ind:
                    #     connected_node_set.add(min_node)
                    #     connected_node_set.add(cur.val)
                    #     if cur.val!=min_node:
                    #         key = get_pipe_key(min_node,cur.val)
                    #         connected_pipe_set.add(key)
                    #     continue

                    ind, min_node, min_cost,with_additional_node,with_additional_pipe = check_t6(cur)
                    if ind:
                        pp(6)
                        if with_additional_node and (not with_additional_pipe):
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            if min_node!=cur.val:
                                stack.append(cur)
                            
                        elif with_additional_node and with_additional_pipe:
                            connected_node_set.add(cur.val)
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            if cur.val!=min_node:
                                pipe_key = get_pipe_key(min_node,cur.val)
                                connected_pipe_set.add(pipe_key)
                                pipe_status[pipe_key] = 1
                        
                        elif (not with_additional_node) and with_additional_pipe:
                            connected_node_set.add(cur.val)
                            if cur.val!=min_node:
                                pipe_key = get_pipe_key(min_node,cur.val)
                                connected_pipe_set.add(pipe_key)
                                pipe_status[pipe_key] = 1
                        else: # (not with_additional_node) and (not with_additional_pipe):
                            pass
                        

                        continue
                              
                    """
                    True, min_node, min_node_cost+min_pipe_cost, with_additional_node,with_additional_pipe
                    
                    """        
                    
                            
                            
                            
                    ind, min_node, min_cost = check_t7(cur)
                    if ind:
                        pp(7,min_node,well_status)
                        connected_node_set.add(min_node)
                        well_status[min_node] = 1
                        
                        if min_node!=cur.val:
                            stack.append(cur)
                        # pp('t7',min_node)
                        continue 
                        

                        
#                     ind, min_node, min_cost,with_additional_node = check_t8(cur)
#                     if ind:
#                         pp(8)
#                         connected_node_set.add(cur.val)
#                         connected_node_set.add(min_node)
#                         if cur.val!=min_node:
#                             pipe_key = get_pipe_key(min_node,cur.val)
#                             connected_pipe_set.add(pipe_key)
#                             pipe_status[pipe_key] = 1
                    
#                         if cur.val==min_node:
#                             well_status[cur.val] = 1
#                         elif with_additional_node:
#                             well_status[min_node] = 1

#                         continue
                          
                            
                    ind, min_node, min_cost,with_additional_node = check_t9(cur) # Inner Net
                    if ind:
                        pp(9)
                        connected_node_set.add(cur.val)
                        connected_node_set.add(min_node)
                        if cur.val!=min_node:
                            pipe_key = get_pipe_key(min_node,cur.val)
                            connected_pipe_set.add(pipe_key)
                            pipe_status[pipe_key] = 1
                    
                        if cur.val==min_node:
                            well_status[cur.val] = 1
                        elif with_additional_node:
                            well_status[min_node] = 1

                        pp(9.1,well_status,pipe_status)
                        continue


                    ind, min_node, min_cost,with_additional_node,pre_node = check_t10(cur)
                    if ind:
                        pp(10,min_node,well_status)
                            
                        if with_additional_node:
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            
                            if min_node!=cur.val:
                                stack.append(cur)
                        else:
                            # if (not min_node) or (not(pre_node))
                            pipe_key = get_pipe_key(min_node,pre_node)
                            connected_pipe_set.add(pipe_key)
                            connected_node_set.add(pre_node)
                            pipe_status[pipe_key] = 1

                        
                            if pre_node!=cur.val:
                                stack.append(cur)

                        # pp('t7',min_node)
                        continue 


                            
                    stack.append(cur)
                    

                    
                # pp(9,[x.val for x in stack],[x.val for x in pre])
                    
                if (stack==pre) and (pre_pipe_status==pipe_status) and (pre_well_status==well_status):
                    
                    changed=False
                    for node in stack:
                        ind = clean_edge_II(node)
                        if ind:
                            changed=True
                    
                    if not changed:
                        break
                else:
                    pre = copy.copy(stack)
                    pre_pipe_status = copy.copy(pipe_status)
                    pre_well_status = copy.copy(well_status)

                    
            print('##########################################################################')
                    
            print("stack:",[x.val for x in stack])
#             print("connected_node_set:", connected_node_set)
#             print("connected_pipe_set:", connected_pipe_set)
                    
                       
            pipe_status = {k:v for k,v in pipe_status.items() if v}
            well_status = {k:v for k,v in well_status.items() if v}
            

#             print()
#             for x in node_dict.values():
#                 print('node:',x.val)
#                 # print('info:',x.info)
#                 print('pipes:',[x for x in x.pipes.keys()])
#                 # print('is_outer:',x.is_outer)
#                 print()


            print("well_status=",sorted([x for x in well_status.keys()]))
            print("pipe_status=",sorted([x for x in pipe_status.keys()]))
            
            
            
            pipe_cost = sum([pipe_cost_dict[k] for k,v in pipe_status.items() if v])
            node_cost = sum([well_cost_dict[k] for k,v in well_status.items() if v])

            print('node_cost:',node_cost)
            print('pipe_cost:',pipe_cost)
            return pipe_cost, node_cost,well_status,pipe_status


        
        
        
        pipe_cost_dict = {}
        for x in pipes:
            key = tuple(sorted((x[0],x[1])))
            val = x[2]
            
            if key in pipe_cost_dict:
                if pipe_cost_dict[key]>val:
                    pipe_cost_dict[key]=val
            else:
                pipe_cost_dict[key]=val

        if type(wells)==list:
            well_cost_dict = {(i+1):x for i,x in enumerate(wells)}
        elif type(wells)==dict:
            well_cost_dict = {k:v for k,v in wells.items()}
        # Base
        checked_set = set()
        node_dict = packupNode(n, wells, pipes)
        node_list = list(node_dict.values())

        pipe_cost, node_cost,well_status,pipe_status = getGroupCost(node_list, node_dict)
        group_cost = pipe_cost + node_cost
            
            
        if testing: 
            return group_cost,sorted([x for x in well_status.keys()]),sorted([x for x in pipe_status.keys()])
        else:
            return group_cost


    
    def minCostToSupplyWater(self, n, wells, pipes):
    



        def packupNode(n, well_cost, pipes):
            # Initial Node
            node_dict = {i:Node(i) for i in range(1,n+1)}

            for i in range(len(well_cost)):
                node_dict[i+1].well_cost = well_cost[i]

            # Add Edges  
            for edge in pipes:
                node_a = node_dict[edge[0]]
                node_b = node_dict[edge[1]]
                cost = edge[2]

                if node_a.val in node_b.pipes:
                    if  node_b.pipes[node_a.val]>cost:
                        node_b.add_pipe(node_a.val,cost)
                        node_a.add_pipe(node_b.val,cost)
                else:
                    node_b.add_pipe(node_a.val,cost)
                    node_a.add_pipe(node_b.val,cost)     

            return node_dict


        def disconnect(node1,node2):
            # pp(' Disconnect',node1.val,node2.val)
            if (type(node1) in (str,int)):
                node1 = node_dict[node1]
            if (type(node2) in (str,int)):
                node2 = node_dict[node2]
            node1.remove_pipe(node2)
            node2.remove_pipe(node1)

        def get_connected_group(node,checked_node_set):
            stack = deque()
            group_set = set()
            stack.append(node)

            while(stack):
                cur = stack.popleft()
                group_set.add(cur)
                checked_node_set.add(cur)
                for pipe, cost in list(cur.pipes.items()):
                    adj_node = node_dict[pipe]
                    if (cost>adj_node.well_cost and cost>cur.well_cost):
                        disconnect(adj_node,cur)
                    else:
                        if (not adj_node in group_set):
                            stack.append(adj_node)   
            return group_set


        def get_groups(node_list):
            group_list = []
            checked_node_set = set()
            for node in node_list:
                if node in checked_node_set:
                    continue
                else:
                    group = get_connected_group(node,checked_node_set)
                    group_list.append(group)
            return group_list


        # Round1 Edge to Fork (or edge)
        def get_pipe_key(node1,node2):
            if (type(node1)==str) or (type(node1)==int):
                key = tuple(sorted([node1, node2]))
            else:
                key = tuple(sorted([node1.val, node2.val]))
            return key

        def nodeGroup2dict(group,output_format='dict'):
            well_dict = {}
            pipe_dict = {}
            node_dict = {node.val:node for node in group}
            for node in group:
                well_dict[node.val]=node.well_cost
                for adj,cost in node.pipes.items():
                    key = get_pipe_key(node.val,adj)
                    pipe_dict[key]=cost

            if output_format=='dict':
                return node_dict, pipe_dict,None
            elif output_format=='list':
                wells_key_mapping = {x:i+1 for i,x in enumerate(well_dict.keys())}
                wells_key_mapping_reverse = {i:x for x,i in wells_key_mapping.items()}
                wells_list = [well_dict[wells_key_mapping_reverse[i+1]] for i in range(len(wells_key_mapping))]

                pipes = [[k[0],k[1],v] for k,v in pipe_dict.items()]
                pipe_list = [[wells_key_mapping[i],wells_key_mapping[j],x] for i,j,x in pipes]

                return wells_list,pipe_list,wells_key_mapping_reverse

            else:
                return None,None,None

            
#         # Main
#         node_dict = packupNode(n, wells, pipes)
# #         node_list = sorted(node_dict.values())

#         node_list = list(node_dict.values())
#         # import random
#         random.shuffle(node_list)

#         group_list = get_groups(node_list)

# #         return group_list
# #         group_list = [x for x in group_list if len(x)>100]
    
#         tmp_group = group_list
#         new_group = []
#         for x in tmp_group:
#             new_group+=x            
            
            
#         base = new_group

#         wells_list,pipe_list,wells_mapping = nodeGroup2dict(base,output_format='list')
#         n = len(wells_list)

#         print('n=',n)
#         print('wells_list=',wells_list)
#         print('pipe_list=',pipe_list)
    
        res,well_status,pipe_status = self.minCostToSupplyWater_helper(n, wells, pipes,True)
    
#         print('wells_mapping=',wells_mapping)
        return res
    
    

In [15]:
n = 9
wells_list = [97956, 97846, 57087, 78910, 72729, 19568, 46937, 2355, 4726]
pipe_list = [[1, 8, 18125], [1, 9, 85177], [1, 6, 33651], [1, 3, 18892], [9, 2, 95623], [2, 7, 53418], [2, 4, 31108], [8, 5, 18070]]

s = Solution()    
res = s.minCostToSupplyWater(n, wells_list, pipe_list)
print(res)



tata'#####################################################' start
cur.val: 1
well_status: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1}
tata'#####################################################' end


tata6 start
tata6 end


tata'#####################################################' start
cur.val: 2
well_status: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1}
tata'#####################################################' end


tata1.01 start
well_status: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1}
tata1.01 end


tata2.01 start
tata2.01 end


tata6.01 start
tata6.01 end


tata6.010 start
neighbor: 9
tata6.010 end


tata6.010 start
node.val: 2
connected_node_set: {1, 6, 7, 8, 9}
neighbor: 9
cost: 95623
neighbor_node.val: 9
neighbor_node.well_cost: 4726
neighbor_node.is_outer: False
min_cost: 97846
neighbor_node.pipes.keys(): dict_keys([1, 2])
well_status: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1}
pipe_status: {(1, 8): 1}
tata6.010 end


tata6.0101 

In [ ]:
213199

In [ ]:
n = 9
wells_list = [97956, 97846, 57087, 78910, 72729, 19568, 46937, 2355, 4726]
pipe_list = [[1, 8, 18125], [1, 9, 85177], [1, 6, 33651], [1, 3, 18892], [9, 2, 95623], [2, 7, 53418], [2, 4, 31108], [8, 5, 18070]]

s = Solution()    
res = s.minCostToSupplyWater(n, wells_list, pipe_list)
print(res)


In [18]:
n = 10000
wells_list = [63022,34241,79639,49828,90802,71628,76434,75970,617,83091,12290,12673,40439,44792,97197,8009,36287,19520,19994,29732,56208,8424,94868,51735,20649,49542,9821,60934,75368,85164,99235,61408,14694,10860,59478,97880,90755,68844,95705,37386,19332,3219,45002,56691,22243,21849,55319,3506,95568,96135,43184,2938,16893,52653,85662,84197,85805,88732,10200,77564,23429,56944,72830,68342,32131,28438,42282,71486,44519,32354,78255,62466,58658,97233,71811,45042,60895,73949,53986,36442,74612,66011,89884,30792,37294,97716,87393,54285,87374,37224,48095,68021,58886,13451,65878,75932,1591,23004,69056,18588,67571,81060,96182,99404,74515,47663,58608,57157,50637,86377,31387,4141,78170,50596,77532,49084,23404,86384,26832,40355,76123,13828,58057,72950,39571,87233,53623,18150,92403,61104,4702,33281,81366,1299,85886,46169,89196,62798,82159,23965,80611,89180,78312,33913,52700,67348,55180,31139,96235,56869,41097,46604,21158,46670,81098,93666,20126,74263,76197,61151,14861,89409,96140,12562,38351,69168,88614,68902,16306,51028,25115,21390,52398,50532,91964,92053,27033,28986,39244,1433,48379,51632,68426,2686,73166,16960,55584,78368,73604,71824,17230,67135,36452,1779,6427,52326,64027,8519,22509,64726,83488,84797,14596,53845,68342,42348,46183,5188,69006,92372,33530,48656,15543,76885,18176,22402,7247,22844,58276,20084,9798,15058,94693,8358,99063,75036,94887,33414,96228,45637,77953,91262,56971,43650,90582,94134,30191,80553,45986,77348,44525,29969,40970,28763,95540,95007,63339,981,44872,31722,44139,80958,50497,47194,22663,12152,67606,87657,34347,28672,51253,89298,64934,20932,19095,7422,46218,32534,5905,38673,43643,36417,60973,36033,18615,83432,36148,62585,27367,29914,59245,95128,63363,3918,64645,66769,7616,50805,90427,17973,35110,64981,38535,71714,18958,78489,27286,99015,61797,56290,63817,72262,32853,42654,77305,97798,51242,48567,29494,96149,84808,94567,86579,36016,30414,57985,24143,52494,75780,75415,66937,392,54189,89125,134,31436,46194,60843,13520,4815,72302,61111,1357,27049,66156,77896,34273,98771,41858,56458,27092,71581,39463,70118,63872,7442,43892,4575,14946,9710,25073,65138,13412,39435,29292,44001,14732,83537,91242,79542,73208,31206,49247,14926,75786,933,94408,977,12943,95975,33783,60579,1867,36264,55422,43182,21287,17425,64194,23361,22654,30711,50625,7394,14281,55834,42748,1633,85052,48918,68506,83438,31653,40774,55051,76845,1663,79375,35223,68802,48578,84564,50072,59259,71205,65692,67578,16363,16480,95629,21590,11529,75961,76765,34339,63459,85649,8634,55823,58336,99155,43453,71255,87433,98565,76957,82126,20770,24272,67539,58868,88863,16939,40596,66678,50271,48407,50543,4096,53946,47038,42300,10843,31097,80717,35006,18837,42671,74446,26963,52589,54447,7188,58220,5273,60186,52872,94263,28211,16286,41169,58588,78155,48927,14665,23282,86840,32995,81483,80478,48520,13137,73587,27911,37353,21409,31027,49422,86401,27391,76698,57348,50719,64036,74180,30343,11102,80036,16133,99308,24897,79451,92492,92074,91088,80181,75968,93133,65721,51890,42676,55782,40669,49619,12828,20582,16120,33217,60100,21045,46091,29301,99048,41352,18726,60062,13481,39018,95410,60288,5499,24781,56156,10434,68229,17219,76208,35271,17990,20101,80010,69356,83652,95286,98166,49008,77493,57930,64801,78757,28781,79576,13525,27097,62242,70295,39744,40035,40340,24718,47780,86235,1174,7510,65403,60660,60960,24627,24621,73572,59297,63707,97812,62853,84280,48338,44458,18186,85157,34421,7621,80103,87626,10311,10666,92946,12727,28517,61779,44769,63658,42579,74920,32385,31786,70978,82029,48375,6074,95393,26782,63294,80056,51385,64037,89707,48565,61928,24958,47108,33719,97836,44360,72289,22683,88134,24092,25276,82288,47839,64267,8219,64684,62650,19785,26722,42015,95387,17357,12557,50246,3461,26536,61253,15353,17923,22928,14567,6967,97612,48242,43319,34289,30566,32693,30206,21864,39465,45968,1975,26375,50543,2936,44323,29460,23000,1236,51585,5255,2509,83661,7085,69651,8039,10238,71786,83029,35340,51364,92068,4877,64274,5012,14778,72931,16898,80755,21235,38165,65629,53335,45426,24880,41904,11327,90847,83166,94513,19932,83311,53168,99023,54599,70382,1458,20440,25829,67154,15010,95618,94620,16842,29060,99978,55777,5209,76457,99686,52093,55619,4007,66680,86605,60073,52370,15655,61392,56135,86086,86014,869,33802,20636,55954,49277,33806,26488,2488,50993,87146,1042,76098,5562,5914,9252,60434,78993,90039,90513,77366,9215,68994,40218,54206,10991,37491,49617,70586,31205,487,40223,41526,87562,74377,86000,62639,32815,33551,76076,86957,92232,2227,74612,99606,12871,80565,64117,72161,58467,14141,6748,12275,56112,48631,76367,91435,96856,10290,55654,10007,99373,67118,22796,41960,98912,82234,62412,9009,98125,19862,98986,70524,42407,94750,94249,48865,27277,11695,85942,7169,10600,39860,49456,56857,90363,50891,29454,27004,8255,11379,47006,21380,72658,555,28912,59616,90928,26776,444,98850,1941,92954,60230,96687,18491,84014,51019,431,80531,98864,49623,603,31092,67858,89511,3208,90973,14333,16738,44228,13691,65398,53403,59388,24194,84101,66442,94952,42892,67060,62115,10633,4867,71592,72459,83686,64366,2039,22539,94818,37102,93580,26556,82790,21861,74569,57744,89894,49510,22479,61055,2270,27700,48249,72883,63189,47182,33105,52831,63878,86727,33214,16527,37963,62641,2324,54143,18529,61388,37530,91639,19395,86733,8137,9461,23451,94557,92863,96413,83792,1248,20258,30384,20428,30153,70479,3355,12260,47850,64783,78121,85060,12558,55804,56596,55298,52401,92556,53407,46297,74989,47147,76300,89636,2542,18002,73178,20505,51612,97420,27210,95520,93277,16954,84354,76608,62593,7065,34123,98289,79706,38394,93205,11063,46556,58309,51441,69564,47121,866,81637,70211,45793,6997,42377,85653,22415,69765,49973,68964,11723,18349,24296,47552,94676,48519,21808,13017,67892,83081,76626,23813,91127,48139,4703,96653,20428,59856,22976,40025,18343,56917,34485,76349,90901,97218,6800,44614,83037,62067,96337,98089,11967,42796,664,64966,21438,48605,59172,37030,11763,63344,21047,44273,75463,15363,4954,78460,15396,56037,97479,2711,26361,24533,79565,95148,99910,64573,50020,64331,55417,78730,70570,97098,22603,52744,51926,14372,49375,96703,75736,62395,81785,83945,35144,67472,83601,3511,93573,94808,15353,3230,53479,95680,90595,43066,93787,15495,22467,8890,8698,15115,81266,14436,64670,17307,30795,37180,12823,67430,87061,24592,16639,55906,24641,85142,44439,26932,41720,79500,47030,5466,79529,72742,42991,56265,69151,53006,75191,92588,2916,30357,90929,70603,60044,81177,71244,19344,16889,87206,79567,56056,5521,37844,40387,46206,45982,45771,12692,44337,82260,29433,30692,28476,69565,83044,74370,60070,50720,72255,4846,10494,13158,47812,33368,64376,77335,7797,58146,34606,13910,59183,95639,92205,78832,70191,46011,29874,98139,94085,24378,41,89344,90189,78473,53797,5842,97216,77334,18483,96001,78497,88528,75742,11917,240,89927,366,10793,26499,17040,74947,43802,56107,66564,64259,29594,24429,72667,58664,23778,91129,76332,70350,2578,80924,88526,66951,85950,75079,50306,46978,67726,85193,66489,92087,12578,29990,39826,20102,90923,80391,89019,61724,33075,21158,99366,37906,31951,21393,61210,48427,2997,41359,8636,80722,86599,35483,55030,75961,73963,69367,90532,24441,4994,27333,2038,42386,99193,92382,16973,44913,33655,67787,45567,16046,68505,72608,47125,49095,372,70106,27105,38673,2355,33214,4334,28214,10735,8733,57888,94749,2621,8207,76323,32830,22401,91079,7166,65763,35009,30914,20910,62678,58435,65007,60591,58571,88199,48954,73058,57098,30188,17520,76200,86196,37766,3429,47497,35029,18293,22702,42925,18544,35333,88898,32144,40031,69178,80565,46852,19000,10782,86497,83714,83213,72379,17075,69326,67341,88225,19204,51279,82722,75756,99271,74196,89122,7,1630,9754,92903,93039,13599,28487,74930,35987,81405,62997,85314,79307,28631,76404,55454,42785,37798,78655,73978,87477,76868,48361,42128,65867,79592,47719,89099,99605,62189,93316,34180,35966,31124,72298,51120,65998,53877,85354,66843,63919,51932,40454,30444,54732,47349,50676,29794,4231,75453,38083,1749,80738,43677,23576,86832,28531,99374,96315,87051,52339,15324,56601,53648,55245,29280,86281,10484,57931,1546,23891,52867,10214,88431,9818,33919,83664,98554,12433,12377,7993,17792,10556,71155,63470,11463,9479,79328,88036,43631,68708,37287,16885,17493,41333,13317,60373,81091,97157,16117,13259,65533,47835,53259,55702,70028,90433,37042,20362,54994,7508,37241,85944,87941,22518,41774,43212,39279,55216,58709,13314,17278,42980,4844,42582,28524,45343,87785,7064,21115,91346,15963,46934,9129,30266,62039,34439,16657,85178,35870,69075,46297,16473,32317,31040,70178,5636,45111,79779,75449,80520,63325,64843,57623,37882,47441,62150,30779,63249,8278,29471,72113,26527,57031,4088,72059,21044,85427,71703,66447,23478,18328,33419,85095,77300,2832,85725,51570,87727,10307,50087,16824,14219,77346,45060,48534,38225,5498,66517,86890,42145,67803,16235,78885,7543,27559,38711,98597,55685,6011,64915,25674,20371,60722,54468,45892,31261,91426,58828,97736,20363,80286,1360,95380,49136,68521,92049,59848,64674,17174,94552,96192,13991,49455,14284,57533,3565,77271,51287,94051,6036,39040,45156,35534,54425,24210,57896,71802,53101,52941,22089,91143,21383,16499,67433,89059,58529,81150,83604,68607,28379,65027,86436,52243,73421,51021,5103,2565,74982,40694,59962,83177,78045,21756,47476,34,25984,31881,67337,17894,59403,87854,19442,12811,91330,74444,26958,53727,89075,43450,90833,3859,14505,29236,81346,21316,59041,29636,59614,98132,66432,54590,35347,36916,97895,82869,40597,63380,57755,31298,27515,56088,95471,98171,52466,53389,47897,26704,63456,26454,80189,55732,58038,25697,93613,85244,60259,50346,19575,85196,73706,33681,3957,71085,11823,90122,93481,15551,31043,76496,14863,40733,22498,18334,3003,28858,92021,51652,42703,4103,85444,19396,67632,87046,23417,93629,72776,54687,16456,5835,69032,67380,64356,79633,93486,41759,3944,78862,96347,85039,23788,41560,7785,85036,12552,30700,2131,80550,75303,75732,60216,64552,43238,31135,83875,43408,15343,20941,41935,153,57589,65515,19889,62505,93494,255,90742,53288,61035,53306,91243,39202,4320,94213,56250,28627,9650,70279,58219,34579,14095,40111,99508,7021,38417,83806,69767,57538,92457,14439,60587,13696,63001,42918,24590,56258,53993,29605,83944,43379,80217,52551,53136,62449,88124,81590,83561,78525,43624,32226,5128,55072,3051,92344,16484,80527,72529,56446,15587,43258,79535,60056,6358,17528,82960,17716,6937,61675,4218,11540,19916,33587,69507,57645,93829,91253,6181,92459,83336,66251,77802,10196,63118,32932,68025,40618,70673,14938,27804,7902,88998,44094,94955,74252,10173,76112,14881,68303,96988,36067,34506,4952,97956,94709,85513,37717,74898,6173,67581,70167,78916,61475,35531,23296,94401,42066,9867,56821,69786,226,1590,56534,64984,80165,2355,75961,89647,59880,16290,25651,30701,67768,51230,48456,98780,8404,63639,47765,94505,68855,33953,43648,68039,29902,50280,23645,91792,61319,53037,85198,21903,41290,46378,41189,62359,18246,73011,96749,10264,83906,85940,74642,36362,74707,99548,97920,21138,49376,80344,51206,35789,98250,66235,4809,9981,81779,11056,71801,79644,82273,34157,7838,91211,20911,94163,41104,51645,59343,30092,10550,35127,22706,9725,80138,87816,31837,90647,21005,21882,79388,96576,7102,34791,42030,23182,36150,80358,37892,51141,79286,13601,66697,9021,57738,58232,84381,39221,53170,10531,22043,31948,13498,75621,33119,54987,69414,85497,85460,11321,94011,68915,34405,85071,37471,42016,77493,8459,46000,71933,7863,95289,38210,40847,48355,83253,7337,21381,65425,40963,44230,64658,27093,49214,83446,57385,12355,43605,18092,9891,50902,93967,44824,51519,89260,47097,91480,74617,2419,50197,11878,9336,34593,75644,61405,62338,79834,92178,16057,70425,34268,17273,85054,88379,19478,94119,30053,89893,91085,51796,45751,18904,57132,18646,10181,296,52898,87682,9378,85422,83795,58151,75097,49053,94113,87555,19551,41462,10275,24524,4609,72239,75636,42518,54212,89349,14383,2375,3230,10455,98992,34194,69504,11936,35883,23224,74690,49632,95475,62642,32550,69804,62447,9810,11301,30149,67209,5211,60408,76902,78069,20099,30705,38827,91508,83637,15350,98117,43782,92787,29496,16177,61802,5293,91605,64833,25636,10876,82772,84369,64507,48413,97789,77954,69469,52864,97839,35691,74249,48022,80304,46803,53069,58518,68321,56328,72631,14382,85497,9511,29737,41233,47102,70433,84636,81878,25742,37706,31445,49609,67069,53189,90561,2269,27845,96633,95788,17936,35934,14172,19616,20933,68039,38331,40396,98122,42769,66261,36733,13945,90877,69856,44121,47674,67562,58394,11506,26940,84967,58354,90787,71209,40938,43087,6737,51045,6861,40485,83961,19322,77315,54661,56279,64459,82319,72457,98645,1733,39706,48672,38162,56516,19430,12402,87355,94359,45198,7735,65926,70599,18609,35031,33172,9388,86720,11146,75326,35615,92921,85260,49380,59170,38383,58765,33665,94728,32685,39447,2440,32828,56541,76699,35894,37140,30211,66555,99717,34384,6257,84848,17915,83267,16494,48868,73776,84461,42784,86191,96663,1853,36182,1986,28769,68296,7942,40895,88363,10767,51506,29097,58312,56662,81652,26401,21525,28932,30040,50060,69874,17446,11004,6492,51843,5165,29565,76821,31757,82170,31936,46428,70811,96747,13549,63456,12008,78564,31517,91672,19178,26590,27035,22431,16217,55344,56412,20707,33688,65028,61881,99744,56253,97358,52495,1543,27932,89656,17354,18999,15885,55200,70527,48276,77122,40448,8613,26813,15197,87774,62439,5427,83831,47535,4644,76617,39057,56755,73860,74203,45353,72381,40858,36465,88623,38889,64528,95410,88230,8129,82331,11547,67482,90345,97080,12068,92055,65039,527,4577,39217,97077,54116,86806,21935,41529,98319,21569,49176,7823,45619,55988,8040,55943,23189,66608,80352,55037,4625,56035,31293,15451,60118,67204,40648,34638,72349,64114,80028,27875,68566,67184,62245,13440,38352,86370,49303,17817,80828,20739,44218,15170,41253,23895,55465,78328,12902,60218,73841,187,53927,72328,80635,46194,97646,63509,53168,95426,27890,87877,5676,7042,31477,77990,57080,92456,63479,26112,8217,83841,28124,38024,13842,15494,98440,11366,6508,76940,9306,6681,63352,85864,5122,72929,23885,2138,91778,48452,20372,49655,84525,38360,73175,30497,61157,47392,56972,62866,67225,29558,28774,14502,55317,50040,17559,91593,38244,24142,88861,56269,72474,3432,24092,92749,92138,62486,94978,54009,13330,80204,69069,71135,73677,55760,11869,1370,89037,81519,41088,76886,82445,63613,71150,35179,52573,21033,99048,91772,66747,73002,79028,94954,11950,54932,70617,48954,86568,28711,126,60724,40306,51665,57578,4853,81004,4363,58236,89475,68286,79090,88183,70191,82805,33755,63372,20725,2627,25327,9306,21189,20107,81074,61428,10891,25001,40411,25997,36045,84172,89273,50437,77184,79512,10805,53693,2298,1660,4663,45012,67227,72640,45745,57665,26015,21470,79479,58822,49296,13908,36960,61,53818,83021,46362,52115,25000,6831,25475,42697,21952,46618,40081,59251,97592,7314,99435,56045,12070,66018,72527,1446,57961,39149,82248,3075,77841,8537,61369,25644,81850,32601,51353,68374,3774,24689,92258,14634,50402,94281,57088,47121,98900,1650,31934,66142,79526,59451,45807,17369,82287,83865,2838,10077,47727,58999,1696,11472,62877,85292,3059,4417,38031,94618,34321,38932,43783,29044,76578,2580,56519,65428,61995,58149,81300,79988,42426,98745,81870,93240,85047,79052,57201,69799,86610,69460,11543,80128,77094,74159,5902,47502,77983,30021,51054,64397,36176,14058,70565,11157,41197,16202,58467,77283,27919,23131,49682,88261,31572,70121,72547,49666,68040,40802,90169,71641,24467,26266,6839,40790,77406,99387,72911,76373,72787,16490,81656,14607,7827,64603,60872,39464,27032,77311,15629,14915,88820,7182,47813,4726,51250,399,58383,43868,24058,38498,45770,14226,49756,492,15505,1917,4567,45007,35557,42944,46957,61988,10561,64675,58197,6587,72842,74285,96548,99065,7761,74508,27041,4548,48581,77317,8934,94745,47043,55922,24891,84019,9886,47268,76396,65974,10152,40550,72788,90432,9069,25313,64320,9643,79618,62766,92323,157,68630,37865,78478,42580,71750,55675,15821,57643,23987,54329,72186,10798,95791,21654,33742,47977,51132,86262,522,32346,59073,17205,83077,85733,81620,56830,85078,7165,73695,296,84562,45673,54386,18484,47085,2398,96615,53128,99989,60465,62560,52178,15496,83062,35293,78058,9446,13889,47828,81863,35107,63688,56501,17736,32400,15492,94488,70005,20881,21264,69220,94139,86908,35704,40468,5030,93569,87827,51657,7759,5504,26050,96048,27270,19938,92677,94367,25834,1279,9328,67778,90639,49241,62713,934,42156,21113,91149,94543,50096,54545,72700,19568,71337,56876,49386,40680,26022,6094,81118,8698,88237,80029,79219,14047,76164,62700,16372,72147,81351,70010,173,62891,49782,40768,38438,17830,55109,34323,19792,59170,67906,76275,69606,43688,37089,27203,85464,64885,85007,13472,42882,19198,73937,80454,40417,76920,69316,87443,77401,71442,6223,55474,19532,81313,93779,94469,98430,34053,4486,49658,46034,84262,66321,21731,43307,6036,55007,39674,98845,29472,79637,27659,30236,49723,9867,92505,42540,44696,88820,21124,69808,84134,83406,59061,72558,81497,7663,75715,40494,87881,90246,5396,3912,63840,82438,5918,86592,24711,25770,22873,26739,46974,16055,68484,2382,2254,35454,25728,98436,73100,773,46083,55232,40216,66715,6884,14340,40948,65092,59900,64428,73666,12825,26579,39859,22804,33275,85924,10442,50511,15312,60494,27441,28627,95615,32838,35376,39150,17396,73783,20553,47957,19709,55497,66260,72339,55597,5030,43608,97533,22335,57954,82051,65072,58849,37001,81442,35178,92647,88758,10642,54154,1562,59456,68449,84566,82853,68914,35763,89618,6245,9757,97064,67983,99305,68762,56768,8099,75000,1960,42487,88512,2213,58952,4894,32989,35265,58081,46107,16224,98388,14834,12780,34584,344,33507,27668,39554,95104,11947,15756,82733,77264,21730,45923,89813,27157,5244,64642,77219,15120,55480,6273,98810,71698,13426,57065,73733,50102,77055,88414,26911,61520,41683,63490,30809,29975,3754,76087,20550,57240,26597,52136,13828,13135,21921,58584,56651,40122,8330,78457,77263,21945,59570,17607,17486,61412,37407,30025,79084,81626,60130,6380,25889,24572,65009,83508,3524,35377,66967,24833,96517,37139,42962,9891,4091,29586,35846,92741,8277,67006,56699,73460,31851,53342,41147,62510,6920,69913,33255,55327,87953,23575,42134,59032,16705,87578,27028,77757,80869,49787,54371,78060,35635,43183,5919,87947,67283,57822,87838,58182,85681,95429,2042,64579,51368,69218,65493,18847,80788,37290,18823,86567,89384,90340,93366,96767,16618,98799,70706,50941,90066,21204,73822,28973,56287,73946,63774,67372,82510,91659,46504,39884,15894,17355,58939,1165,39703,28404,83939,20779,1030,70952,88480,77075,17515,72313,71612,38270,11325,6862,79256,62877,97235,60924,70939,88089,60197,88010,71184,99858,74726,40111,7374,48900,76988,85449,12176,64348,54685,81652,26115,92585,92819,54796,41996,72984,48532,73417,64930,49797,53569,38749,5763,47475,54820,67099,41423,44302,53568,89127,53240,39637,60217,61402,9052,77303,23090,22974,88270,23648,98976,9144,71762,94659,14821,96399,89367,88082,72159,39784,59508,14075,69939,84444,58540,43506,55633,11399,81632,44173,71718,93690,46469,57081,67845,66530,50825,62806,35606,88212,8646,95763,77175,21007,18851,73727,58440,16791,49349,45301,93459,2774,74330,31157,88229,52750,23235,35039,44812,62245,3730,37200,19615,61255,46519,58902,93087,96519,46456,27918,18993,52613,44830,77610,68864,57299,76915,43811,65945,85025,44221,44061,58462,96433,78061,38249,12383,26756,80057,74199,71034,6484,26403,92640,88509,94374,22584,39930,12728,67284,47677,49458,27349,37323,1407,50905,87442,94456,64958,47010,59505,93553,78865,15045,94150,98228,39506,73036,20060,68118,36489,48790,60843,51668,98067,45103,66170,66822,72545,90031,88485,94144,73416,78810,52196,8281,92732,39396,53495,31009,91056,10445,46695,36707,62215,65424,36314,37948,39593,18184,53365,52967,80533,15672,80533,25662,77453,81527,21065,77527,18989,18699,30049,91978,29905,93086,97478,58048,90926,86700,89153,48228,53105,79752,48643,34499,22348,63977,23680,87414,4945,34272,44145,50172,23543,76438,2471,70364,56056,45591,13423,51609,12829,10455,52912,15531,16830,90159,89352,44708,18865,45387,4032,31293,13524,87248,78418,70349,46338,10439,1750,31792,17524,23797,26241,43458,96720,68046,15894,1729,49833,44633,95253,80637,86075,38762,76378,88763,19758,97185,2279,49075,73624,30756,12255,147,31311,22952,18633,26111,95625,1673,89688,31579,10730,20253,89527,49811,40297,83805,92462,24454,8465,45562,58381,2054,76899,69890,21518,65207,89053,68409,71590,5527,40904,48539,34136,81701,13217,29439,75219,65119,10288,35981,92169,96801,83891,28562,93423,4016,37666,4691,24779,16567,76774,21857,46691,53261,69195,31529,86164,96900,78715,6923,52036,78178,48835,31230,39879,3155,72990,8861,92053,94882,43436,9117,17851,63221,41605,62182,1544,74282,90732,9721,62167,6087,31108,4177,92492,94292,36642,64525,6294,81517,41032,43576,35751,13215,804,87914,57264,20915,52204,47109,383,58241,4045,84141,50534,90121,8565,72814,13623,60452,92216,53121,2820,31495,49370,2783,12413,18047,27515,70028,63938,69428,25338,15423,56658,74883,67274,11295,47696,78389,85500,74423,6228,30488,71346,91771,48729,55397,98639,85171,20095,79836,14689,72362,279,75144,44020,52654,74128,27716,69115,29369,54555,91380,80664,80212,5601,7889,35009,75397,16232,27444,18028,30728,98558,62544,30210,3119,46614,29831,32025,18234,22302,16510,33904,53671,60494,28626,78453,17005,67614,19770,46685,23435,96723,1017,67677,83605,75882,60816,17243,88995,98746,96836,25381,85682,86267,23987,4437,93034,72499,65636,82882,55433,38665,86074,58572,29994,67788,8330,43307,94881,31196,1526,53648,47691,38093,72827,75115,68273,53375,64209,30641,91033,9575,64631,76845,73546,22097,91598,18827,1152,49513,9344,45395,35611,71350,65609,93773,21827,92495,84797,10368,91716,63497,28831,5984,74255,94431,36592,96708,22170,67962,29414,39928,84064,91073,18634,46824,57635,17597,90490,14055,82017,11782,44560,2919,99630,41628,42093,11921,48967,46716,30627,11761,40707,92718,99002,38384,35067,98761,34653,58280,66160,53794,57023,89494,32989,17996,26528,97493,53528,500,19739,16264,23383,37467,55266,88107,45817,15800,39972,67264,11478,40427,77627,74901,92783,21001,58713,4896,16369,70198,37520,62266,35195,21796,46262,20836,71923,35941,82515,16902,52774,53202,50821,47040,54646,10804,82909,17570,35993,46664,85967,61448,36954,12615,69553,80228,91051,58746,53697,64406,68010,52534,17360,63236,21601,93747,54529,19960,93772,41615,88080,50658,90504,13440,56011,54867,98557,90105,46410,38291,80608,12611,25471,14608,13021,20781,40825,84989,20694,95605,88155,8519,77448,39766,12084,4398,62251,50153,55699,62554,17515,8085,61103,12736,384,7883,97775,6270,9435,99595,74627,67731,23248,20923,84480,28161,52448,33510,66940,29942,27112,13328,20316,3247,70570,52261,12616,9967,3855,50608,83371,84955,20477,94299,288,3484,23477,57641,98906,55286,75427,43821,23526,10436,50508,42012,32444,45875,64822,49764,15888,60301,89435,54436,56819,52405,4036,83741,61895,36960,35502,95391,65100,59625,19392,13574,56144,66797,5934,41706,23355,17762,24801,6943,25485,11339,78229,34235,8629,7449,18280,56437,90671,37833,63293,48290,2457,26058,4668,69993,53305,65225,81074,16028,57037,49828,88947,30004,25801,18734,32481,75781,2573,90411,85017,18730,60112,51448,75595,93930,28771,23916,78776,17905,69498,29509,86686,41631,90031,71851,75081,35583,48090,36004,34730,82763,225,22621,37343,24001,85880,42278,48684,13298,63245,54236,29657,24601,16719,87363,13903,15685,56792,44452,25948,36414,92096,3202,33915,73630,48434,33383,62865,67461,36202,46920,99129,76205,79310,6432,39175,74969,70409,62302,1939,23918,50249,88251,44947,75144,81987,40974,73162,34372,62990,60971,76107,6118,13141,57026,83327,95853,27120,52927,90239,24640,5239,12539,55873,16812,75997,55573,42246,89382,26356,61841,50617,68465,11143,40501,82743,20782,13102,93721,97461,60793,87746,81482,27892,49391,10028,63508,16611,3461,89362,35716,39567,70625,90185,59086,14905,41445,71563,4978,44292,61836,31587,27272,10388,43976,25073,9371,67731,23355,98194,68234,25644,48334,21172,71707,15000,35901,14842,74800,44338,28551,92303,47712,18584,51234,29161,17419,73785,51792,37842,96654,24421,5520,71951,26507,6673,31897,76137,71366,62997,85198,86167,27481,32351,36677,75429,77415,57343,37824,24957,80086,83201,47226,56713,6582,4624,53656,82837,7116,80857,12937,63857,7670,11229,91806,86428,8805,69439,11383,44164,56962,51145,82729,20563,5387,51776,91159,90507,70929,62192,96840,5220,43690,24847,56093,55093,60560,78039,51423,93497,97052,72972,88058,90050,9072,97950,69990,90074,47512,61520,47029,51454,97846,77851,44560,9428,76239,39674,54597,93811,3805,7691,64048,36129,75721,15409,52661,29293,4049,12283,43004,99062,5815,67951,74514,8234,20505,54372,52805,97702,31309,48795,16446,26320,25792,73622,8599,26810,9315,5132,52186,30118,85703,14815,75026,21956,10263,84197,82589,1185,47874,33974,22908,61874,74981,78225,20201,28234,23814,35985,49388,84308,69874,12773,35075,14860,21217,54102,95265,4752,79541,72847,7249,52970,41997,44001,80694,300,1190,85700,37930,90055,48662,4961,25727,80316,89465,25320,21562,77550,46912,53206,56736,68663,52282,58238,6268,85188,87439,87135,13007,12670,86099,31231,81711,90638,49910,42536,71780,56211,73232,65316,80765,1209,57636,26184,92344,80078,90983,90522,93529,59949,47643,82807,30199,24768,50327,60856,61895,76177,92736,32253,94001,43899,42064,83530,89453,53357,44540,49755,85619,23733,79053,42479,57885,65386,49619,8632,18867,30671,73743,8391,30507,20217,70911,42585,15126,16981,33071,32123,68680,64777,95175,93460,48082,19747,98524,39169,57802,42546,22639,60706,62578,64462,29413,19742,56173,68096,69592,27626,12228,4342,71262,11992,73488,5030,94461,16580,96398,32976,43589,96162,10484,25816,1968,93129,9057,8396,84636,13901,33381,3841,45756,28020,25511,47630,23563,13772,96332,63807,39688,65633,90687,74183,46615,53085,59306,95850,24147,40305,91938,40389,81061,33211,26014,62752,13758,55686,83750,58195,35733,45216,86292,95686,19991,93671,74871,20648,94640,27406,74088,98383,46333,65723,57786,69661,19888,31015,10369,34334,1582,90994,61624,12266,77238,10561,72688,84917,79074,89516,62789,93042,81259,49189,24077,56692,98815,55926,75663,32399,3088,53467,21957,71132,98823,60661,49267,96740,22468,2841,83645,90148,45260,86068,65768,74602,51958,31493,34553,68011,16048,53799,59036,50575,7829,38784,10075,62408,84831,40699,33609,52295,81737,38208,98923,40966,67599,21463,98961,28426,85346,14553,57488,19323,93725,57596,33396,97372,5296,22911,94224,6229,97183,8017,34170,1621,77515,99600,93228,71272,10639,28857,61197,34205,23265,88566,46807,40520,5141,80490,50240,54685,33053,32119,18772,88281,70766,31254,18396,58411,57345,11023,81416,16548,11739,11697,5308,115,39321,99612,4423,30896,97299,49139,9436,27456,40111,34779,74996,43633,32718,18627,49098,64876,6932,38832,89796,97715,98792,17727,53513,55361,34914,20085,93366,47553,31109,65955,79538,39515,36357,92654,30052,55281,84159,10306,91762,35368,58849,73713,2538,63789,2014,82442,36697,2665,16314,46410,24069,68044,75193,18302,47738,10779,94421,23956,57092,43093,90609,23258,81563,70368,79739,30500,21576,15352,54913,80887,7238,31699,23390,1071,23967,78241,27644,28058,10037,79313,69858,11744,19397,12217,39310,15943,61763,53800,49882,22891,98234,5250,46282,91397,56375,57558,80369,44831,31506,38192,94174,13316,80200,21464,68955,89749,41329,37121,45987,73827,22569,241,45623,37373,73024,91917,8655,3053,29492,71596,22933,76141,43998,14686,81326,33224,94561,13052,68164,59038,47102,78124,18916,93833,95166,41591,96932,9412,5384,92966,24130,35649,39416,86989,86894,22706,70647,62683,61671,77887,44613,26502,32195,5920,86604,66441,33097,74978,50271,69546,24102,86532,67034,97040,29608,15096,30547,86829,62634,93074,60816,45336,1321,26001,19031,74428,18661,56987,33504,14223,96021,98997,11408,6216,87193,65037,40321,12334,18490,63268,34612,73567,86271,67173,91727,8805,23300,48139,91169,44763,61333,94813,55077,72730,74328,59519,93149,56291,56628,34994,20612,42279,5836,44126,53333,31341,12529,14170,15527,36854,68899,632,79536,74362,32895,542,62002,21511,33588,30730,27744,62962,48305,18815,70366,74219,57849,59474,80136,66659,13069,25944,29950,48791,85381,72656,71831,5312,47363,16124,54292,91547,74542,99148,90538,6433,50311,55136,87554,46913,26870,74833,97139,83771,84289,87664,64883,12628,61264,37227,84505,52606,6163,18980,19699,35225,99334,19466,22614,53802,99426,94275,77198,33381,79293,59716,59462,92106,70736,36655,85755,53245,8386,3445,11630,73277,31408,79437,39767,18596,55039,3603,42616,17672,73263,45716,82687,62107,28595,30752,30580,64519,27993,18751,27481,37083,86379,50361,81795,56461,50752,6256,77282,20577,95916,92977,31032,37808,90018,98833,28774,32879,87668,67297,40537,85896,4891,91209,23835,99048,56977,57554,15654,15553,90012,16415,31083,6805,64877,73849,96593,16234,89688,77589,32930,2181,17756,34750,69206,52620,46262,97981,46866,56867,46526,46463,25386,67844,55187,53577,81009,84552,12593,83568,83093,34021,82380,72887,78316,23434,25068,30129,47080,84031,16701,21311,75238,41761,14211,31527,88518,62001,44380,28611,3953,19518,10434,77751,74117,54001,81788,28351,66953,94995,12773,2392,56779,8068,44772,64532,65565,18237,30432,52114,90782,81882,95885,41667,38475,42880,32182,93926,79667,75663,93084,97838,80178,32716,47981,52013,96908,69521,61278,60525,3866,9278,54472,23771,83856,16329,88744,44149,52334,47844,43709,75125,27714,44701,58707,2019,67434,75018,59004,79740,53312,25544,79047,50715,5180,98938,40346,56243,43468,38421,42250,6338,52158,37670,43388,67231,75976,55662,57253,88798,46136,31830,79025,76053,18576,37168,98498,28054,88534,63004,9532,75069,17010,59861,26870,9085,51787,58561,25983,90339,33671,95093,88433,1684,30750,62433,89277,3237,5433,29921,45267,53580,56818,4232,82926,18687,18389,65246,87102,93177,73151,8311,2731,25082,99244,42661,28968,5715,58580,27323,22616,97362,94141,67342,82836,97750,29985,71519,92314,6200,57583,24106,43438,40976,98229,26455,81605,16448,70564,84507,3779,84191,49411,59590,94672,6590,16299,55938,97475,58708,48147,67751,70239,68232,93288,90286,89826,15139,21416,80294,56815,47163,40587,63391,25803,38342,25654,41121,86055,78484,96739,28484,63031,42848,27883,56520,50109,59578,22372,62101,31651,45176,62841,64644,82249,27583,62110,78728,94141,95319,83844,30813,9951,36175,45457,62543,59048,24771,10367,12710,87671,74796,86007,84104,8521,10066,80526,96411,72657,19146,52425,5314,62341,828,21197,2001,42094,54802,15994,76566,33768,83009,2480,72312,13735,19942,58632,79559,14279,33212,69486,61964,61113,41600,22227,44648,24578,16444,1897,94948,3100,73181,45611,25553,13375,34330,7788,16446,82741,8504,50946,69358,46470,17222,89599,87166,1532,94903,25356,68523,70291,41536,83830,85062,13551,51893,73578,14435,38169,15313,6948,39556,76689,62925,41666,78789,43196,99150,58887,1386,6287,95497,48998,71752,46404,89588,2046,37514,9500,55900,85824,8019,47946,82584,21835,12591,31278,18529,63114,67784,16614,60078,32420,38486,5724,53891,12997,2677,66624,62072,57813,55486,58539,89807,77385,58458,81324,62610,44038,46819,21163,79475,52343,40807,39580,38064,8888,6720,31283,61904,30534,83481,29434,31263,83607,10183,6504,95245,65974,66540,63075,51464,70070,16126,92421,99309,80664,3808,73629,57198,94796,28143,52930,54184,13077,26859,5753,30245,258,36507,91583,69230,84997,16372,24599,68553,57766,39887,60397,15830,48834,73971,58142,49361,40476,23665,756,14546,49597,47635,94507,93036,81488,73763,85002,68733,76743,74481,92077,12426,28420,15704,97846,52153,55113,57065,46579,68040,30520,1410,27605,69836,45021,5968,5127,58637,85053,93472,21533,80307,13550,9722,82873,3286,93697,37039,74069,23451,68456,82723,16306,58338,51869,50888,70337,40993,77274,54490,74962,8265,76928,68368,94466,78670,13503,33612,60733,47220,88928,99190,40277,47775,1316,26931,73700,55469,2218,14523,73593,80849,65820,26651,69878,9366,70083,41738,87019,79939,70326,29738,7593,86151,17747,49194,81816,8981,65713,93487,31520,76583,30137,63073,54628,45339,52466,35297,62821,17788,92425,27551,72650,60748,57138,18655,56584,39280,9865,4493,42302,10618,20457,44551,52840,15252,64585,14159,97738,38667,57440,39099,25580,21887,36782,85107,60982,52806,23974,87160,95069,96306,85953,31634,11858,39310,61818,33905,17681,56201,29691,23812,40407,34936,40077,71793,29507,83702,28076,17155,59602,35712,30322,58444,85215,62954,24563,72017,80939,69811,55769,55817,25746,87540,92366,78499,30227,17432,4324,91063,11751,28649,33335,92016,97850,3600,69053,82894,15362,44506,68488,90910,62658,58502,3953,42259,33214,47096,40812,93030,2082,24510,87924,36312,39054,35079,47785,85396,82877,73473,79892,93523,82559,53097,95071,75228,44873,85397,19855,91699,63344,18745,12452,57051,26697,37666,17645,12211,92303,45680,90797,89424,82419,28806,50879,57654,31079,43572,88011,83960,96990,2348,40742,13568,76989,22770,36326,19719,75852,46657,16319,68534,72139,70635,54545,62015,45687,68328,76188,40787,74115,18745,30236,36293,51569,76424,68853,97829,30503,92580,54248,53947,67468,41024,17400,35345,91392,46723,99359,44676,37984,52898,26167,72890,34256,18229,9094,82209,82914,48586,96997,68252,80347,21026,13609,54932,25225,84355,20269,84850,38943,26492,7413,1394,88318,56084,13525,34111,48884,92012,47375,25018,70642,84349,47630,14449,85036,5596,98476,86550,56124,63879,99178,84965,80508,17661,46033,52581,95228,52859,87635,30722,26995,8032,63163,6637,65146,69499,10876,73418,23632,96224,7004,35203,93728,98962,12184,49703,75444,95173,31603,63194,72756,76383,48338,80801,67267,95109,42141,34549,10150,23797,11859,29606,99137,90255,743,56607,73169,27237,15183,48784,75956,94906,175,80466,26064,88581,69437,18902,86021,42276,11726,63673,71889,66677,23640,54094,66555,56396,13476,96032,31877,69878,77702,13846,72984,80178,68728,83739,26523,73512,34975,90872,54049,62892,83747,29995,43465,77519,45458,97372,98899,44161,49782,42253,41069,76794,35375,40095,47053,55051,62387,44847,62043,85344,552,70479,23239,64924,28887,66970,30772,76141,18342,67983,21077,63193,60478,18443,8234,94461,98427,74660,31250,63497,28502,63091,84889,3404,71059,76120,35799,42489,58197,46908,72536,59481,8884,87199,82402,89622,28967,32021,28137,84948,40323,51649,11537,83925,55642,41465,24406,41248,9917,73935,20987,90656,46056,52777,44204,75838,90996,43736,51688,96499,64063,62679,61118,18281,25642,76374,11399,79623,57727,60943,25448,24222,4702,21969,6277,78143,98687,10869,75248,68225,74415,82783,51650,61747,66373,31614,78942,9129,66308,52811,32248,68293,26865,1150,24684,19465,8046,53363,35819,21782,77583,38268,90843,56623,85042,73939,81400,68367,67555,93463,14015,15230,91919,4451,3137,78153,38768,15074,48690,8723,98637,27050,20794,86985,11031,4277,7429,32275,11991,66529,84131,49149,24955,3302,46801,43725,61932,81489,1574,71171,18763,69531,8971,27635,2822,83645,79488,67311,47994,42245,55915,96343,7078,31185,17759,55054,16705,1224,63427,86583,59131,9973,77558,78402,91875,156,63469,66125,85549,72024,37196,35576,61465,76083,10558,5893,34108,57600,83682,96805,28362,18026,23170,9681,82905,31947,52117,26135,21078,14760,74513,17218,88102,89616,98099,78246,96546,93476,65913,72876,29239,47209,51648,35837,30825,14848,14822,80391,54171,22518,15573,182,41027,76438,73840,17219,6298,14687,23610,18891,66714,44853,34871,10743,94322,48548,43929,96784,62112,57153,35500,97189,65253,21935,29625,69700,29730,24767,57220,31701,56100,84366,79649,64362,9677,79620,4998,4733,17353,39344,53150,43718,81968,53499,57801,95641,48363,87321,45343,32197,44469,68491,45905,64692,94263,96748,72458,50306,47975,79371,42174,97840,99266,58256,87909,35996,73642,46177,32086,8356,56940,45454,33111,3487,8903,69834,936,45287,85214,17021,6979,65013,69798,88355,74093,84445,26539,61788,96588,77873,71612,56434,30385,59004,57087,94584,8264,52181,36810,90811,70386,82846,13580,3224,23637,3368,56612,59219,84186,90702,641,81292,80459,2704,78764,88298,70736,55470,24649,90735,18328,80518,45388,13843,6413,59109,75513,40379,56741,82286,53144,8920,12042,56097,55811,96976,78764,481,72430,95345,17286,17581,91495,91118,35911,43758,18694,40777,8282,30038,52828,93220,78972,66482,17062,48632,1557,16656,87829,11746,70059,20582,68090,24742,1546,2263,6730,32414,50230,78860,84583,56607,60989,11340,33650,76582,21875,27724,25301,87466,87888,39040,175,82032,11195,54393,18794,53399,77221,17031,374,98521,87216,68763,25054,8350,84067,59342,32572,74877,38822,87884,67517,60631,97862,5785,75967,2202,90528,34801,51933,37529,16767,99128,81610,74001,69068,50058,90442,51272,31407,61446,88450,14747,10843,52642,66128,31861,12777,53272,84952,23540,3039,20074,14781,19406,31603,44497,59414,338,38625,51672,62974,95249,16924,10352,23039,56983,26865,46435,84528,52595,82960,23162,19556,22135,26921,45364,58558,78373,99484,44768,38292,20473,85809,90273,84957,68920,97985,18279,72403,43283,18950,33512,74259,55408,73757,55742,81661,4916,4123,71126,43681,57496,1575,436,9170,21769,66406,22178,2434,97892,66292,99826,56264,70209,59214,86195,39578,97659,99621,69845,99762,68057,16884,66995,26416,63379,25877,71813,3952,49809,15484,43103,46289,9182,94886,74469,76385,96024,2938,26394,68280,92204,76293,98976,72396,87314,50393,58232,19351,77969,27586,37253,5740,97469,64571,26848,9553,2946,80827,85673,7399,1638,49944,82131,84209,80722,3551,13926,37211,56919,60278,8368,37689,81968,66772,64046,26772,23527,59629,11187,35222,78781,71516,82130,39193,24276,30228,28244,4789,47836,45519,7541,42365,37778,1747,33082,45952,95981,61059,41456,38763,66580,72431,64227,13187,12498,93176,97159,60380,1601,7711,77241,66550,81224,90789,28200,97738,32526,54874,31825,24130,84557,96157,21495,81274,86834,51973,70461,80663,24963,24386,56058,17531,55796,77958,58563,86550,26357,43921,24325,80988,62383,72927,50987,87744,71845,61601,37927,46609,5724,81773,58462,44494,66710,38465,16064,4665,58705,81021,84100,60394,49460,14345,25263,41638,31244,79281,9513,20978,21215,89261,85038,12294,84377,47022,95249,82181,79744,91401,80894,33851,37618,39018,45369,39631,73405,88173,2213,80249,99282,28740,8301,51494,56698,19513,31086,44484,74662,45493,2902,41440,86376,95815,92094,92110,25438,14353,51544,49931,51434,80294,26729,82540,15742,47470,28504,72914,6261,23295,14090,99390,65709,33056,40434,6720,37349,9443,72824,86451,69457,58541,44967,33118,53182,44126,16837,25900,4206,41350,78255,11117,95991,42928,2322,16540,64552,24851,74507,70383,37204,21259,53743,24412,19214,42117,82765,7389,87,1021,32397,1731,53529,50393,41479,90627,99138,88015,98979,72837,40005,66226,47369,48554,96938,20938,52942,71685,24127,66997,9012,54335,8358,73720,86369,54827,645,66755,97114,742,5714,46999,52586,73259,83957,84794,45076,20343,75748,78067,86461,69002,96204,71613,37332,92200,63882,19104,58214,62160,14713,54291,45755,20719,95580,15069,26272,57795,47370,61742,70155,23803,56492,34813,26998,37341,22222,54298,31497,31769,30033,80305,89139,40119,11014,88543,91669,62078,57645,12347,42213,61946,82576,49200,85036,19570,76540,42706,61406,91608,6389,41720,30602,64760,58055,21473,37055,21971,4289,23285,42019,88386,10360,94287,96028,28139,9545,1440,5941,39000,92371,29813,30126,57426,18672,72973,37924,16120,34406,48923,6873,25532,92425,77751,17345,48744,64946,34662,80593,82410,9547,90144,96719,38136,14583,96341,29257,596,26039,37880,11431,20661,40188,81266,55506,37378,27081,79046,29681,16598,71727,18619,64487,68235,11155,89170,28986,2658,42746,18302,84227,68685,63293,77186,40290,57740,44900,65173,99457,15812,7867,63448,88793,71402,11372,44056,97988,64041,28837,35193,5009,67910,11877,29390,11784,93031,25801,35188,48726,48351,89595,39592,80420,82688,22450,88844,19795,99562,56213,41995,81155,53213,5419,41716,73292,40286,23430,16253,14661,95902,6835,44853,40025,58816,45248,54570,23460,12709,10215,41220,63565,75559,73096,45198,63101,70847,17211,1916,87236,55327,48017,18005,33207,8333,2779,93639,46895,91300,30232,92889,97189,26465,71600,73567,41569,24561,57377,59999,17138,4509,4294,99037,41863,67987,66168,16761,59774,33764,30284,97078,2471,21394,62330,69892,883,20790,86597,44290,266,19700,12667,29706,67714,19723,98070,91329,78631,89239,21048,70090,64181,95568,69900,30615,19037,8549,24582,70276,38229,31107,62107,3270,12630,49236,42571,75097,17387,93415,59927,87662,25127,87165,91928,46749,3626,60530,37333,82678,8286,78426,36034,96887,30013,66377,92193,4692,80571,80933,5977,52348,34853,33938,77147,58448,28947,18771,72231,70844,38850,39126,81916,44119,67696,80775,57193,32633,1010,38550,42744,35406,28638,74776,66563,13652,25127,20498,81850,68154,2397,7190,98982,80256,94733,10212,51307,81210,63424,12922,50038,4865,96831,45870,83868,22263,64296,90018,6643,28740,99187,4130,19181,63885,78119,9198,34644,70798,37839,68517,92242,57191,22795,95665,91139,51330,34445,78432,30911,85684,10014,59088,82277,97371,19686,4642,47002,95837,30315,53993,34513,34368,23201,27520,87292,33285,77759,97445,96166,34393,9372,5760,94475,71937,74166,18716,19046,83350,6778,29615,42479,12469,46371,57342,36984,72265,44825,80377,13185,57074,98641,8670,74389,79105,26717,82033,24742,90358,64965,96374,17932,74922,33809,18371,65209,14859,98722,50079,72296,91014,82419,56884,8516,95108,80562,97234,3317,17337,26354,12231,46294,47708,32199,61834,70467,63590,82973,57405,67440,93658,84750,64661,64327,92666,90872,12550,83275,56789,4935,74949,86362,45734,14400,75592,13988,36826,6408,51626,67812,46835,12418,65525,43220,99136,73560,9992,44981,35226,66298,77869,89961,22308,21174,19238,25378,27311,52835,33258,90308,67304,44561,97537,51437,49827,57921,33015,57627,98575,49982,28406,75533,64981,74301,86775,35368,84383,49908,32276,96589,87210,1839,42801,79032,30796,71744,14457,98581,62429,34247,42485,68350,2818,82803,80955,36161,21887,80594,85736,50263,21577,11706,96179,34618,55790,29632,65689,67316,86010,84080,83170,16321,18442,12109,28402,7971,84976,76509,26253,14022,61766,77282,90023,76480,66711,62211,62782,36098,34502,9499,25559,71578,10474,42649,77214,73990,70152,63426,80090,39487,45234,80634,65006,97085,76367,26569,47298,30221,29534,49216,47057,29985,54765,52486,42222,44747,7535,5153,90095,24701,55827,65514,97147,36526,60765,92129,45910,31670,29878,11200,68030,50804,77765,67183,58271,43617,5760,40863,44769,55960,8764,60548,40213,821,46234,89992,85170,20815,41570,93504,25880,56372,33161,53653,97251,78824,93536,88792,48425,74850,57533,79197,19065,63371,7574,42380,88551,39291,34211,13722,1601,37191,1015,22865,35164,61882,68906,67087,67667,7754,51852,4119,49650,51946,29571,51494,59383,3645,64002,21229,53266,93658,15998,73850,49263,12717,58213,81741,84068,88419,24405,17640,6246,18219,9782,73779,89882,16899,90817,60888,38931,21844,95899,34281,28889,47614,39380,42819,7749,49544,44599,73197,83498,94050,48922,38899,51238,45354,24116,96800,72523,85401,59276,25156,57394,76345,20164,7187,91436,11195,48428,27903,89187,91642,67717,61303,38072,26712,87210,89743,63427,11961,66537,6515,54211,91298,63528,56582,59090,13172,73028,57267,24808,79090,51043,43191,71096,59700,8246,2216,20824,98099,54770,67322,84475,62299,73170,57344,54677,4205,62313,42187,94908,36948,32131,28697,59419,14603,858,17644,73817,96725,90125,36411,65570,25542,15740,72153,8017,49410,30630,89278,72992,54907,99658,8585,7075,64777,7774,73096,26680,35926,98832,98943,38566,85360,72600,62613,16429,48399,93967,90729,65778,29964,1958,65618,43657,36101,46742,21866,96796,21611,84678,63655,34774,1382,19765,15916,64133,60809,45744,50311,30986,16728,42419,62946,65484,42923,3736,47075,2261,4660,68499,75446,70920,18082,58649,4392,29459,11241,76114,40386,28935,66250,76640,39902,83525,71910,69598,50716,59380,44567,25902,24388,19465,44469,98154,67932,14595,68086,54,7746,70840,40084,32741,63809,63389,59406,44599,8099,23427,37624,20902,24982,35314,78901,46937,17061,19139,49793,39926,87260,94262,73293,92251,17395,71389,33955,29506,52054,16822,86295,34068,64214,75681,25569,16284,53161,29696,22558,382,87403,14783,67254,93508,93697,20148,23414,89264,68471,258,8101,29089,13171,77777,19037,82165,22360,80099,64532,98251,3496,66292,99222,53317,60757,46893,17529,27655,79782,14750,44225,90594,73110,66523,7019,41923,15780,58515,19799,56965,22826,90028,16234,68944,50019,99878,71559,51227,1193,87145,50835,76670,26032,22405,95932,89861,77275,99643,41104,38899,51366,23152,920,34410,18866,3370,51498,3960,58636,38322,26817,4210,70333,27256,10508,39902,16257,87615,86623,98950,83146,44280,25839,31196,54839,56987,97018,6956,83166,49725,44641,21646,45609,97083,52615,70053,12406,15675,83608,44654,77555,4239,127,2988,40822,50727,70985,40184,50664,48844,52476,61252,43444,98633,30566,72205,33229,69588,79006,44618,1842,16636,77945,85337,31523,90969,68590,12472,61408,37305,42433,77473,11635,69326,25307,89115,53806,24439,43636,30106,99352,82466,2085,76816,46949,92827,80244,75233,52536,72703,75792,85195,99026,19693,76613,57135,78689,36138,52217,58558,37099,67392,56234,51353,92473,5317,87714,53158,77097,96931,38575,36837,58155,81440,22852,92797,71690,88172,51674,52776,79878,23364,37951,57695,80767,70487,4771,90256,27488,74246,50471,98957,36536,18926,97033,33235,4508,86715,37912,23471,6646,21031,14758,17593,65039,99777,10090,1936,86439,7596,17588,86131,6596,2968,76938,11419,41057,28813,50520,94966,66007,46609,169,9506,52951,84358,10295,33660,29289,83847,42038,87406,76756,67428,7451,12869,7575,47549,91985,15317,22668,18638,84499,67398,57125,5069,49559,42850,72477,76,71011,60689,5711,98331,30031,49381,58935,88533,48506,80315,27273,95301,41825,44549,26183,84995,69635,12460,39367,52438,59544,26706,45218,46586,43341,99467,26253,80738,66657,65896,68491,45412,61260,35372,76698,15559,8702,63630,83336,19828,98011,46955,30508,23179,51879,54132,32915,16702,11190,51893,6476,79310,15579,7319,93797,79043,64629,28963,50473,47864,17727,92921,61336,54911,53312,99173,95920,21555,34131,65544,58372,77696,80235,51572,95252,18657,70185,51731,20478,95513,6852,10954,58955,1882,4487,4639,9966,73502,35366,65637,1946,90111,58632,32171,26656,81872,52150,8859,46093,66079,59533,62765,80742,95239,18619,89718,21276,78808,61651,58047,44284,57347,22160,88564,73832,41756,67119,37452,41611,64860,43035,77728,32663,72269,49452,35959,59409,16297,83886,44320,10962,82833,55640,20695,37280,33109,51058,98203,96430,16226,91178,22850,62866,77726,64712,92688,18286,91663,2421,46135,80488,89277,61475,89391,76422,79288,1460,17483,53930,45830,10096,6548,79442,65617,4754,65093,81803,42777,48299,75243,2266,96496,55121,30864,61917,46605,94328,75181,83686,32372,63045,31051,79928,34441,64671,78910,98982,12680,55716,42251,17916,56431,94422,34860,42195,93421,21758,47260,47583,78777,33959,29877,88901,97786,79404,64667,93059,37424,49626,5805,91569,99306,52448,42977,30462,2054,85441,132,56241,13876,92453,12064,38766,20551,33797,2644,69146,57553,81179,60924,61777,91822,46715,56257,61186,71302,21669,5625,73814,63536,17056,38430,68869,91377,74096,15993,69515,30308,21170,16252,49794,65459,22859,51483,70874,86588,22152,22809,38416,28029,75671,34054,47780,33540,59986,83890,45558,34332,73801,39547,77405,25247,20393,94159,15267,10918,15386,58790,65035,86222,30081,56215,31565,68665,9135,15880,47828,56540,75654,36079,49291,79821,65144,85928,79287,5741,939,53662,76499,38753,27048,6082,86860,39072,6112,27250,11325,58250,53603,86680,49318,91498,89037,69953,75101,91034,29533,55904,25912,48234,83777,91816,72232,43501,67099,38904,20729,19402,81282,63234,30413,22841,26122,15993,58192,60586,24327,13148,66934,1947,82535,42753,91965,11424,89045,93405,74764,78316,17374,83328,72729,67978,77293,39206,5034,58684,95138,44644,62931,57546,72473,69704,7200,53097,9156,10677,81902,14010,8714,58194,83089,81471,48774,17208,36585,10581,58042,97755,46865,35086,36699,99499,15648,7354,11646,75616,31946,5212,82205,94095,95449,12281,69418,54445,34393,73770,92088,60529,54817,90503,73798,3996,63260,12338,45934,42782,84384,50295,73286,86802,56542,52932,13231,50518,41547,75640,83963,2326,75892,78663,50952,94196,7218,47955,14148,90312,67022,56686,70596,79166,76514,53007,30333,12529,85292,56414,45051,25854,33695,51798,1993,36186,74594,60729,91125,77754,39811,69962,62335,5838,74478,26928,42425,10760,61780,16616,70144,95250,81738,17323,36839,63930,91356,91904,47124,10631,69523,21627,83799,6807,38994,91404,20229,6492,64221,10130,45030,31835,60892,36464,37692,32454,26775,63353,92373,13386,95031,91442,54508,38183,16762,52509,62992,89700,17877,24356,66143,11120,7880,48599,72276,71734,4476,4354,14917,9162,2843,42178,37116,31665,97934,52078,96391,50186,66434,375,88737,33024,52382,19717,6895,73179,71514,23589,98582,10826,32969,68368,97243,53963,97091,10589,60804,4786,89579,36144,72933,39051,14752,45018,52655,8265,53332,72014,1922,12669,62662,39138,3092,21655,789,97004,93965,7746,3253,55501,56309,71173,10359,67298,63071,68015,12587,96790,72849,48803,34361,1107,11397,61847,55352,75195,33527,76969,30983,54778,43384,54025,28250,35044,47366,23159,321,55531,54721,23282,32157,7757,56293,52516,59005,95724,44815,7305,61614,90756,34745,57438,54357,25139,48836,78983,37811,56053,35260,32839,89117,74147,95129,17308,96532,89595,35123,21761,99905,35031,39939,89227,13892,40123,54698,59416,44126,83735,13390,38906,16495,3166,65365,99031,1526,69312,60642,1330,4130,79588,89834,84743,3297,52795,30310,35538,21351,87256,71084,51026,34013,61063,57233,83780,6562,86486,20270,81841,42982,4937,76008,33903,48346,73836,89509,60317,4413,45965,83134,87480,59796,2009,52651,59375,16370,17464,72485,20064,45410,5696,52655,74665,74612,20687,81285,24348,18714,88618,65861,15052,22170,69863,79521,49825,61063,77287,52738,68650,75453,35784,84264,65793,72083,76432,67076,86303,6360,37541,38949,68279,62767,95351,27090,57997,30189,97605,35838,17561,82224,93573,81712,92441,47480,22518,22142,48297,40320,8971,68640,58684,40018,40439,41046,62126,74919,9263,87884,34410,50751,86220,43366,47030,11751,9660,46507,83180,6803,78112,39990,3442,57058,60872,17954,69895,64087,42467,8312,5197,1847,39762,45375,4716,90079,63659,43166,32500,88640,20684,64695,40234,67148,8271,79799,40532,58427,78635,50824,48398,44089,63688,38712,54392,41138,8813,64363,42791,25304,67405,60089,33,75388,87949,19208,44620,50828,80209,26555,23168,92507,33384,86447,93323,44598,81540,63407,57790,87221,3634,45802,93761,6151,26468,94347,59499,43847,77859,389,79438,88562,41641,45440,65275,71873,68831,21152,65776,53243,25469,82504,27920,17279,35379,35472,15607,68520,49741,84686,8350,84605,9915,67025,2731,83107,83694,69254,4625,65545,79837,93325,91109,79684,76817,18823,88901,47933,52932,84410,15830,65745,31013,94072,61374,74922,58630,48370,41367,73483,99695,44221,47200,7719,98165,64494,69311,83763,47254,84463,73058,38677,15410,24807,87992,67349,13556,71539,57247,25207,26266,26256,69727,28680,50743,53259,54354,2272,78938,29394,18606,6259,7907,34608,55465,76118,91102,35708,71680,5349,82890,51180,4021,44862,53131,93335,54166,37985,36324,82929,7620,22444,87126,10180,56197,91383,68194,19370,51261,52669,27467,2710,85583,43097,85400,56355,7315,78403,71781,32325,47943,77190,42842,21487,41861,61945,84982,12021,11676,89136,72662,27432,84317,68259,19022,13976,25673,98282,66461,84163,83985,12877,22938,40125,48720,9465,97107,88084,10530,31622,68270,4357,50925,79391,79202,397,21738,27000,88043,75650,41751,79779,43552,82591,71742,85927,96093,75604,38104,88334,73580,63115,99016,32851,45184,86100,15283,5936,55670,89760,34833,82516,53875,73929,61381,43266,30750,32019,42827,74774,76182,33809,97347,59658,89791,3639,39913,48883,28747,72448,93286,31674,18232,46792,15073,30993,40607,92993,52390,97591,80927,51422,23102,31337,64230,33663,27058,64091,76940,30565,32605,57937,12203,82512,35099,45789,32524,67031,86081,73473,66294,93803,94657,13245,21480,14229,55663,2895,30696,84647,28242,67629,47950,98208,73900,14749,18977,28395,80808,50985,11009,98863,95926,39219,48426,31937,22254,68124,94665,98727,41609,58159,43360,23031,33223,83184,12277,48729,98645,40385,39002,58780,57414,54700,55038,55866,19781,75512,80093,70452,95926,25388,18093,37914,95102,87495,53603,57913,39485,14158,75167,84708,968,25297,67690,80195,22058,10651,86411,40078,35471,15095,48621,64298,93108,93279,90103,18306,7488,68895,59295,55830,50731,93009,54219,49007,61393,48449,52177,47049,95830,71260,10627,51892,9110,41340,62219,31498,93456,93540,13350,47352,62000,79652,31501,22536,45763,24363,6774,47535,70810,85043,54850,25282,48341,96441,83843,21317,94052,78244,62372,27302,25456,28522,84473,31798,33761,25755,1336,2079,46659,74223,52989,8541,96490,47366,33119,69164,69865,43313,99296,1007,40564,81480,5394,99947,54064,16350,98929,18730,71530,30471,6151,42858,51676,70132,24079,68642,88317,85515,46922,98626,151,60428,29639,56997,8489,7648,77708,8266,93168,16540,68004,77478,79393,62220,37360,34427,23732,94858,21516,66614,61639,990,95728,33164,16558,9567,37190,69761,65451,34025,45012,28218,319,49498,86305,69733,10204,19426,11692,59709,49916,50308,98621,14504,88907,48950,58878,86839,76059,22989,26241,3729,16472,8883,43350,18429,49346,99864,24829,5167,42213,68970,45052,17510,84855,69721,83293,73147,60209,98882,34152,29057,50206,89021,74652,99392,91324,13132,4974,69601,58550,15587,60646,9641,79679,71136,34531,76321,78215,93944,98766,39617,96807,46373,42321,10606,92188,91968,93121,76606,1724,59222,69438,98131,75079,38218,7699,93870,78882,82751,96454,33158,20086,17169,32845,13378,54752,76032,37562,19459,79240,49979,97313,77469,53757,11042,54484,28379,75427,62910,36768,22510,38595,28160,73018,16993,11463,21627,60485,82013,54782,82972,89993,27450,28465,44977,73515,74985,30652,67960,41824,3918,12278,24087,56935,3311,7245,79188,77246,93584,28081,54289,30,11764,3365,49668,61948,16013,36158,47950,96607,55393,73028,34283,83432,17981,20867,81113,70931,99077,9960,96734,72696,80082,86723,34996,68733,34833,41107,45285,28499,25141,38739,82349,53988,80031,1397,12066,41123,89301,83123,86817,16193,63270,43358,66027,14291,47014,16008,4965,92322,3766,48229,81378,59714,66188,34462,97482,49225,6745,60217,93446,38947,95605,63090,70835,41562,34921,13411,23598,55768,56232,46361,43636,3930,8370,22120,47382,81455,99214,94947,28046,98389,46851,54405,66470,53713,96294,49505,35953,81596,88134,89480,1943,45216,42334,99622,65587,57301,72334,61623,58371,37131,49289,41769,64791,29195,92184,4519,81455,32405,15829,7185,70262,96625,89833,79142,80969,33166,55742,41435,95327,53569,72386,72359,3605,23777,53653,96611,64028,9365,86461,98496,35359,84272,81578,34691,67176,93806,42536,17773,91004,23098,68948,48698,18823,76742,45622,10337,92275,83745,75541,69637,82397,95771,45513,7894,8632,7584,37330,15135,76664,48226,94683,86712,47965,47284,27104,4049,86642,61952,94464,26649,63500,86843,75947,17550,69007,78212,12715,1576,78238,44744,64940,48341,76933,20429,46288,32392,16626,2143,4399,49296,23721,66731,85158,11949,58859,68944,62683,45777,23326,54304,90549,89539,63550,44548,94713,1943,65206,36918,61076,2086,18795,72413,97252,29207,47877,18002,72750,63040,99224,55788,18187,83179,88041,6840,22760,3480,22027,57312,86834,93758,68638,86050,45999,14473,32638,92617,68387,46450,59544,43594,57703,51311,49574,19225,5401,50223,19186,29277,4157,13853,89556,29346,23994,79329,42551,11226,59143,39052,26038,32279,77925,9469,96390,35783,16031,91180,39073,76160,47320,35766,42364,63729,23213,11152,93667,67414,97049,20076,99206,57839,56462,1324,49703,67239,33640,65453,62037,45745,46188,13851,53665,47196,85941,98036,44475,76581,95972,93403,23650,73449,63556,27443,47146,53370,46298,81565,47697,45331,98553,48649,44173,12380,73972,52914,21916,4118,27908,63028,63735,27566,30222,21259,38877,32523,98688,5374,40475,49838,5523,67211,68437,18160,69128,71099,53808,10830,16230,49572,57284,45680,35622,64343,84214,46299,36958,98861,98514,1607,30473,6411,65507,17247,63192,97245,18460,36310,98486,94057,88724,41603,10276,4080,40154,67484,2539,44092,33822,60077,64010,15541,59361,47273,68233,86727,57594,72929,93403,26535,51591,5917,72322,99184,37613,90165,55682,55264,86123,38712,14703,56853,42322,46951,78304,98804,52967,86232,42796,83910,32498,73260,65517,82886,73983,72959,58596,65918,4861,91800,79778,53402,62731,92591,81916,78565,35706,58933,6944,50104,22595,7652,50655,53910,16892,56330,29389,63808,96128,85769,57587,97105,70295,7630,1661,3180,76388,25603,87170,47722,4340,81917,86304,70701,63330,48793,46781,33681,63776,22222,4327,14562,11721,78926,15168,30240,25014,70965,29707,51602,39954,86053,59549,10142,11937,79155,82508,3501,5196,56634,62782,77010,8055,34015,35997,29643,64079,36409,87929,3437,13088,52759,47909,69657,848,28144,19145,65567,43621,33056,23664,66209,81885,84852,49058,14346,73170,43729,24197,9976,28971,94016,26560,82294,92966,29450,20628,36670,71178,99504,53249,63890,13464,8074,73942,32056,29542,12768,93342,37681,48656,35685,48572,29409,80239,45980,27794,31987,81260,70020,45954,26756,67649,43154,12534,87066,18398,70282,72257,7193,26426,4810,22278,43008,37250,43121,38855,47863,20378,70795,64075,13472,80186,85417,1461,37483,42906,77065,48933,45439,33888,69096,30611,18823,8070,30651,89643,53791,61549,93571,14489,22831,2532,87863,77528,97427,2675,98961,89059,97714,64353,9081,19371,92829,87128,58062,80384,14273,74957,99381,77176,66155,96681,57201,62350,98971,28246,65629,41236,87189,79760,30753,66917,93699,9640,82319,19924,61629,98844,20833,69063,3318,16573,13211,84397,89109,917,25768,70176,24682,10515,49194,87362,41761,43690,6352,54832,51407,69141,96163,15144,72723,89811,13890,77465,9657,34545,14512,495,19795,37451,25376,83230,92483,9456,64564,79594,72711,15254,91821,3213,30517,44082,17803,37674,39308,82166,53462,75150,69659,69,43870,49032,42409,32043,59161,99608,91982,7031,74011,7477,849,99839,83681,52309,10490,75043,59277,91223,84500,76003,63589,47464,14658,81314,99235,6929,59327,30012,83402,74899,10516,14115,81910,2018,7316,96559,16904,62787,91015,76111,23411,93764,44861,51191,80501,67235,88011,87446,3267,39565,49583,42094,87053,64399,82761,62278,14550,48281,30747,65782,75594,97740,85582,64226,58948,47233,83175,11356,84862,98152,42737,49682,45375,14929,92644,14217,80478,91966,58481,7830,87391,87533,89917,26458,41263,35177,52409,28632,66451,99330,30449,32763,62998,48592,58465,40223,9458,92210,25767,70927,76118,41276,85322,87107,9121,12891,17189,82574,11914,38299,32915,34258,86392,68799,36286,35906,24397,56586,65486,62617,45225,79622,99094,37393,35520,33553,36600,74122,26218,83113,69773,75968,38842,55619,42933,55208,76226,33021,53407,39396,90300,47198,50797,79030,81995,68562,8566,79215,29868,74722,74991,38440,78467,3096,42325,91478,66533,77016,11145,87781,97840,19186,99068,32077,72648,50976,23331,47479,35367,99738,11208,48159,44840,85612,16761,24627,4543,90354,91635,31029,61125,86439,38461,28920,44475,43179,8471,24324,83379,65517,84365,93380,51362,8559,2454,19925,64116,50290,80216,72491,59598,57704,78010,92720,98678,33304,96792,51430,8724,77001,56865,32490,32332,84643,50143,32246,16575,50676,16983,84283,10787,37074,61054,37694,46540,82422,27136,22580,89989,19226,61853,30347,78118,90165,61060,81906,82192,41000,37104,83647,54847,30963,81292,52300,64823,72469,34695,89188,71576,89074,14671,58385,44211,95292,10836,40581,83580,59333,83352,66440,82481,97406,27514,87418,53049,12217,17728,81503,53465,36660,48629,65447,49594,36123,4066,51782,27913,76122,85300,64598,98620,79611,25057,36850,59786,80294,94115,93456,27109,26548,57217,43716,64823,36646,61190,95437,64888,8110,51426,5715,73951,70447,72302,17456,78842,45223,91489,62344,56866,2900,18744,29111,42556,20351,48410,10674,15400,33316,55203,81487,21424,55853,4750,19803,10637,72283,98639,88852,42917,26791,98645,77792,56117,25899,87613,37620,53300,39727,34138,56478,18329,12853,6075,86450,43193,36589,73567,25563,85336,92806,39668,71405,98724,87778,7948,78432,99421,63745,63122,78799,29151,40669,60337,68451,22333,70348,45498,81719,63943,38769,6466,24397,38867,3931,65203,74621,86122,97090,77147,98420,9141,47844,89856,54658,91964,71508,41087,1247,7777,83260,26863,91037,54606,14585,37431,18425,18047,12442,36804,20924,37724,18333,96284,4954,78510,73632,56039,81513,77427,80891,57437,73566,72747,82724,51559,93071,94179,62766,71142,70218,51233,55608,79737,96039,76611,40443,78004,90071,94742,22275,32627,73956,46595,60883,13364,62501,97303,97598,73931,52082,51380,98883,55612,85639,76233,11965]
pipe_list = [[2,1,38376],[3,1,90515],[4,1,44872],[5,4,27098],[6,3,81923],[7,2,7883],[8,1,48187],[9,8,88980],[10,5,82565],[11,6,21256],[12,11,56007],[14,10,55080],[15,6,89746],[16,15,23040],[17,6,62244],[18,16,64191],[19,11,50962],[21,17,42574],[22,15,56091],[23,18,84253],[24,12,94155],[25,1,8237],[26,19,87380],[28,13,23488],[29,19,52088],[30,14,36409],[31,16,54578],[32,7,66614],[33,28,88545],[34,24,95153],[35,19,18324],[36,22,47992],[37,28,615],[39,10,35003],[40,30,75021],[41,15,3006],[42,15,51109],[43,36,95190],[44,23,40331],[45,40,1540],[46,28,34459],[47,43,12651],[48,10,11434],[49,48,2286],[50,28,24814],[51,48,11827],[52,5,30604],[53,35,75738],[54,35,10349],[55,21,28631],[56,1,18536],[58,52,6291],[59,42,34966],[60,16,21743],[61,33,76401],[62,9,48821],[63,46,86913],[64,3,69976],[65,14,61072],[67,50,99077],[68,60,7905],[69,66,51967],[70,46,21722],[71,44,99139],[72,19,6289],[73,27,94929],[74,40,1231],[75,51,4323],[76,7,97059],[79,53,94273],[81,5,22224],[82,21,16440],[83,45,21343],[85,75,28771],[86,82,38767],[87,66,98977],[88,79,6733],[89,69,97300],[90,79,48424],[91,79,49163],[92,51,88129],[93,1,94953],[94,15,95156],[95,48,85473],[96,31,21687],[97,31,9809],[98,53,31870],[99,13,35681],[100,20,18250],[101,82,6289],[102,11,8600],[103,35,90055],[104,21,16642],[105,11,4756],[106,36,31442],[107,8,92449],[109,74,63921],[110,10,61174],[111,86,45057],[112,77,68118],[113,98,89147],[115,53,3913],[116,53,85030],[117,85,75044],[118,45,32192],[119,111,35251],[120,100,19055],[121,98,79639],[122,15,15907],[123,28,95603],[124,52,33255],[125,66,28907],[126,110,24952],[128,35,93857],[129,57,81507],[130,59,95185],[131,74,5934],[132,107,75365],[133,33,2299],[134,14,9015],[135,110,92459],[136,49,39595],[137,27,91467],[138,82,90279],[139,20,36507],[140,74,84902],[141,129,82528],[142,73,73475],[143,41,11124],[144,95,2470],[145,48,26590],[147,125,76254],[148,97,20708],[149,57,28042],[152,36,81415],[153,11,67492],[154,62,96251],[155,64,24084],[156,120,48363],[157,144,98651],[158,133,59053],[159,106,29286],[160,118,23985],[161,36,55896],[162,145,91466],[163,78,66067],[164,122,54289],[165,5,62624],[166,135,36539],[168,76,20860],[169,163,65089],[170,23,24394],[171,170,35012],[172,164,82824],[173,36,86778],[174,173,55229],[175,40,31746],[176,107,19801],[177,168,7507],[178,39,23713],[179,162,88765],[180,51,94342],[181,154,675],[182,126,20952],[183,26,57186],[184,174,9669],[185,96,52524],[186,78,84558],[187,111,26422],[188,29,99507],[189,178,75874],[190,166,34871],[191,74,73933],[192,93,78715],[193,32,90745],[194,173,95789],[195,17,18723],[196,92,53133],[198,187,64642],[199,68,26121],[200,107,55310],[201,101,5737],[202,4,33174],[203,52,95369],[204,196,60223],[205,124,81639],[206,20,9807],[207,125,95800],[208,203,30506],[209,102,97700],[211,146,41945],[212,70,96647],[213,159,46347],[214,50,82501],[215,70,87197],[216,167,86263],[217,76,70157],[218,61,71741],[219,116,60950],[220,145,64363],[221,64,39875],[222,218,59772],[223,89,35036],[224,150,89588],[225,70,43600],[227,102,58796],[228,107,7924],[229,48,16400],[230,31,72418],[231,215,42108],[232,36,40978],[233,224,44959],[234,1,83903],[235,132,622],[236,30,48201],[237,13,64693],[238,179,51785],[239,207,64471],[240,55,773],[241,142,92943],[242,45,95096],[243,100,77928],[244,235,11782],[245,51,87547],[246,245,53980],[247,168,69034],[248,23,52022],[249,111,84416],[252,217,49282],[253,99,80185],[254,126,72320],[255,148,86238],[256,226,39344],[257,238,55190],[258,94,1380],[259,96,38274],[260,242,93345],[261,202,14692],[262,140,70594],[263,186,54906],[265,235,35517],[267,42,53322],[268,127,81117],[270,102,89663],[271,16,1730],[272,266,24933],[274,247,83472],[276,101,91429],[277,39,66159],[279,192,35471],[280,258,49817],[281,156,43938],[282,137,2054],[284,228,23143],[286,214,9378],[287,71,25863],[288,265,20845],[289,63,90054],[290,37,73884],[291,95,96735],[292,220,94771],[294,239,64439],[295,230,75768],[296,125,31133],[297,178,50585],[298,5,15484],[299,174,98597],[300,188,50127],[301,195,70259],[302,254,53151],[303,239,83187],[304,255,57805],[305,291,84482],[306,168,3149],[307,32,92687],[308,197,15623],[309,192,1228],[310,31,50883],[311,188,99095],[312,109,94648],[313,301,82661],[315,310,43576],[316,282,47276],[317,282,56650],[318,139,66148],[319,318,88345],[320,99,8452],[321,256,68908],[322,159,55088],[323,209,32295],[325,310,92361],[326,256,87383],[327,236,35821],[328,51,38819],[329,241,43583],[330,131,59043],[331,116,33326],[332,179,99356],[333,152,51781],[334,62,90587],[335,275,70702],[336,249,26586],[338,268,16870],[339,104,91733],[340,194,64439],[341,120,8664],[342,287,88757],[343,246,68549],[344,8,72765],[345,200,54765],[346,227,56062],[347,93,91131],[348,34,53509],[349,43,4178],[350,178,71075],[351,26,49384],[352,181,28395],[353,81,4863],[354,80,13002],[356,114,60369],[357,167,89092],[358,327,52165],[360,268,69161],[361,75,79525],[362,67,15568],[365,44,33119],[366,212,994],[367,313,83831],[368,235,2424],[369,246,44625],[370,143,37077],[371,196,3910],[372,137,55588],[373,252,48978],[374,77,31049],[375,201,61624],[376,347,11516],[378,58,44680],[379,180,96275],[380,280,63586],[382,332,33936],[383,126,84049],[384,28,98644],[385,274,65321],[387,95,68873],[388,373,81589],[389,321,83883],[390,152,44259],[391,57,71020],[392,155,18617],[393,234,95472],[394,108,24648],[395,51,38123],[396,19,47750],[397,218,66079],[398,271,74169],[399,34,16872],[401,368,28341],[403,293,47496],[404,67,86744],[405,162,96011],[406,101,89459],[408,404,1672],[409,406,62650],[410,398,3177],[411,70,86765],[412,42,80506],[413,2,31871],[414,94,49736],[416,344,95120],[417,27,90205],[418,213,39231],[419,161,40388],[420,186,15809],[421,205,98293],[422,5,10909],[423,381,29461],[424,266,39146],[425,397,12065],[426,412,91566],[427,81,69790],[429,44,98608],[430,334,63759],[431,144,75339],[432,151,49373],[433,340,56598],[434,226,35712],[435,327,88796],[436,242,84907],[437,380,92224],[438,68,60906],[439,168,59543],[440,69,37243],[441,292,84819],[442,168,99103],[443,376,88265],[444,225,93408],[445,419,92993],[446,108,84525],[447,347,3048],[448,31,17538],[449,150,22620],[450,276,1252],[451,215,20733],[452,33,64616],[453,129,68680],[454,141,30472],[456,227,10849],[457,446,55154],[458,301,14338],[459,297,52787],[460,392,72002],[461,202,56462],[462,305,27536],[463,12,43791],[464,80,79672],[465,196,30385],[466,184,50136],[467,41,87673],[468,392,37425],[469,465,87104],[470,126,64894],[471,48,83679],[472,368,84459],[473,219,8609],[474,421,9135],[475,464,72865],[476,341,53729],[477,375,34249],[478,231,44012],[479,460,96617],[481,224,99444],[483,44,92772],[484,81,72226],[485,290,11706],[486,402,26275],[487,461,37388],[488,7,34878],[489,100,77154],[490,192,31258],[491,290,37395],[492,121,86214],[493,169,89630],[494,308,45570],[496,279,98060],[497,6,32982],[498,236,1133],[499,356,8015],[500,457,58031],[501,100,57758],[502,244,95718],[503,331,89263],[504,466,3550],[506,380,21153],[507,313,88594],[508,154,54340],[509,317,32539],[510,46,74312],[511,83,73027],[512,511,31433],[513,176,82132],[515,268,5465],[517,324,68899],[518,27,97843],[519,99,40558],[520,210,58226],[522,328,59383],[523,363,77079],[524,506,29996],[525,308,41009],[526,188,12852],[527,429,43113],[528,153,40296],[529,384,36153],[530,224,66067],[531,370,64532],[532,183,78619],[533,127,62289],[535,357,644],[536,205,18343],[537,112,29666],[538,131,80885],[539,74,47327],[540,413,22951],[541,174,26881],[542,275,34518],[543,36,62687],[544,251,74634],[545,117,70508],[546,364,4071],[547,76,71144],[549,208,5676],[550,218,38580],[551,49,79626],[552,137,9787],[553,516,80028],[554,142,96696],[556,297,40296],[557,327,86527],[558,458,82982],[559,535,73492],[560,165,99904],[561,380,84424],[562,479,34153],[563,102,49267],[564,504,63692],[565,490,65224],[566,197,14103],[567,267,19362],[568,306,14102],[569,394,79543],[570,517,2530],[571,505,57908],[572,223,79346],[573,507,1318],[574,282,82836],[575,395,2170],[576,419,97446],[577,206,47043],[578,408,78207],[579,495,31115],[581,245,77875],[582,283,5817],[583,1,75887],[584,71,96982],[585,153,44521],[586,291,30315],[587,345,80259],[588,141,54261],[589,29,80791],[590,308,87255],[591,371,64032],[592,520,5454],[593,116,21760],[594,96,6429],[595,196,74847],[596,428,82765],[597,410,71406],[598,112,7173],[599,59,34077],[600,113,14958],[601,344,70620],[602,565,28451],[603,197,9977],[604,343,56139],[605,424,77853],[606,471,10440],[607,410,3297],[608,161,78866],[609,523,8399],[610,191,27413],[611,38,65666],[612,439,12865],[613,476,8046],[614,541,18491],[615,508,73151],[616,227,81131],[617,460,81673],[618,49,96867],[619,253,55985],[620,372,74611],[621,326,85616],[622,509,46670],[623,493,43763],[624,309,97275],[625,251,81900],[626,54,98730],[627,109,99410],[628,139,42319],[629,109,40544],[630,272,82800],[631,44,21563],[632,212,25638],[633,550,69402],[634,35,87832],[636,183,54153],[638,143,97450],[639,283,15858],[640,347,28016],[641,501,79450],[642,72,84861],[643,486,82826],[645,63,531],[646,11,54983],[648,268,48524],[650,88,41561],[651,536,15429],[652,495,90350],[653,472,89470],[654,355,57509],[655,575,88698],[656,220,13299],[657,113,17139],[658,521,38591],[659,414,30004],[660,406,6202],[662,62,61069],[663,101,94555],[664,458,29358],[666,528,94558],[667,602,57947],[668,378,70813],[669,600,66685],[670,416,40150],[671,611,51627],[672,437,67821],[673,260,74036],[674,446,62776],[675,663,63453],[676,409,65132],[677,639,84727],[678,307,65636],[679,603,36372],[680,350,27176],[681,559,8071],[682,253,11126],[684,201,7698],[685,446,17414],[686,178,691],[687,257,68999],[688,90,19816],[689,484,23379],[690,467,42976],[691,174,30630],[693,471,79993],[694,10,71],[695,208,59031],[696,522,1084],[697,533,93163],[698,60,9976],[700,46,66297],[701,643,39950],[702,354,70381],[703,535,89687],[704,36,12943],[705,624,55345],[706,318,24236],[707,435,66986],[708,521,92546],[709,692,74281],[710,566,62304],[711,76,48486],[712,229,16856],[713,338,1581],[714,488,36051],[715,565,7705],[716,500,64324],[717,210,96987],[718,116,40398],[719,229,73333],[720,347,48436],[721,465,39594],[722,607,98634],[723,458,68305],[724,518,61076],[725,437,18379],[726,344,41181],[727,115,83278],[728,29,89786],[729,635,77170],[730,419,2593],[731,259,12234],[732,653,88905],[733,111,80133],[734,391,44470],[735,630,49914],[737,717,8405],[738,667,79154],[739,440,171],[740,412,46314],[741,79,68828],[742,185,11710],[743,233,3981],[745,279,4515],[746,29,54372],[747,138,8362],[748,309,30963],[749,538,27726],[750,415,91438],[751,398,7163],[752,473,96423],[753,8,84158],[754,387,70178],[755,444,3063],[756,205,82071],[757,312,33409],[758,73,23319],[759,228,84452],[760,118,47447],[761,694,52196],[763,509,91326],[764,307,43928],[765,376,33323],[766,582,95457],[767,271,11424],[768,384,54776],[769,650,94727],[770,61,87060],[771,45,544],[772,540,71193],[773,674,20371],[774,48,48384],[775,219,83682],[776,195,14040],[777,478,6984],[778,220,20771],[779,694,81482],[780,278,31430],[781,439,44397],[782,282,36851],[784,56,80768],[785,117,57298],[786,524,15717],[787,255,60578],[788,783,19299],[789,483,19986],[790,575,65102],[791,612,69521],[792,680,95753],[793,432,48514],[794,151,37896],[795,652,7408],[796,223,66598],[797,492,30033],[798,776,56358],[799,478,32849],[800,791,75241],[801,442,28762],[803,565,90822],[804,71,69277],[805,665,59771],[806,524,82424],[807,806,58930],[808,370,63473],[809,734,73123],[810,630,48272],[811,555,97350],[812,390,67639],[813,520,58411],[814,381,16077],[815,737,68991],[816,809,98300],[817,448,8647],[818,236,67766],[819,409,31965],[820,108,48743],[821,68,55182],[822,111,82034],[823,52,97191],[824,703,45727],[825,574,70273],[826,797,95063],[827,287,10767],[828,791,61021],[831,148,42983],[832,493,99146],[833,270,50965],[835,776,67931],[836,192,77428],[837,253,52157],[838,449,7566],[839,582,59631],[840,111,11973],[841,34,7815],[842,33,11253],[843,548,54543],[844,663,32575],[845,215,27895],[846,38,70889],[847,124,57814],[848,580,34619],[849,291,52385],[850,512,4747],[851,376,67747],[852,607,29785],[853,211,45600],[854,806,9788],[855,677,7955],[857,844,17275],[858,713,97619],[859,355,17926],[860,167,64606],[861,820,81033],[862,858,59644],[863,601,51628],[865,47,79481],[866,227,88305],[867,532,94260],[868,374,83572],[869,598,61895],[870,855,15236],[871,130,25633],[872,686,38774],[873,621,10259],[874,96,15419],[875,453,27054],[876,629,19160],[877,512,2288],[878,796,5658],[879,800,78594],[880,146,81356],[881,158,4839],[883,350,5776],[884,705,265],[885,675,36652],[886,270,10824],[888,460,78885],[889,243,24285],[890,104,78885],[891,497,75478],[893,20,92406],[894,849,7713],[895,832,28785],[896,346,3992],[897,780,17913],[898,475,70460],[899,330,73576],[900,33,1924],[901,538,82153],[903,338,70776],[905,89,9500],[906,118,70504],[907,167,20002],[908,559,81693],[909,817,72571],[910,114,56298],[911,633,70071],[912,319,75221],[913,113,14551],[914,246,36113],[916,600,88131],[917,183,51586],[918,76,97690],[919,21,99916],[920,315,37962],[921,188,15171],[922,285,2222],[923,659,15671],[924,790,86391],[926,695,92825],[927,154,40077],[928,490,46082],[929,596,51299],[930,375,6943],[931,257,58170],[932,469,38310],[933,476,52035],[935,347,81084],[936,294,57162],[937,457,2583],[938,212,85393],[939,744,48894],[940,933,73169],[943,7,11252],[944,562,72130],[945,539,75014],[946,303,27393],[947,269,59636],[948,246,69461],[949,163,97941],[950,499,73027],[951,902,46943],[952,56,16556],[953,129,16549],[954,900,41293],[955,664,21722],[956,152,85551],[957,3,85668],[958,336,68755],[959,465,69298],[960,681,36322],[962,557,9555],[963,379,54884],[964,773,17225],[965,812,85672],[966,658,18135],[967,924,21923],[968,89,73005],[969,137,95240],[970,715,85625],[971,24,19706],[972,195,18337],[973,477,33052],[974,33,89802],[975,442,95350],[976,477,25489],[977,276,77640],[978,880,7275],[979,577,22124],[980,455,24659],[981,810,78747],[982,915,54665],[983,341,89916],[984,852,77570],[985,859,20382],[986,906,39182],[987,212,17211],[988,206,54168],[989,58,7226],[990,976,19533],[991,554,33028],[992,27,66449],[993,44,797],[994,313,94690],[995,388,17946],[996,456,60864],[997,348,12238],[998,339,54204],[999,413,6567],[1000,718,34806],[1001,663,52893],[1002,715,10923],[1003,876,76054],[1004,521,52474],[1005,875,30029],[1006,296,66736],[1007,653,52137],[1008,274,99142],[1009,1003,88886],[1010,264,48377],[1011,973,73879],[1012,948,28097],[1013,131,22029],[1014,717,21592],[1015,435,93427],[1016,434,32747],[1017,14,66783],[1018,585,28232],[1019,530,39567],[1020,282,80508],[1021,918,89033],[1022,138,28863],[1023,38,87620],[1024,93,14005],[1025,424,91014],[1027,71,68837],[1028,977,45564],[1029,763,17630],[1030,592,53590],[1031,328,4519],[1032,943,64373],[1033,398,83398],[1034,371,12262],[1035,612,29623],[1036,420,65198],[1037,549,78188],[1038,633,97720],[1039,497,16664],[1040,913,26695],[1041,335,99471],[1042,88,73477],[1044,939,47333],[1045,387,79330],[1046,769,73612],[1047,549,49982],[1049,857,61079],[1050,388,21150],[1052,57,64516],[1053,166,33680],[1054,401,85366],[1056,486,18138],[1057,820,44854],[1058,573,53756],[1059,984,98328],[1060,285,13650],[1061,22,80092],[1062,676,83981],[1063,915,93778],[1064,910,42664],[1065,397,75401],[1067,190,72568],[1068,980,68391],[1069,528,62082],[1070,227,83292],[1071,607,40000],[1072,828,45911],[1073,998,26987],[1074,777,23168],[1075,1006,68844],[1076,513,3780],[1077,570,63097],[1078,271,34390],[1079,645,85182],[1080,961,75090],[1081,1040,62599],[1082,350,71944],[1083,1076,72688],[1084,710,61518],[1085,1019,32457],[1087,1000,82759],[1088,501,27507],[1089,350,71785],[1090,527,49450],[1091,497,86007],[1093,806,24086],[1094,952,96417],[1095,1015,55294],[1097,1024,29800],[1098,218,4350],[1099,331,48417],[1100,506,76276],[1101,1,24576],[1102,727,86520],[1103,900,49790],[1104,439,38204],[1105,1057,34892],[1106,813,78841],[1107,873,48091],[1108,454,6095],[1109,819,76069],[1110,804,19175],[1111,726,80376],[1112,85,85147],[1113,495,16233],[1114,67,24108],[1115,913,95519],[1116,233,56966],[1117,82,40094],[1118,96,44437],[1119,998,83016],[1120,316,17629],[1122,506,49058],[1124,565,1164],[1125,844,54500],[1126,270,58687],[1127,815,52660],[1128,146,11200],[1129,387,49339],[1130,283,45013],[1131,124,3035],[1133,968,73274],[1134,347,28487],[1135,503,83194],[1136,549,542],[1137,902,81631],[1140,604,15710],[1141,667,62132],[1143,538,56937],[1144,1094,75317],[1145,302,86453],[1146,256,59134],[1147,971,49858],[1148,484,72085],[1149,1026,40819],[1150,1110,59208],[1151,703,49002],[1152,912,13841],[1153,828,21364],[1155,581,92107],[1156,109,64830],[1157,689,89900],[1158,936,22889],[1159,487,56484],[1160,1035,75188],[1161,388,93620],[1162,366,66857],[1163,667,47678],[1165,648,98643],[1166,628,28168],[1167,850,45102],[1168,223,22717],[1169,1023,14688],[1170,515,83081],[1171,348,37889],[1172,391,22883],[1173,296,28038],[1174,543,60457],[1175,1036,47149],[1176,1173,37213],[1177,1159,51251],[1178,431,37257],[1180,72,86631],[1181,672,32007],[1182,806,97307],[1183,1017,49155],[1184,660,29016],[1185,355,14539],[1186,400,49259],[1187,910,60891],[1188,1140,63939],[1189,411,89633],[1190,931,26882],[1191,618,91690],[1192,1044,23963],[1193,1012,27301],[1194,470,85635],[1195,496,94491],[1196,770,26789],[1197,62,98717],[1198,952,84559],[1199,924,35933],[1201,191,55935],[1202,371,83873],[1203,357,25553],[1204,389,14439],[1205,895,9215],[1206,576,11936],[1207,80,75090],[1208,289,34988],[1209,1022,98487],[1210,315,65363],[1211,818,58274],[1212,968,12345],[1213,335,64034],[1214,996,62808],[1215,918,81220],[1216,450,62342],[1217,218,99342],[1218,843,15908],[1219,478,11106],[1220,628,75828],[1221,1003,44950],[1222,505,89018],[1223,899,58682],[1224,159,71190],[1225,1002,75386],[1226,599,72659],[1228,1073,9009],[1229,45,66697],[1230,1126,96574],[1231,1194,39247],[1232,214,35426],[1233,294,27529],[1234,800,89289],[1235,758,67755],[1236,1078,42303],[1238,792,49745],[1239,468,82879],[1240,598,36074],[1241,33,28915],[1242,1151,74968],[1243,449,77596],[1244,206,38814],[1245,757,15221],[1246,731,44219],[1247,149,80008],[1248,706,4224],[1249,1103,87878],[1251,819,1506],[1252,735,66670],[1253,315,98100],[1254,1191,1993],[1256,1083,40399],[1257,1018,23807],[1258,1233,25190],[1259,186,8347],[1260,1029,7704],[1261,759,543],[1262,119,94172],[1265,991,75898],[1266,707,7499],[1267,794,21632],[1268,1218,33424],[1269,37,53056],[1270,166,16613],[1271,763,85921],[1272,120,61032],[1273,1060,24937],[1274,991,14490],[1275,1160,74143],[1277,50,7548],[1278,827,4106],[1279,228,32584],[1280,511,51782],[1281,818,1186],[1282,1135,95940],[1283,316,91466],[1284,788,91964],[1287,579,91399],[1288,1023,84263],[1289,926,30710],[1290,1034,2989],[1291,443,24998],[1292,600,43001],[1293,1227,18551],[1294,857,34420],[1295,811,45869],[1296,467,20375],[1298,1039,10277],[1299,946,98398],[1300,577,96111],[1301,777,10323],[1302,528,55336],[1303,841,54430],[1304,917,55592],[1305,18,22946],[1306,692,81040],[1307,619,7597],[1308,146,192],[1309,149,88619],[1310,869,50228],[1311,440,74495],[1312,509,64687],[1313,114,74746],[1314,54,15955],[1316,1152,60500],[1318,159,40679],[1319,1296,90994],[1320,1212,99809],[1321,775,35391],[1322,315,92325],[1323,479,19626],[1324,420,95888],[1325,34,13673],[1326,1307,94679],[1327,167,81137],[1328,1322,63732],[1329,1129,83968],[1330,691,93913],[1331,109,68944],[1332,128,85022],[1333,681,20240],[1334,862,42893],[1336,557,56667],[1337,353,3855],[1338,1167,3219],[1339,357,53157],[1340,338,40744],[1341,536,42599],[1342,602,1147],[1343,516,24795],[1344,579,98848],[1345,941,15099],[1346,723,82037],[1348,260,45381],[1349,1057,68797],[1350,1149,51649],[1352,488,63765],[1353,819,49044],[1354,1327,18139],[1355,1204,26960],[1356,77,66751],[1357,1228,42051],[1358,328,35247],[1359,1023,12249],[1360,539,88088],[1361,1307,39388],[1362,1323,52520],[1363,567,42294],[1364,490,10588],[1365,1253,35249],[1366,798,67626],[1367,738,48429],[1368,654,69265],[1370,199,47556],[1371,1206,79745],[1372,172,66738],[1374,6,1697],[1375,659,88085],[1376,661,41937],[1377,940,9839],[1378,1056,34955],[1379,170,4238],[1380,201,91313],[1382,20,20211],[1383,526,12445],[1384,373,3355],[1385,372,775],[1386,1042,35496],[1387,427,6141],[1388,100,63592],[1389,1225,66481],[1391,439,64863],[1392,1040,18699],[1393,586,6752],[1394,898,60781],[1395,952,6693],[1396,275,33458],[1398,252,82936],[1399,659,84222],[1400,376,10915],[1401,888,19026],[1402,1075,28376],[1403,244,58746],[1405,1002,74949],[1406,1386,13236],[1408,1383,1213],[1409,1348,92806],[1410,1003,85553],[1411,582,52212],[1412,1370,67408],[1413,480,44941],[1415,249,7836],[1417,471,66215],[1418,342,15777],[1419,331,18013],[1420,945,88752],[1421,429,12074],[1422,1144,41127],[1423,1322,76109],[1424,1112,66372],[1425,1118,1544],[1427,150,74366],[1428,205,54114],[1429,583,27564],[1430,779,44218],[1431,289,16518],[1432,32,57403],[1433,965,10257],[1434,231,62033],[1435,309,72223],[1436,1128,99043],[1437,1281,7237],[1438,300,55090],[1439,269,14725],[1440,887,43064],[1441,1297,91043],[1443,367,27940],[1445,1017,72913],[1446,238,47948],[1447,1015,59139],[1448,1253,54497],[1449,43,43917],[1450,632,68574],[1451,94,18745],[1452,867,81003],[1453,316,33194],[1454,905,39151],[1455,309,44131],[1456,305,11225],[1457,1321,43662],[1458,711,8657],[1459,185,6920],[1460,607,61908],[1461,1002,50327],[1462,951,53324],[1463,1244,201],[1465,338,19107],[1466,985,63313],[1467,506,99025],[1468,194,8730],[1469,187,22369],[1470,1407,39433],[1471,215,25917],[1472,182,90505],[1473,789,71623],[1474,860,16221],[1475,7,65343],[1477,1366,7250],[1478,247,39399],[1479,1262,56784],[1480,1369,34557],[1481,700,90204],[1482,346,36395],[1483,65,11158],[1484,38,92602],[1486,1018,99903],[1487,426,66254],[1488,331,91274],[1489,1193,47312],[1490,1339,2991],[1491,831,11545],[1492,660,19466],[1493,63,61949],[1494,1113,74929],[1495,1262,98362],[1496,1366,40435],[1497,1119,59401],[1498,981,26247],[1499,673,46503],[1500,1201,16308],[1501,809,1552],[1502,697,48573],[1503,1446,866],[1504,924,76961],[1505,1313,55840],[1507,35,18665],[1508,1110,15473],[1509,943,52105],[1510,625,14047],[1511,764,66778],[1513,306,87027],[1514,13,36020],[1516,829,13093],[1517,944,84575],[1518,799,66408],[1519,1104,24390],[1520,1239,40578],[1521,566,2176],[1522,446,33626],[1523,1152,70268],[1524,761,44575],[1525,1416,8698],[1526,621,52236],[1527,536,23608],[1528,824,35517],[1530,1109,34553],[1531,1377,25752],[1532,158,85425],[1533,413,28761],[1534,865,8859],[1535,1080,29913],[1536,548,92779],[1537,1373,83823],[1538,1006,21101],[1539,637,77402],[1540,1203,96778],[1541,484,18305],[1542,520,89538],[1543,1423,481],[1544,1447,80687],[1545,674,59570],[1546,320,57089],[1547,635,1651],[1548,162,55669],[1549,1152,78652],[1550,929,74813],[1551,756,10862],[1552,241,6206],[1553,89,71609],[1555,952,88750],[1557,274,87144],[1558,844,96952],[1559,127,5586],[1560,463,27493],[1561,77,62556],[1562,1363,66282],[1564,331,38706],[1565,1353,15542],[1566,1553,56047],[1567,811,36755],[1568,397,12279],[1569,1475,34653],[1570,73,69760],[1571,264,48036],[1572,599,25664],[1573,858,6205],[1574,809,82241],[1575,1296,83630],[1576,786,93592],[1577,1410,2979],[1578,1555,30128],[1579,198,13895],[1580,763,13591],[1581,903,30566],[1582,288,33927],[1583,586,14424],[1585,1466,4551],[1586,1281,83280],[1587,959,78461],[1588,1573,79064],[1589,390,35516],[1590,650,89040],[1591,1034,65346],[1592,378,92438],[1593,614,13575],[1594,274,84195],[1595,1432,85829],[1596,1177,3953],[1597,707,53597],[1598,905,75780],[1599,1566,89765],[1600,980,92279],[1601,807,40299],[1602,1297,43105],[1603,1547,61352],[1604,998,24293],[1605,700,70623],[1606,1578,95322],[1607,1252,49389],[1608,665,39164],[1609,1421,89275],[1610,764,50253],[1611,14,8862],[1612,700,17666],[1613,542,80364],[1614,1491,74716],[1615,689,82883],[1617,771,48839],[1619,267,43548],[1620,83,33384],[1621,191,63993],[1622,887,54363],[1623,1304,25381],[1624,844,51983],[1625,559,53389],[1626,301,28111],[1627,458,96891],[1628,937,73012],[1629,1044,74949],[1630,1035,40190],[1631,1199,88750],[1632,457,99703],[1633,682,24820],[1634,1173,3652],[1635,227,70939],[1636,1076,37736],[1638,486,91722],[1639,401,5430],[1641,1055,59616],[1642,1480,92550],[1643,1108,96346],[1644,1018,69934],[1645,1086,18710],[1646,8,62143],[1647,1104,40022],[1648,407,20047],[1650,1022,37445],[1651,1308,13757],[1652,733,29309],[1653,1246,78302],[1654,762,48436],[1655,781,34956],[1656,491,27036],[1657,246,80958],[1658,457,23784],[1659,437,29992],[1660,370,48708],[1661,35,58583],[1662,147,94693],[1663,1630,75919],[1664,206,12788],[1666,61,34517],[1667,878,18784],[1668,1189,48708],[1669,199,48916],[1670,708,85085],[1672,384,17549],[1674,1126,1102],[1676,559,21765],[1677,296,73795],[1678,267,66373],[1679,443,75033],[1680,1626,26452],[1681,818,27380],[1682,1138,26025],[1683,1555,87921],[1685,1374,29724],[1687,1060,22439],[1688,948,9272],[1689,371,85235],[1690,56,86413],[1691,468,97194],[1692,718,15584],[1693,522,89007],[1694,1258,50104],[1695,1653,5218],[1696,464,8843],[1697,13,79995],[1699,1390,80240],[1700,1501,43156],[1701,1453,75753],[1702,1550,38374],[1703,165,56067],[1704,707,96498],[1705,1382,95772],[1706,1610,1262],[1707,225,55719],[1708,784,53344],[1709,376,84265],[1710,1201,87636],[1711,619,97001],[1712,79,67312],[1714,1418,931],[1715,1681,59584],[1716,855,66056],[1717,674,21114],[1718,208,30344],[1719,1052,8393],[1720,515,24945],[1721,1319,11977],[1722,566,53409],[1723,877,77569],[1724,1040,79897],[1725,436,21290],[1726,1231,80915],[1728,365,62802],[1729,724,5721],[1730,335,18938],[1732,291,46271],[1733,1359,42008],[1735,82,38476],[1736,1501,50855],[1737,898,41879],[1738,75,91421],[1739,987,66775],[1740,543,23709],[1742,259,6762],[1744,64,54418],[1745,1363,50644],[1747,1629,34052],[1748,914,71485],[1749,467,42855],[1750,1629,36827],[1751,1479,12346],[1752,1134,49422],[1753,621,920],[1754,129,29728],[1755,630,32706],[1756,980,63456],[1757,456,57500],[1758,1629,37852],[1759,1693,11689],[1760,334,65057],[1761,339,99253],[1762,693,47503],[1763,1179,14067],[1764,705,44885],[1765,692,21846],[1766,1460,42729],[1767,1268,11417],[1768,1272,2802],[1769,1556,74315],[1770,1519,59012],[1771,454,90930],[1773,1718,77923],[1774,881,1505],[1775,1010,10125],[1776,1441,75563],[1777,990,66603],[1778,482,64967],[1780,811,87590],[1781,1573,60432],[1782,941,75424],[1783,1761,18125],[1785,705,16243],[1786,1776,77322],[1787,698,84882],[1788,1169,39215],[1790,1250,16397],[1791,403,17692],[1792,360,30039],[1793,1151,10948],[1794,911,61819],[1795,141,89061],[1796,1,76342],[1797,1311,69389],[1798,1088,86627],[1799,113,83946],[1802,37,81401],[1803,1336,82307],[1804,692,27886],[1805,191,87994],[1807,1662,75020],[1808,1092,34628],[1809,959,68044],[1810,1463,85751],[1811,177,84666],[1812,1413,65355],[1813,110,75208],[1815,756,56967],[1816,1345,56378],[1817,1008,87775],[1818,1796,97227],[1819,749,22425],[1820,1694,44410],[1821,850,87918],[1822,1353,25991],[1824,1444,11596],[1825,708,54308],[1826,274,83557],[1827,1476,14271],[1828,513,46057],[1829,1246,94001],[1830,843,35439],[1832,607,92215],[1834,94,14412],[1835,1308,44664],[1836,726,57840],[1837,870,56391],[1838,17,37889],[1839,870,21426],[1840,941,59173],[1841,44,37076],[1842,1625,21019],[1843,665,57755],[1844,1018,65107],[1845,1570,87502],[1846,23,65440],[1847,1392,50045],[1848,511,60465],[1849,126,5212],[1851,1575,57605],[1852,1815,10321],[1853,202,70020],[1854,1335,75077],[1855,1406,95549],[1856,1749,29479],[1857,1695,26054],[1858,570,23144],[1859,323,7126],[1861,1335,57987],[1862,495,54157],[1863,656,24363],[1864,1035,5144],[1865,723,48652],[1866,1275,86760],[1867,559,45589],[1868,193,61814],[1869,1696,59877],[1870,1419,60991],[1871,420,13921],[1872,51,39565],[1873,203,57006],[1874,1339,18378],[1875,1820,62096],[1876,423,98921],[1877,791,99346],[1878,1647,99099],[1879,97,74135],[1880,12,1474],[1881,1853,64953],[1882,391,67293],[1883,1041,74741],[1884,1609,87256],[1885,780,24514],[1886,1173,3625],[1887,622,4152],[1888,263,47135],[1889,601,93135],[1890,107,54359],[1891,1623,95903],[1892,1770,48979],[1893,1099,66524],[1894,1728,47050],[1895,1291,72394],[1896,580,91551],[1897,333,55274],[1898,1335,21585],[1899,659,99413],[1900,329,98000],[1901,1469,33656],[1902,678,87551],[1903,1748,9959],[1904,523,95831],[1905,569,8846],[1906,613,81552],[1907,493,58977],[1908,509,4333],[1910,732,59086],[1911,984,71417],[1912,391,32327],[1913,1712,48042],[1914,178,97943],[1916,950,86240],[1917,778,17761],[1918,1493,247],[1920,869,57082],[1921,707,26700],[1922,773,9306],[1924,1437,35174],[1925,1751,79411],[1926,719,81081],[1927,839,99780],[1928,648,79307],[1929,160,59930],[1931,119,30895],[1932,1675,5344],[1933,1051,59231],[1934,1829,13584],[1935,914,56272],[1937,9,39351],[1938,289,16910],[1939,935,83570],[1940,951,4546],[1941,667,3388],[1942,932,85889],[1943,1695,88622],[1944,142,84647],[1945,1756,20574],[1946,116,80904],[1947,1652,15420],[1949,28,14964],[1950,548,23157],[1951,1356,29050],[1952,467,2488],[1953,1372,65035],[1955,108,58506],[1957,934,21603],[1958,1042,86377],[1959,670,89637],[1960,1067,30303],[1961,301,95717],[1963,1202,97816],[1964,1034,60627],[1965,1069,33213],[1966,1891,61043],[1967,1307,88882],[1968,564,79812],[1969,1029,11218],[1970,1304,31907],[1971,92,46201],[1972,1133,38751],[1973,695,54616],[1974,1750,85435],[1976,677,21529],[1978,1301,48204],[1979,1306,33031],[1981,1390,25344],[1982,1872,7387],[1983,1740,14565],[1985,121,36723],[1987,762,57696],[1989,704,37807],[1990,415,52150],[1991,105,88813],[1992,1950,45877],[1994,1892,78781],[1995,67,44412],[1996,74,53354],[1997,1015,17927],[1998,330,47607],[1999,1670,80948],[2000,972,14678],[2001,1235,13507],[2002,415,20772],[2004,710,41063],[2005,201,26513],[2006,1028,27873],[2007,1285,27302],[2008,1910,23548],[2009,192,11644],[2010,89,26124],[2011,1703,87850],[2012,1065,14299],[2013,888,1131],[2014,493,44657],[2015,1545,73818],[2016,237,48238],[2017,1014,75763],[2018,1675,85219],[2019,880,25356],[2020,125,6059],[2021,1754,62084],[2022,928,93569],[2023,795,47369],[2024,1774,11361],[2025,327,6297],[2026,715,41358],[2028,1288,55451],[2029,1648,67237],[2030,1297,74592],[2031,74,30263],[2032,139,57131],[2033,1033,65959],[2034,615,72312],[2035,1755,96772],[2036,1212,58479],[2037,364,36835],[2038,921,55953],[2039,92,74249],[2040,1041,415],[2041,1581,80267],[2042,820,37392],[2043,1397,36352],[2044,1434,47085],[2046,1954,85658],[2047,1082,39839],[2050,918,61523],[2051,835,95566],[2052,774,83285],[2054,512,83362],[2056,1482,4382],[2058,1040,43362],[2059,1104,2276],[2060,1526,93207],[2061,327,23112],[2062,1890,15322],[2063,859,36475],[2064,789,72567],[2065,921,64117],[2066,1101,66559],[2068,1884,10417],[2069,581,95410],[2070,894,33213],[2071,250,62942],[2072,803,91821],[2073,380,8662],[2074,1221,15694],[2075,39,89059],[2076,2067,90425],[2077,236,5479],[2078,1198,80996],[2079,774,67278],[2080,1242,1104],[2081,1685,44489],[2082,1517,82651],[2083,873,35859],[2084,1888,46030],[2085,475,82307],[2086,486,39714],[2087,1831,44254],[2088,210,74175],[2089,542,25669],[2090,588,34010],[2091,1307,67867],[2092,825,82360],[2094,376,96049],[2095,1277,97380],[2096,297,53778],[2097,1822,44514],[2098,1399,64501],[2099,415,68087],[2100,1224,94606],[2101,293,99852],[2103,2099,5612],[2104,1248,20860],[2105,970,81168],[2106,1127,32298],[2107,634,82386],[2108,1871,38000],[2110,59,63254],[2111,172,39684],[2112,1720,41084],[2113,1534,64156],[2114,822,63619],[2115,127,65393],[2116,1849,37022],[2117,2097,42037],[2118,1647,55823],[2119,995,29746],[2120,1913,82983],[2121,876,36229],[2122,893,758],[2123,832,96313],[2124,431,88408],[2125,2088,78623],[2126,2009,26519],[2127,338,40406],[2128,103,90501],[2129,2036,41450],[2130,1504,9978],[2132,1745,91935],[2133,76,17718],[2134,1311,85216],[2135,679,14350],[2137,738,20665],[2138,1399,20064],[2139,832,93137],[2140,1737,42071],[2141,1824,33807],[2142,972,4444],[2143,528,21421],[2144,1652,32914],[2145,664,26677],[2146,2127,14260],[2147,583,42539],[2150,655,12790],[2151,1518,49807],[2152,26,99895],[2154,1127,90129],[2155,2088,10724],[2157,1755,90234],[2158,389,15370],[2159,820,7147],[2160,76,72030],[2161,625,19306],[2162,77,25219],[2163,1595,32735],[2164,301,85536],[2165,361,40261],[2166,724,98728],[2167,1240,89002],[2168,2073,43034],[2169,1341,42903],[2170,491,63403],[2171,57,63603],[2172,1351,36884],[2173,567,67782],[2174,1216,3041],[2175,1920,65484],[2176,347,41692],[2177,1429,25219],[2179,1433,17508],[2180,737,57202],[2182,849,96569],[2183,2028,37847],[2184,897,96962],[2185,416,97856],[2186,1751,23129],[2187,1735,33861],[2188,128,47193],[2189,1888,71635],[2190,361,57490],[2191,1228,29430],[2194,1044,70570],[2195,587,13200],[2196,1754,6485],[2198,54,77327],[2199,602,23783],[2200,1841,26517],[2201,2147,78894],[2202,1417,46689],[2204,49,78906],[2205,470,84505],[2206,1481,25696],[2207,1130,48980],[2208,1956,7444],[2209,1551,50956],[2210,578,88818],[2211,1178,63010],[2213,236,21738],[2214,1976,15048],[2215,1104,45769],[2216,1281,83852],[2217,1689,69746],[2220,1541,65594],[2221,343,26631],[2222,2049,35879],[2224,555,80181],[2225,1995,40282],[2226,386,88628],[2227,1732,84637],[2229,695,87567],[2230,899,24036],[2231,199,19188],[2232,1530,33176],[2233,272,2791],[2234,1628,85131],[2235,398,33172],[2236,47,21862],[2237,2170,25915],[2238,105,26907],[2239,1728,572],[2240,1167,91663],[2241,74,25912],[2242,1356,40347],[2243,356,18957],[2244,271,91023],[2245,440,30974],[2246,1201,54423],[2247,2043,36488],[2248,2118,31770],[2249,600,16691],[2250,1259,12393],[2251,2245,1665],[2252,1104,11180],[2253,1424,18993],[2254,1167,54008],[2255,2158,70666],[2256,1440,16165],[2257,1679,26669],[2258,905,57945],[2259,112,84377],[2260,235,24915],[2261,478,69037],[2262,1444,51962],[2263,2064,38840],[2264,892,57061],[2265,1623,56407],[2266,2252,21253],[2267,1815,99431],[2268,401,49900],[2270,96,77024],[2271,598,94859],[2272,1053,69142],[2273,1852,97407],[2274,342,77522],[2275,1395,46939],[2276,451,37878],[2277,607,59552],[2278,462,71503],[2279,1685,17677],[2280,589,42216],[2281,611,56919],[2282,265,90926],[2284,256,45223],[2285,1479,18716],[2286,1327,52390],[2287,1742,11209],[2288,689,68277],[2290,2273,36432],[2291,1018,35629],[2292,223,83694],[2293,1817,5884],[2294,1508,32686],[2295,1505,55057],[2296,945,38245],[2297,23,58566],[2298,1031,36133],[2299,1704,48927],[2300,2210,71398],[2301,1129,5067],[2302,1276,30229],[2303,1887,38785],[2304,370,37192],[2305,540,84507],[2306,800,21257],[2307,2098,32257],[2308,1313,75339],[2309,967,54497],[2310,912,80527],[2312,1210,81603],[2313,385,43130],[2314,2047,25846],[2315,590,84056],[2316,688,94244],[2317,1859,8779],[2318,1987,21996],[2320,265,75943],[2321,2205,49463],[2322,535,30152],[2323,2211,53115],[2324,2016,1415],[2326,1785,62830],[2327,30,70021],[2328,1603,38881],[2329,986,39737],[2330,1641,96114],[2331,1338,72018],[2332,2042,84298],[2333,1011,34807],[2334,360,67420],[2335,117,52159],[2336,1482,59001],[2337,428,52218],[2338,68,59890],[2339,538,50508],[2340,788,10416],[2342,67,27040],[2343,48,80009],[2344,2174,35749],[2345,909,70960],[2346,779,13885],[2347,186,59099],[2348,1615,45029],[2349,2064,19175],[2350,2190,71571],[2351,2192,40233],[2352,823,40711],[2353,740,7691],[2354,849,29868],[2355,1182,20149],[2356,413,22],[2357,127,535],[2358,1600,88561],[2359,1054,65346],[2360,1696,81406],[2361,1330,46468],[2363,63,27287],[2364,924,95023],[2365,1605,48973],[2366,567,7123],[2368,1839,35196],[2370,1412,31239],[2372,1004,23456],[2373,1357,40992],[2374,1121,42154],[2375,323,63239],[2376,898,29672],[2377,1063,41190],[2378,22,42261],[2379,268,25089],[2380,715,75359],[2381,1114,96805],[2382,1434,79549],[2383,108,7098],[2384,98,86415],[2385,583,42909],[2386,1200,54301],[2388,41,24773],[2389,385,87514],[2390,1012,31318],[2391,1694,55342],[2392,896,7497],[2394,1063,75025],[2395,2370,27069],[2396,236,48497],[2397,659,11157],[2398,2286,98605],[2399,1914,42499],[2401,669,21596],[2402,808,12260],[2403,1527,61302],[2404,179,26235],[2405,933,23640],[2406,119,1382],[2407,734,79860],[2408,1838,15482],[2409,946,29015],[2411,664,11753],[2412,2056,62811],[2413,626,13188],[2414,1351,57459],[2415,1983,99460],[2416,675,46198],[2418,1240,93550],[2419,6,46272],[2421,1624,85911],[2422,388,40957],[2423,1003,25908],[2424,592,99275],[2425,2207,75011],[2426,604,66442],[2427,947,72834],[2428,1246,21382],[2429,559,18264],[2430,1434,97381],[2431,752,92400],[2432,364,86338],[2433,1999,34787],[2434,1227,67430],[2435,948,18535],[2436,1449,72362],[2437,2299,41745],[2438,1731,18436],[2439,2282,97626],[2440,1253,63882],[2441,1039,51634],[2442,144,96181],[2443,1697,8031],[2444,2211,28048],[2445,319,43172],[2446,647,11220],[2447,2277,27965],[2448,2152,23742],[2449,1445,38732],[2450,258,7692],[2451,2356,9264],[2452,1560,72880],[2453,1640,1366],[2454,1194,32202],[2455,1974,68355],[2456,2047,68280],[2458,1422,90781],[2459,581,47033],[2460,675,23296],[2461,868,16627],[2462,952,11067],[2463,2108,4752],[2464,1615,22626],[2465,824,43802],[2466,2190,6377],[2467,1296,51916],[2468,1964,6370],[2469,775,21085],[2470,228,33651],[2472,327,59404],[2473,2043,71279],[2474,62,66551],[2476,857,33414],[2477,387,96140],[2478,1229,8346],[2479,1403,24832],[2480,2373,42358],[2482,1662,14269],[2483,2058,48394],[2484,507,40834],[2485,965,83026],[2486,1351,45402],[2487,1706,9879],[2488,1779,1459],[2489,356,96847],[2490,2179,62795],[2491,2461,10600],[2492,1002,61530],[2493,666,14680],[2494,1553,63045],[2495,2248,48608],[2496,2221,27636],[2497,1047,61415],[2498,2319,53978],[2499,1536,26480],[2500,2328,25793],[2501,2418,95671],[2502,1463,58406],[2503,961,69248],[2504,1122,10374],[2505,1308,85393],[2506,1822,14955],[2507,1715,49225],[2509,1355,24281],[2510,1039,68445],[2511,1316,55545],[2512,350,44577],[2513,732,47262],[2514,1865,92617],[2515,1135,70423],[2516,1782,87822],[2517,663,31405],[2518,2006,32428],[2519,1262,63578],[2520,1210,77216],[2521,2017,85354],[2522,1537,63824],[2523,1555,17628],[2524,2003,44208],[2525,2023,78656],[2526,2394,50286],[2528,1243,96070],[2529,414,63478],[2530,987,9960],[2531,1378,34812],[2532,1432,65078],[2533,457,23276],[2534,2319,40004],[2535,1401,80726],[2536,894,27648],[2537,1562,53033],[2538,1394,77230],[2539,970,9907],[2540,1053,28051],[2541,19,64449],[2542,608,64251],[2543,1515,84097],[2546,1542,84175],[2547,2013,89319],[2548,2161,37658],[2549,1675,7772],[2550,1890,66059],[2551,50,37409],[2552,1051,1164],[2553,2278,45024],[2554,1794,34924],[2555,1943,20888],[2556,944,28408],[2557,1584,41994],[2558,718,69154],[2559,652,1888],[2560,1831,70812],[2561,2130,51143],[2562,2215,73829],[2563,922,76705],[2564,864,94547],[2565,1758,87738],[2567,517,68963],[2568,717,7581],[2571,143,90313],[2572,1960,83697],[2573,74,12600],[2574,2134,94481],[2575,2540,6518],[2576,2190,92478],[2578,702,4206],[2579,2425,90726],[2580,315,14878],[2581,1964,78276],[2582,924,54431],[2583,569,20401],[2584,1761,85177],[2585,1018,42050],[2586,1593,56464],[2588,1991,97069],[2590,841,10080],[2591,1971,37384],[2592,617,7104],[2593,522,46969],[2594,2395,65578],[2595,2240,6128],[2597,2066,83651],[2598,1741,48750],[2599,1504,92027],[2600,335,87154],[2601,2347,92227],[2602,664,62025],[2603,95,20963],[2604,1197,46625],[2605,1540,83791],[2606,628,87327],[2607,1432,60867],[2608,101,42892],[2609,334,8383],[2610,1413,58083],[2611,1085,78445],[2612,2321,33942],[2613,1056,11268],[2614,1188,18914],[2615,1075,72908],[2616,1668,9010],[2618,1300,36228],[2619,1026,13086],[2620,588,56003],[2623,363,20487],[2624,1542,34341],[2625,1833,2201],[2626,1734,85086],[2627,1725,89966],[2629,1029,92348],[2632,598,14192],[2633,1981,49106],[2634,1768,72203],[2635,1573,34944],[2636,1860,87831],[2637,846,98362],[2638,2096,84341],[2639,662,13918],[2640,577,29953],[2641,1683,73658],[2642,2229,29281],[2643,1842,14910],[2644,1510,81840],[2645,1991,30275],[2646,1345,9344],[2647,1667,93732],[2648,1284,29243],[2649,1699,85081],[2650,22,83780],[2651,1394,66418],[2652,1015,35651],[2653,1469,64406],[2654,600,82550],[2655,648,84100],[2656,2353,13051],[2657,419,62622],[2658,1651,6344],[2659,1868,16077],[2660,332,86527],[2661,1171,82300],[2662,2499,32359],[2663,2027,68743],[2664,182,83746],[2665,2243,9235],[2666,2032,53512],[2668,1268,67390],[2669,723,16227],[2670,965,33153],[2671,1922,85327],[2672,2249,99740],[2673,182,94633],[2674,1740,44404],[2675,1238,20920],[2676,2348,40695],[2677,48,64479],[2678,411,61922],[2679,2337,94677],[2680,1089,55313],[2681,2182,30949],[2682,969,53227],[2683,2390,58143],[2684,652,97218],[2686,515,76940],[2687,2135,75302],[2688,319,96301],[2689,2286,95808],[2690,435,36583],[2691,1132,6503],[2692,646,75733],[2693,1499,12339],[2694,833,83007],[2695,649,31958],[2696,2518,72649],[2697,1960,61155],[2698,2408,18814],[2699,775,74228],[2700,760,23797],[2701,283,32498],[2702,740,86475],[2703,1261,37304],[2704,2275,69997],[2705,258,70604],[2706,1917,75852],[2707,536,64069],[2708,2321,81200],[2709,2531,57164],[2711,327,96895],[2712,2254,69957],[2714,270,16641],[2715,1540,33124],[2717,2403,35088],[2718,2281,84550],[2719,1197,73393],[2720,1883,37467],[2721,988,42782],[2722,1990,26468],[2723,2003,16063],[2724,375,86438],[2725,1204,89374],[2726,2546,1936],[2727,1932,52121],[2728,1390,62303],[2729,1706,86152],[2731,1761,33651],[2732,1230,6977],[2733,76,95067],[2734,2000,95975],[2735,264,35914],[2736,2365,77006],[2737,1355,12647],[2738,1227,41371],[2739,2307,99639],[2740,64,68536],[2741,2366,55897],[2742,2337,71260],[2743,45,32627],[2744,327,40949],[2745,470,4310],[2746,2466,51553],[2747,1944,62245],[2748,1420,35086],[2749,1893,14499],[2751,434,56966],[2752,2076,94480],[2753,2542,8256],[2754,773,57965],[2755,329,65189],[2756,84,25222],[2757,1397,83854],[2758,2043,36443],[2759,1244,10089],[2760,1746,42979],[2761,347,21019],[2762,1271,10553],[2763,2187,22546],[2764,2318,23385],[2765,548,64707],[2766,1766,11862],[2768,2391,24022],[2769,1863,31695],[2770,837,44249],[2771,1466,10709],[2772,1611,68793],[2773,987,8901],[2774,2088,73341],[2775,438,57932],[2776,2201,91214],[2777,1358,553],[2778,1269,74893],[2779,2431,60977],[2780,382,19302],[2781,2563,49586],[2782,1984,68877],[2783,1821,48740],[2784,2524,39449],[2785,2732,97095],[2786,2528,88498],[2787,1286,30514],[2788,409,43543],[2789,2073,98394],[2790,1700,39577],[2791,408,70167],[2792,1684,57704],[2795,1469,90786],[2796,878,85015],[2797,1708,67324],[2798,1706,40806],[2799,752,30015],[2801,1375,45780],[2802,647,18330],[2803,2604,87326],[2804,461,80397],[2805,840,38348],[2806,2182,86475],[2807,2120,91533],[2808,103,35056],[2809,602,72193],[2810,2714,39877],[2812,2760,95230],[2813,1813,57177],[2814,2471,60877],[2815,2646,55638],[2816,2528,20359],[2817,1931,16778],[2818,712,21177],[2819,910,10437],[2821,1687,84842],[2822,152,26083],[2823,2457,82774],[2824,2002,93186],[2825,2147,79863],[2826,2799,45782],[2827,773,63145],[2828,332,14836],[2829,2066,610],[2830,1659,78608],[2832,1449,99393],[2833,863,54492],[2834,547,74311],[2835,2678,96063],[2836,2677,80562],[2837,1321,97784],[2838,2714,42661],[2839,2476,29815],[2840,2164,96993],[2841,1217,46391],[2842,2379,73203],[2843,4,28838],[2844,551,84612],[2846,1684,94484],[2847,953,73994],[2848,2238,26708],[2849,832,99553],[2850,139,70917],[2851,1563,32108],[2852,316,92159],[2853,2787,19647],[2854,58,22121],[2855,2840,3138],[2857,1534,65115],[2858,2081,38630],[2859,1161,13128],[2860,445,48378],[2861,2541,77884],[2862,1537,5487],[2863,1150,89104],[2864,1505,76007],[2865,2571,40578],[2866,679,51896],[2867,71,83860],[2868,2813,26011],[2869,2813,89432],[2870,817,88437],[2871,901,59356],[2872,2408,34897],[2873,1106,70952],[2874,1609,69907],[2875,1678,40059],[2876,1818,56692],[2877,2482,57094],[2878,362,46501],[2879,895,11008],[2880,2078,15225],[2881,889,27381],[2882,1922,77961],[2883,1075,72588],[2884,1058,35070],[2885,1887,13611],[2886,995,81219],[2887,80,49953],[2888,2139,40932],[2889,2231,49462],[2890,2820,22780],[2891,2703,70125],[2892,2113,58344],[2894,1286,73236],[2895,1012,26196],[2897,531,91004],[2898,1802,73830],[2899,1956,55056],[2900,652,92683],[2901,2168,1372],[2902,2553,4472],[2903,2437,37829],[2904,2337,20066],[2905,2848,2323],[2906,2039,48845],[2907,2485,18300],[2909,2371,75854],[2910,754,67836],[2911,2539,54462],[2912,2062,79174],[2913,2240,25605],[2914,2429,9575],[2915,163,21750],[2916,2807,64852],[2917,490,87046],[2918,2489,58062],[2920,61,60506],[2921,354,79503],[2922,229,53732],[2923,607,19449],[2924,1647,51600],[2925,313,56558],[2926,2697,20875],[2927,1073,65261],[2929,1385,48076],[2930,2663,19917],[2931,1023,39833],[2932,775,3185],[2934,280,23413],[2935,260,94899],[2936,2478,39391],[2937,2119,43515],[2938,2834,60872],[2939,2358,1641],[2940,2806,68296],[2941,2731,72863],[2943,64,22808],[2944,416,8222],[2945,2052,5819],[2946,1905,6441],[2947,1013,87893],[2948,1026,93970],[2949,1773,35815],[2950,883,36067],[2951,2149,26614],[2952,2409,4923],[2953,771,75509],[2954,1237,12899],[2955,2715,33197],[2956,2107,53875],[2957,2444,70457],[2958,1701,41994],[2960,1347,49932],[2961,773,15019],[2962,1041,218],[2963,1080,56278],[2964,1298,29707],[2965,1011,40091],[2966,2356,69712],[2967,1726,82907],[2968,688,33570],[2969,695,51621],[2971,2629,87159],[2972,599,89368],[2973,2407,49259],[2974,2537,83163],[2976,1782,34018],[2977,2179,68586],[2978,1582,19019],[2979,1100,64615],[2980,1350,50347],[2981,1396,47793],[2982,402,81882],[2983,790,20394],[2984,2001,48884],[2985,1091,48443],[2986,1857,43079],[2987,2644,73813],[2988,2141,10709],[2989,928,91873],[2991,2938,16369],[2992,2184,90164],[2993,2161,2020],[2994,434,20338],[2995,1724,30787],[2996,533,17593],[2998,1653,11745],[2999,2532,82697],[3000,2326,30695],[3001,2826,59220],[3002,2124,78239],[3004,1747,93140],[3005,2148,98238],[3006,2708,33550],[3007,906,83585],[3008,1068,93936],[3009,1806,79028],[3010,2467,62311],[3011,682,88722],[3012,228,54395],[3013,2800,79550],[3014,1774,57134],[3015,2116,84479],[3016,2556,37893],[3017,1806,98988],[3018,2945,79246],[3019,509,43963],[3020,1309,20350],[3021,2436,15082],[3022,1617,36333],[3023,1860,99281],[3024,2644,34360],[3025,751,28374],[3026,1437,32846],[3028,2837,61706],[3029,705,25192],[3030,275,43364],[3031,2503,45055],[3032,2371,14180],[3033,2576,32264],[3034,1372,7674],[3035,1513,45894],[3036,974,97164],[3037,23,67837],[3038,377,11637],[3039,1916,47094],[3040,1769,79449],[3041,2917,8430],[3043,508,88442],[3044,761,80340],[3045,1263,301],[3046,348,91114],[3047,2316,72971],[3048,2050,97349],[3049,12,97969],[3050,1051,27859],[3051,360,68515],[3052,2348,23083],[3053,2582,89949],[3054,688,34629],[3055,1848,79616],[3056,17,39676],[3057,879,38293],[3058,1940,27345],[3059,927,68597],[3062,185,24216],[3063,1547,94216],[3066,432,21647],[3067,1867,20078],[3068,2075,55500],[3069,1906,16376],[3070,525,62096],[3071,888,75854],[3072,1516,47631],[3073,47,62821],[3074,1559,30041],[3075,659,68902],[3076,2117,14225],[3077,327,18657],[3078,1294,57081],[3079,576,56554],[3080,884,19932],[3081,553,39294],[3082,2996,56843],[3083,2834,25118],[3086,2149,36064],[3087,316,42579],[3088,188,2003],[3090,2169,24882],[3091,1387,71042],[3092,2680,79002],[3093,1816,62914],[3094,618,96632],[3095,1050,55352],[3096,344,90483],[3097,635,53374],[3098,2430,50032],[3099,2451,75821],[3100,1617,5889],[3101,33,89733],[3102,1617,26300],[3104,1586,10632],[3105,823,31817],[3107,2584,71895],[3108,2589,87575],[3109,2292,92150],[3110,848,65160],[3111,2248,69852],[3113,3053,32659],[3114,974,14721],[3116,2834,35874],[3117,8,91945],[3118,73,19695],[3119,1538,44706],[3120,2598,20790],[3121,2330,56978],[3122,2368,71358],[3123,2357,42135],[3124,848,30512],[3125,1457,51209],[3126,319,96467],[3127,895,53525],[3128,2528,36009],[3130,2571,92236],[3131,371,49483],[3132,3109,56231],[3133,2023,19639],[3134,3100,21763],[3135,697,5709],[3136,1863,42836],[3137,529,21400],[3138,3034,97660],[3139,2253,10962],[3140,1259,87933],[3143,1500,54626],[3144,2548,25970],[3145,2210,92324],[3146,252,85466],[3147,287,88919],[3148,1942,23633],[3149,1087,5713],[3150,2948,56361],[3151,1245,50796],[3152,1859,33181],[3153,824,4446],[3154,1878,65672],[3155,1628,57225],[3156,2904,34928],[3157,2224,12629],[3158,1103,97392],[3159,1260,38375],[3160,1537,84681],[3161,1707,41393],[3163,1475,91040],[3164,189,95152],[3166,1657,97315],[3167,79,31369],[3168,1744,90008],[3169,2034,91326],[3170,1147,53816],[3171,1854,58034],[3172,2274,450],[3173,816,48380],[3174,2587,9597],[3175,954,70783],[3176,135,6654],[3177,2141,7171],[3181,1789,25016],[3182,851,90583],[3183,38,65439],[3184,1712,74856],[3185,3093,41971],[3186,1224,63961],[3188,129,44384],[3189,1150,34862],[3190,1758,15131],[3191,1121,71112],[3192,1763,95579],[3193,1028,94139],[3195,2893,51059],[3196,679,5746],[3197,409,84894],[3198,1738,15322],[3199,3055,93065],[3200,1572,28421],[3201,1823,6971],[3202,1843,33299],[3203,1925,79819],[3204,974,9706],[3205,3066,86966],[3208,809,48947],[3209,3187,62960],[3210,2912,4729],[3211,1944,18893],[3213,1117,66579],[3214,3158,53475],[3215,372,883],[3216,318,1179],[3217,2458,46409],[3218,58,18710],[3219,609,62311],[3220,1125,19743],[3221,1799,39470],[3222,3092,22968],[3223,2049,83388],[3224,2426,2763],[3225,3160,90545],[3226,1933,66201],[3227,844,82480],[3228,809,98101],[3229,2884,23547],[3230,457,77420],[3231,262,1701],[3232,755,87892],[3233,1964,59958],[3234,2642,92938],[3235,2104,28100],[3236,2413,92779],[3237,2315,64392],[3238,2092,12197],[3239,49,69726],[3240,2615,59115],[3242,10,85186],[3243,1328,82281],[3244,2188,53013],[3245,634,53104],[3247,930,30398],[3248,384,44493],[3250,1367,22060],[3251,1206,36977],[3252,1329,7650],[3253,1424,49233],[3254,1415,72639],[3255,93,23337],[3256,1265,97493],[3257,244,25086],[3258,260,90004],[3259,2687,94801],[3260,1750,20576],[3261,613,9644],[3262,2909,69692],[3263,2065,25738],[3264,1456,98518],[3265,2333,40368],[3266,1572,60098],[3267,3238,60569],[3268,2287,12856],[3269,2089,19048],[3270,2619,23465],[3271,1010,50472],[3272,942,93751],[3274,2503,57124],[3275,74,96184],[3277,1750,69257],[3278,2098,61635],[3279,260,94388],[3281,3073,60568],[3282,1222,43146],[3283,424,95780],[3284,2125,93347],[3285,3046,62725],[3286,652,41248],[3287,2080,32226],[3288,664,43953],[3290,937,62492],[3291,1040,48315],[3292,1398,29933],[3293,1245,76725],[3294,2028,52130],[3295,1642,64903],[3296,1795,97892],[3297,1164,73351],[3298,722,26379],[3299,1461,32931],[3300,1966,19075],[3301,2676,9365],[3303,2802,18662],[3304,2836,68444],[3305,2165,93712],[3306,379,97847],[3307,1998,84565],[3308,2693,34501],[3309,2137,76628],[3310,1000,44035],[3311,1678,33968],[3312,1695,69487],[3313,2993,12707],[3314,2866,46946],[3316,438,7058],[3317,2153,40084],[3318,2868,93170],[3319,2329,13343],[3320,2695,13691],[3321,556,97239],[3322,2591,81578],[3323,2576,4165],[3324,2383,64189],[3325,3209,85905],[3326,1942,81772],[3327,2748,64606],[3328,1386,36558],[3329,3120,17546],[3330,2069,17190],[3331,1123,466],[3332,211,53423],[3333,1629,28894],[3334,2098,11367],[3335,2840,83674],[3336,2410,88707],[3337,1648,21004],[3339,405,16926],[3340,124,64763],[3341,2913,12495],[3342,2991,26757],[3344,406,28832],[3345,2406,980],[3346,2986,92894],[3347,1200,49020],[3348,56,19755],[3349,2445,46785],[3350,2101,79299],[3351,1557,36982],[3352,1859,83099],[3353,1355,42988],[3354,207,42176],[3356,1792,82788],[3357,1473,84254],[3358,1776,95151],[3359,1788,74468],[3360,1233,83742],[3361,1561,26379],[3362,2240,33217],[3363,507,28770],[3364,283,12936],[3365,1057,45774],[3366,835,96798],[3367,1008,83671],[3368,3289,108],[3369,2168,74746],[3371,964,19246],[3372,642,48799],[3373,1788,54091],[3374,1949,65102],[3375,1140,44020],[3376,1758,92374],[3377,2973,45605],[3378,2479,33869],[3379,2088,94400],[3380,3219,70649],[3381,326,48578],[3382,2520,68471],[3383,1208,41784],[3384,2937,9450],[3385,1850,64663],[3386,1979,92198],[3387,234,76816],[3388,2858,82522],[3389,2220,81109],[3390,67,71734],[3391,2453,79661],[3392,554,55325],[3393,1909,82062],[3395,2617,35789],[3396,938,90402],[3397,1299,45780],[3398,359,27449],[3399,1733,96014],[3400,2498,41074],[3401,2888,1336],[3402,177,66744],[3403,1339,29456],[3404,69,57377],[3405,48,58073],[3406,1802,47282],[3407,233,11521],[3408,489,87461],[3409,884,95465],[3410,973,56958],[3411,169,94410],[3412,2072,15806],[3413,570,32683],[3414,2657,31539],[3415,2527,85253],[3416,3137,99539],[3417,325,82816],[3418,1192,80081],[3419,1926,60073],[3420,2077,32543],[3421,22,23833],[3422,735,75905],[3423,1285,5210],[3424,3201,50480],[3425,1827,35110],[3426,1656,43200],[3427,2039,82505],[3430,257,67044],[3431,2407,26188],[3432,1297,50337],[3433,1848,93268],[3434,91,14108],[3435,1063,91933],[3437,1590,8915],[3438,2399,93310],[3439,489,13138],[3440,394,67362],[3441,30,81810],[3442,2356,49224],[3443,2276,77182],[3444,2351,93822],[3445,594,36842],[3446,1409,25980],[3447,297,47261],[3448,1204,49861],[3449,870,22214],[3450,1171,1576],[3451,660,49519],[3452,118,74775],[3454,1480,10442],[3455,3371,20530],[3456,869,84837],[3457,708,66105],[3458,872,61463],[3459,3286,12768],[3460,1125,38958],[3461,2524,49668],[3462,1167,25637],[3463,1205,75427],[3464,675,90240],[3465,185,2292],[3467,1790,53814],[3469,206,52144],[3470,2598,75050],[3471,2987,89165],[3472,1576,9993],[3473,2078,26059],[3474,2375,90693],[3476,572,34282],[3477,2796,9638],[3478,2931,62045],[3479,3303,76287],[3480,1979,70185],[3482,3421,84873],[3483,2306,49619],[3484,3104,11669],[3485,2657,68153],[3486,415,30779],[3487,1796,61328],[3488,1879,23852],[3489,2712,89756],[3490,275,3273],[3491,1303,35388],[3492,2810,80368],[3493,3484,62759],[3494,2422,74946],[3495,1767,14508],[3496,264,19372],[3497,1576,21623],[3498,1012,15448],[3499,1548,58187],[3500,416,38321],[3501,2220,13378],[3502,446,87541],[3503,82,1865],[3504,1134,42808],[3505,2016,20401],[3506,2841,80424],[3507,2580,26688],[3508,2181,24502],[3509,164,93502],[3510,1267,95267],[3511,2991,91112],[3512,1292,55496],[3513,3455,23876],[3514,1076,75784],[3515,820,39656],[3517,1801,31571],[3518,2352,86290],[3519,2128,83237],[3520,1068,2414],[3521,2173,74086],[3522,1834,73534],[3523,1425,75305],[3524,1370,69685],[3525,454,94912],[3526,2898,52987],[3527,988,14667],[3528,1283,88722],[3529,1605,75675],[3530,972,94111],[3531,2700,85228],[3532,2330,33779],[3533,1025,62708],[3534,1381,46354],[3536,3371,57234],[3537,2006,67352],[3538,23,4372],[3540,741,38646],[3542,1095,53440],[3543,1179,49190],[3544,3496,33137],[3546,2919,29525],[3548,2494,18931],[3549,1336,96001],[3550,3323,91737],[3551,531,29542],[3552,27,79772],[3553,3548,33467],[3555,305,15549],[3556,28,44026],[3557,864,56832],[3558,2421,56672],[3559,617,70683],[3560,3547,13795],[3561,2649,73009],[3562,633,47672],[3563,1368,62712],[3564,2670,58142],[3565,1558,32935],[3566,854,60464],[3567,14,19081],[3568,267,40693],[3569,1655,19564],[3570,3356,13661],[3571,514,87357],[3572,1205,49957],[3575,699,87845],[3576,1088,30266],[3577,2862,49124],[3578,1967,81391],[3579,1438,46408],[3580,1533,79956],[3581,2416,34856],[3582,670,20184],[3583,2242,44064],[3585,3344,87002],[3586,1309,9411],[3587,755,57327],[3588,1785,56463],[3589,3211,91044],[3590,290,82200],[3591,2276,91437],[3593,2102,22798],[3594,2008,69323],[3595,686,23436],[3596,3351,4906],[3597,3546,83299],[3600,1184,95647],[3601,2574,42958],[3602,3499,38914],[3603,1265,62489],[3604,2320,33287],[3605,1841,84036],[3606,1280,50669],[3607,447,47536],[3608,2550,17145],[3609,1187,97385],[3610,96,19345],[3611,2171,60114],[3612,2886,54692],[3613,1520,73070],[3614,323,9019],[3615,2584,54989],[3617,2814,88392],[3618,1733,66165],[3619,2624,8843],[3620,2106,32270],[3622,265,29192],[3623,3425,56467],[3625,2884,97306],[3626,1585,13221],[3627,1403,60254],[3628,1200,90450],[3629,1545,91570],[3630,2804,52200],[3631,917,77736],[3632,100,77883],[3633,1115,22689],[3634,999,21342],[3635,1043,20843],[3636,1559,56899],[3637,2369,63909],[3638,2761,69668],[3639,1027,15860],[3640,988,79657],[3641,1388,44609],[3642,2372,18514],[3643,300,48309],[3644,99,57143],[3645,1484,70446],[3646,2045,98133],[3647,3220,82653],[3648,3310,40269],[3649,2992,42755],[3651,2972,39275],[3652,3420,90900],[3653,2497,10731],[3654,658,61702],[3655,904,2498],[3656,1494,90217],[3657,930,91344],[3658,2109,44310],[3659,3142,22793],[3660,3138,53733],[3661,1979,97458],[3662,2133,82905],[3663,3272,2581],[3664,1649,19201],[3665,1556,4443],[3666,591,42292],[3667,3493,29760],[3668,56,36777],[3669,893,93661],[3670,1070,63525],[3671,3662,21044],[3673,2485,79249],[3674,2457,64931],[3675,2895,3719],[3678,1283,51979],[3679,3156,89599],[3680,797,67756],[3681,2686,51118],[3682,2808,14871],[3683,1830,41670],[3684,456,42766],[3685,1338,51433],[3686,2604,9311],[3687,3530,12687],[3688,916,69899],[3689,2437,79447],[3690,1862,49152],[3691,2310,15386],[3692,2462,83019],[3693,549,86562],[3694,1533,56715],[3695,2488,74290],[3696,1537,53152],[3697,3188,85429],[3698,703,63044],[3699,852,39692],[3700,609,61496],[3701,3079,33536],[3702,2206,84661],[3703,343,53574],[3704,725,35999],[3705,1816,79090],[3706,363,21362],[3707,2382,58535],[3708,236,91097],[3709,1194,15819],[3710,861,48684],[3711,577,4794],[3713,3512,1608],[3714,2445,93739],[3715,193,97938],[3716,3334,9433],[3717,2000,12679],[3718,2611,91459],[3719,3436,21432],[3720,1083,75220],[3721,2897,18274],[3722,424,94215],[3723,947,59107],[3724,1422,83036],[3725,954,60170],[3728,909,66230],[3729,1053,17192],[3730,2195,95386],[3731,381,21092],[3732,1289,89974],[3735,2281,59387],[3736,127,25665],[3737,1862,13252],[3738,148,25756],[3739,616,28421],[3740,2857,75064],[3741,2779,33008],[3742,2238,74398],[3743,2945,14985],[3744,3032,46830],[3745,3610,9626],[3746,1574,76696],[3747,922,19118],[3748,973,39071],[3749,3309,19993],[3750,3184,34058],[3751,1745,71511],[3752,3593,62736],[3755,3578,10730],[3756,21,35382],[3757,2291,97658],[3758,1696,88815],[3759,639,30528],[3760,3000,76453],[3761,1656,7968],[3762,3628,71707],[3763,3657,65479],[3764,2340,72676],[3765,2561,3587],[3766,246,68916],[3767,3561,2636],[3768,3465,5527],[3769,3006,14864],[3770,2288,88196],[3771,3400,40691],[3772,326,36778],[3773,2918,6969],[3774,2637,27141],[3775,3693,22016],[3776,1353,12059],[3777,2678,94738],[3778,1843,3679],[3779,3296,80614],[3780,2687,40657],[3781,1081,67667],[3782,1359,54612],[3783,460,96966],[3784,3664,56807],[3786,2162,56052],[3787,3588,73236],[3788,3717,4903],[3789,3743,86928],[3790,3051,39326],[3791,3198,66023],[3792,3777,76118],[3793,472,76117],[3794,394,78692],[3795,967,32393],[3796,1847,13584],[3797,348,57701],[3798,3737,80847],[3799,344,81462],[3802,1323,79037],[3803,2429,72232],[3804,120,2458],[3805,2871,5978],[3806,2975,39932],[3807,2430,63218],[3809,1709,9146],[3810,2221,8640],[3812,2647,46347],[3813,1457,28712],[3814,2009,54263],[3815,3043,7567],[3816,2701,26077],[3817,3591,73175],[3818,644,20474],[3819,1569,20185],[3820,68,3894],[3821,107,99122],[3823,1187,26413],[3825,1319,33691],[3826,3274,31185],[3827,969,22446],[3829,1536,98364],[3830,3150,63412],[3831,2369,1112],[3832,1613,66906],[3833,1272,67208],[3834,1852,31115],[3835,3567,57385],[3836,2362,69807],[3838,2168,71361],[3839,3817,18780],[3840,576,24790],[3841,3324,58467],[3842,3307,31189],[3843,2376,68810],[3844,2664,88265],[3845,931,18722],[3846,97,34500],[3847,361,9710],[3848,599,71459],[3849,1176,77035],[3850,1936,68117],[3852,3657,11396],[3853,3091,59487],[3854,2735,72957],[3855,2875,19094],[3856,3801,26955],[3857,2192,6781],[3859,3639,6985],[3860,2704,6717],[3861,1253,11713],[3862,1212,24105],[3863,1518,19094],[3864,3363,73074],[3865,1546,99383],[3866,96,56970],[3867,2278,79158],[3868,1268,61393],[3869,1321,35313],[3870,2217,71620],[3871,1860,13117],[3872,843,5948],[3873,855,8750],[3874,1718,55058],[3875,2887,94639],[3876,3544,43490],[3877,2923,638],[3879,3695,10598],[3880,3031,61903],[3882,3734,26082],[3883,2542,46362],[3884,2839,22984],[3885,2583,43887],[3886,1495,40643],[3887,382,48201],[3888,199,22823],[3889,2188,78334],[3890,3311,19151],[3891,1745,48371],[3892,3564,47236],[3893,381,19483],[3894,807,57306],[3895,2609,76404],[3896,3222,73469],[3897,788,93735],[3898,2759,74483],[3899,2994,16942],[3900,3273,18994],[3901,2300,10533],[3903,2092,37765],[3904,328,553],[3905,2715,51846],[3907,3297,65185],[3908,700,46412],[3909,1238,69419],[3910,3761,6276],[3911,1095,74179],[3912,3392,1161],[3914,2176,21145],[3915,1197,74627],[3916,3079,29023],[3917,2645,11243],[3918,2728,85369],[3919,2393,60921],[3920,3465,57860],[3921,2101,21654],[3923,3304,41330],[3924,2946,60025],[3925,1873,42635],[3926,1413,45969],[3927,3256,66332],[3928,1439,61395],[3929,289,79252],[3930,3656,89208],[3931,3858,77853],[3932,1093,91719],[3933,2830,2587],[3934,651,89267],[3935,3279,63816],[3936,1368,91665],[3937,1379,38454],[3938,279,47547],[3939,3517,15944],[3940,3770,48238],[3941,2233,79795],[3942,2754,24926],[3943,2402,45565],[3944,1349,95291],[3945,79,1223],[3946,1825,16430],[3947,3210,65047],[3949,955,79140],[3950,2429,40738],[3952,3150,78404],[3953,3448,79872],[3954,3569,88142],[3955,504,39305],[3956,1230,32611],[3957,3138,5490],[3958,1071,39991],[3959,3400,85697],[3960,3377,41354],[3961,2039,6352],[3962,1005,58712],[3963,3693,38249],[3964,776,79622],[3965,3181,92467],[3966,215,796],[3967,772,83179],[3968,243,82238],[3969,2796,17774],[3970,2029,90607],[3972,3662,54904],[3973,1919,89254],[3974,860,50862],[3975,3243,29169],[3976,3962,40192],[3977,1603,82940],[3978,3003,80670],[3979,739,10382],[3980,3137,10753],[3981,2398,37564],[3982,227,67638],[3983,294,42361],[3984,2993,94478],[3985,1468,16868],[3986,2571,77617],[3987,1999,43462],[3988,1995,76704],[3989,92,13272],[3990,663,99448],[3991,1487,79477],[3992,2914,82917],[3993,1539,73134],[3994,3459,67663],[3995,2338,35733],[3996,908,85458],[3997,2200,60592],[3998,2727,36623],[3999,2355,74363],[4000,3471,21603],[4001,1559,43326],[4003,3143,60300],[4004,245,40262],[4005,2002,92189],[4006,2428,61698],[4008,2199,7078],[4009,971,19357],[4010,2955,3945],[4011,964,41815],[4012,2546,56596],[4013,1899,21979],[4014,1585,53271],[4015,639,76443],[4016,2082,53533],[4018,793,39530],[4019,290,37896],[4020,855,88957],[4021,3182,33349],[4022,2110,65296],[4023,3278,18916],[4024,2915,83501],[4025,3821,73026],[4026,58,19607],[4027,3275,16043],[4028,1747,72271],[4029,2905,4747],[4030,1310,813],[4031,1588,66157],[4032,496,31048],[4033,2280,36037],[4034,2325,82823],[4035,3543,87996],[4036,3705,52972],[4037,551,82037],[4038,2981,59],[4040,3282,2146],[4041,1009,6733],[4042,1071,90838],[4043,3897,89881],[4044,3039,66975],[4045,1509,11347],[4046,1613,52025],[4047,2990,64214],[4048,443,43884],[4049,2104,95352],[4050,2822,37316],[4051,2728,14041],[4052,89,72483],[4053,2466,9060],[4054,1690,9186],[4055,1945,93221],[4056,1512,18894],[4057,2568,51715],[4058,2617,3856],[4059,3001,88220],[4060,3224,41145],[4061,3732,35989],[4062,225,79601],[4063,3781,72751],[4064,3036,15986],[4065,424,6594],[4066,3196,26008],[4067,3789,49333],[4068,2747,25262],[4069,1883,22894],[4070,2316,87109],[4071,992,14875],[4072,879,57429],[4073,4072,59521],[4074,2754,41918],[4075,2111,71597],[4076,1688,30236],[4077,3384,90077],[4078,2981,65648],[4079,236,34770],[4080,3839,28219],[4081,2598,61732],[4082,2019,53737],[4083,624,93423],[4084,1209,98007],[4085,751,86818],[4086,3453,80489],[4087,2643,39903],[4088,1490,63171],[4090,799,90754],[4091,4053,27722],[4093,928,39807],[4094,2316,34790],[4095,368,11036],[4096,135,33207],[4097,1549,41433],[4098,3335,99635],[4099,1880,49194],[4100,2884,66534],[4101,3128,63322],[4102,2419,86590],[4103,58,39372],[4104,3562,71576],[4105,3143,3493],[4106,1679,97104],[4107,1700,20051],[4108,2830,45773],[4109,3984,53944],[4111,719,67410],[4112,2521,38268],[4113,3392,87714],[4114,1406,74606],[4115,2666,36729],[4116,2387,68061],[4117,2836,37102],[4118,1064,63699],[4119,1614,22294],[4120,2535,14337],[4121,898,69449],[4122,1076,52385],[4123,3470,12117],[4124,2831,94940],[4125,1174,24997],[4126,2036,99118],[4127,3200,89958],[4128,2046,55828],[4129,1692,87817],[4132,3821,40207],[4133,1171,64598],[4134,641,66046],[4135,3970,41865],[4137,852,59519],[4138,465,64004],[4139,1238,33550],[4140,3576,43899],[4141,3500,6279],[4144,485,18339],[4145,1714,63519],[4146,1948,57759],[4147,1521,44510],[4148,1394,55604],[4149,1605,1167],[4150,351,62936],[4151,521,90979],[4153,2200,58052],[4154,3292,65190],[4155,1771,69596],[4156,723,93105],[4157,3129,18569],[4158,2470,8438],[4159,2071,49332],[4160,1799,97400],[4161,1205,68214],[4163,205,13080],[4164,338,33691],[4165,3628,84742],[4166,2956,66511],[4167,1556,59472],[4168,1084,69170],[4169,1268,59075],[4170,1810,19483],[4172,3896,81682],[4173,937,22616],[4174,2504,44350],[4175,2220,37224],[4176,1941,17867],[4177,498,53982],[4178,1801,34655],[4179,3075,59824],[4180,3614,8045],[4181,2653,25955],[4182,1730,62178],[4183,2766,91087],[4184,895,48664],[4186,3680,58718],[4187,3478,47271],[4188,2527,42961],[4189,3202,8910],[4190,1827,31670],[4191,1703,30580],[4194,2618,75986],[4195,2030,30444],[4196,1364,64505],[4197,3896,32169],[4199,2066,17746],[4200,892,20158],[4201,2877,49520],[4202,2052,22267],[4203,3149,66910],[4204,3730,67198],[4205,2646,39549],[4206,672,43246],[4207,2494,46777],[4208,45,70802],[4209,1654,57895],[4210,1625,76484],[4211,2577,9493],[4212,2401,83654],[4213,3196,2402],[4214,1671,71589],[4215,1068,64090],[4216,627,85071],[4217,1291,49538],[4218,3944,16237],[4219,1232,3114],[4220,2111,89923],[4221,3928,98567],[4222,3121,36708],[4224,2094,23580],[4227,836,84529],[4229,835,60463],[4231,3030,12008],[4232,241,30090],[4234,3434,55277],[4235,2368,42874],[4236,858,30696],[4237,2747,86463],[4238,3353,74617],[4239,2485,51407],[4240,1556,79124],[4241,2298,90272],[4242,16,77236],[4243,2021,69991],[4245,3718,48289],[4247,1163,78063],[4248,206,66994],[4249,1322,82798],[4250,3518,62748],[4251,3502,79971],[4252,1257,6872],[4253,850,7087],[4254,848,57612],[4255,1766,5715],[4256,1058,18161],[4257,3911,50140],[4258,3536,91031],[4259,2443,60484],[4260,3624,57841],[4261,2725,68828],[4262,989,94101],[4263,1882,55022],[4264,1208,14452],[4265,1656,49761],[4266,2310,14024],[4267,3058,31512],[4268,277,23040],[4269,2008,71234],[4271,324,24133],[4272,2403,62267],[4273,851,38536],[4274,3452,39281],[4275,3394,56532],[4276,1804,79360],[4277,1696,77537],[4278,124,23848],[4279,95,77401],[4280,945,37779],[4281,4090,41005],[4282,3050,4807],[4283,2569,9323],[4284,3314,11527],[4285,1965,93936],[4286,1495,83767],[4287,1225,77663],[4289,4066,50016],[4290,4074,77779],[4291,493,53095],[4292,503,6493],[4293,1767,92389],[4295,4244,20611],[4296,4285,85660],[4297,2463,10756],[4298,1915,95772],[4299,3064,74699],[4300,2192,17275],[4302,300,17680],[4303,1714,35022],[4305,1905,73631],[4307,3067,89527],[4308,1335,14468],[4310,3692,87489],[4312,1845,75136],[4313,2392,20310],[4314,2910,67755],[4315,1100,14997],[4316,1476,64647],[4317,3200,85138],[4318,2164,70865],[4319,408,55414],[4320,3478,36587],[4321,3792,47655],[4322,1565,84721],[4323,131,19818],[4324,2528,96939],[4325,3714,8828],[4326,3832,87498],[4327,3003,15276],[4328,3698,64307],[4329,2579,18647],[4330,3801,45689],[4331,3006,79114],[4332,667,50156],[4333,4030,56301],[4334,2513,53898],[4335,4050,80022],[4336,1308,76297],[4337,3152,8868],[4338,1736,92037],[4339,3594,31128],[4340,2406,1394],[4342,2441,70987],[4343,2072,98916],[4344,1640,21178],[4345,876,35058],[4346,3059,68726],[4347,1118,78971],[4348,1238,64623],[4349,905,17986],[4350,3263,85715],[4351,1161,57519],[4352,2980,45358],[4353,3187,29635],[4354,694,22085],[4355,2711,85436],[4356,787,1578],[4357,992,22057],[4358,1224,46105],[4359,81,57485],[4360,459,79537],[4361,1693,62047],[4362,292,14978],[4364,1857,34165],[4365,3217,46229],[4366,1579,5933],[4367,3824,86154],[4369,3713,7242],[4370,3321,50022],[4371,2872,35516],[4373,2178,39010],[4374,2042,86551],[4376,2375,8467],[4377,787,73489],[4378,2403,56379],[4379,1914,90956],[4380,4257,85647],[4381,1935,94284],[4382,3171,16288],[4383,3382,41988],[4384,2763,58295],[4385,3805,46040],[4386,2432,1768],[4387,3221,66287],[4388,461,24341],[4389,1953,7813],[4390,84,61103],[4391,2737,51210],[4392,590,39207],[4393,210,63599],[4394,2401,9974],[4395,1884,10897],[4396,2420,59996],[4397,1340,50615],[4398,384,55183],[4399,424,42883],[4400,3978,13078],[4401,912,23087],[4402,2617,20615],[4403,1030,14129],[4404,382,40349],[4405,3049,69868],[4406,2964,83729],[4409,464,82718],[4410,4188,82978],[4412,3444,85089],[4413,3031,52607],[4414,4090,94311],[4415,3687,69941],[4416,178,92251],[4418,1856,4228],[4419,2055,76483],[4420,803,92778],[4422,4406,26011],[4423,3986,72193],[4424,107,99947],[4425,3522,36770],[4427,4235,41385],[4429,3273,82712],[4430,4215,8326],[4431,567,19115],[4432,3217,49162],[4434,3684,43100],[4435,3044,34762],[4437,138,64723],[4438,3228,1713],[4439,2951,35682],[4440,3574,789],[4441,1326,24618],[4442,2790,63670],[4443,2550,9869],[4445,3149,46109],[4447,1193,52798],[4448,3485,13264],[4449,1381,28896],[4450,2255,13934],[4452,1836,12208],[4453,3415,52990],[4454,520,33824],[4455,2579,5823],[4456,2628,49327],[4457,2292,54246],[4458,3552,93722],[4459,1558,83879],[4460,1321,82425],[4461,1648,58938],[4462,1484,1079],[4463,2872,24984],[4464,3088,114],[4465,536,86002],[4466,2073,80929],[4467,1735,29363],[4468,3341,50446],[4469,103,58466],[4470,2960,52544],[4471,2590,43081],[4472,745,44168],[4473,2094,23328],[4475,2818,59005],[4476,2387,60961],[4477,4019,92435],[4478,3036,88464],[4479,1970,87997],[4480,1706,73412],[4481,1218,7076],[4482,2152,70320],[4483,1911,23852],[4487,2150,14325],[4488,973,34189],[4489,2676,20275],[4490,741,33916],[4491,1235,45930],[4492,3364,15822],[4493,3690,12141],[4494,1351,33581],[4496,590,35664],[4497,3045,21647],[4498,3529,17859],[4499,4468,81353],[4500,3853,31373],[4501,3191,10768],[4502,2143,20841],[4503,1217,11124],[4504,1196,62285],[4505,2775,52912],[4506,848,51570],[4507,3981,52884],[4508,2460,55103],[4509,3533,21122],[4510,2255,43744],[4511,51,64411],[4512,2639,484],[4513,722,8182],[4515,3163,71306],[4516,532,11565],[4517,237,54118],[4518,1805,90299],[4519,4292,44668],[4520,368,49746],[4521,2191,57089],[4522,1284,35993],[4523,1445,54502],[4524,2341,89400],[4525,1991,71773],[4526,3467,92280],[4527,964,98318],[4528,3922,21296],[4529,3664,45621],[4530,1436,26991],[4531,4491,4635],[4532,3831,76464],[4533,179,27950],[4535,532,24763],[4536,1814,85030],[4537,2919,13964],[4538,1498,60851],[4539,2589,16907],[4540,1991,66754],[4541,3351,89078],[4542,980,9044],[4543,2321,39498],[4544,1660,32006],[4545,920,38805],[4546,18,49658],[4547,1494,26804],[4548,189,36861],[4549,669,79786],[4550,4065,28343],[4551,2532,70004],[4552,2722,27128],[4553,3443,66608],[4554,2796,67584],[4555,1796,36135],[4556,2514,32975],[4557,3559,20174],[4558,2190,33739],[4559,2963,12029],[4560,521,54806],[4563,3445,33136],[4564,1500,76453],[4565,1396,27199],[4566,2289,86884],[4567,4469,88230],[4568,1419,76536],[4569,2006,30700],[4570,4531,80742],[4571,3220,26018],[4573,3785,68147],[4574,690,28443],[4575,1464,91325],[4576,1266,8],[4577,2076,16790],[4578,4543,1307],[4579,3512,98467],[4580,432,13118],[4581,434,62418],[4582,1070,79697],[4583,4355,85343],[4584,3170,93676],[4585,2794,80270],[4587,2045,27570],[4588,2580,8393],[4589,2329,67793],[4590,1889,14355],[4591,1935,79026],[4592,831,87047],[4593,1381,46161],[4594,2989,59212],[4595,1720,94737],[4596,546,49690],[4597,1214,39966],[4599,1650,81916],[4600,425,89718],[4601,3555,78477],[4602,724,58678],[4603,1581,58442],[4605,851,65043],[4606,2991,54174],[4607,4058,90298],[4608,4570,7944],[4609,3641,60437],[4610,3638,88175],[4611,3462,88026],[4612,1352,59910],[4613,2002,76043],[4614,4024,27795],[4615,933,71515],[4616,3100,47486],[4617,3493,4960],[4618,339,77421],[4619,1852,55020],[4621,1846,11041],[4622,866,18472],[4623,1066,75456],[4624,3650,84981],[4625,2339,46161],[4626,1842,15031],[4627,4565,6344],[4628,1755,70168],[4629,827,88018],[4630,220,57971],[4631,967,216],[4632,572,71144],[4633,1956,40163],[4634,1411,13421],[4635,2256,89126],[4636,3648,18945],[4637,3712,49256],[4638,516,62013],[4640,3987,22993],[4642,2779,20474],[4643,103,37816],[4644,946,19710],[4645,15,9415],[4647,2912,86812],[4648,3970,66067],[4649,326,29409],[4650,4513,79126],[4651,2256,43001],[4652,4485,83849],[4653,3287,48907],[4654,97,60576],[4655,4242,38012],[4657,1111,62130],[4658,1125,61128],[4659,3560,50424],[4661,342,15798],[4662,3584,20662],[4663,3227,74501],[4664,2697,14218],[4666,237,18071],[4667,312,87484],[4668,3658,67409],[4669,2845,20027],[4670,3877,58585],[4672,1642,60886],[4673,896,4447],[4674,1221,62720],[4675,837,86386],[4676,591,33915],[4677,3369,72987],[4678,921,74036],[4679,2368,63822],[4680,4415,51617],[4681,1119,33049],[4682,967,76405],[4684,3744,6712],[4685,1353,57479],[4686,3170,15569],[4687,274,81979],[4688,4315,5179],[4689,4667,48423],[4690,200,60620],[4691,4473,90808],[4693,1399,56239],[4694,3359,52722],[4696,151,98005],[4697,4414,87334],[4698,1028,58593],[4699,2084,50761],[4700,2049,45712],[4701,4457,79697],[4703,4637,99505],[4704,3694,10424],[4705,2200,29181],[4706,1759,28867],[4707,2218,43607],[4708,1361,7603],[4709,2696,5641],[4710,3653,75938],[4711,2108,70237],[4712,1792,74944],[4713,1863,21110],[4715,3465,93770],[4716,3905,11610],[4717,2186,10193],[4718,2830,22834],[4719,930,8180],[4720,4348,56838],[4721,1190,46737],[4723,4036,18389],[4724,3713,65044],[4725,3894,64718],[4727,2758,4277],[4728,1763,84558],[4729,1633,17689],[4730,3018,63959],[4731,4330,30],[4734,2881,22061],[4735,901,19386],[4736,1360,78694],[4737,1842,52152],[4739,160,35853],[4740,1857,93637],[4741,3963,36841],[4742,1837,49606],[4743,3875,49625],[4744,2535,23575],[4745,4711,13036],[4746,441,26451],[4747,4354,84361],[4748,4615,97009],[4749,25,55153],[4750,2681,96763],[4751,2238,59516],[4753,3791,31067],[4754,1213,19121],[4755,4683,94869],[4756,1385,29601],[4757,2869,34791],[4758,2787,93227],[4759,2914,25950],[4760,2970,76747],[4761,1823,45253],[4762,2914,28511],[4763,2341,65318],[4764,2647,74108],[4766,1328,66505],[4768,3464,34532],[4769,927,14852],[4770,1214,34585],[4771,182,83511],[4772,3757,57958],[4774,140,49302],[4775,630,11705],[4776,1402,34556],[4777,3113,48105],[4779,3364,14184],[4780,551,57170],[4781,3518,90466],[4782,875,47885],[4783,2851,34721],[4785,230,85619],[4786,3377,45056],[4787,3132,91081],[4789,66,82436],[4790,2548,52704],[4791,4338,56905],[4792,4120,99677],[4793,4419,43359],[4794,435,78087],[4795,2909,22285],[4797,2061,7870],[4798,2515,85191],[4799,2783,66369],[4800,3174,97275],[4801,1697,97736],[4804,4105,89961],[4805,3310,72195],[4806,1870,36942],[4808,3184,57104],[4810,628,21970],[4812,256,38844],[4813,1211,20988],[4815,482,90449],[4816,3073,1152],[4817,2406,88800],[4818,597,65361],[4819,4724,90437],[4820,3209,51686],[4821,1639,7824],[4823,2020,82083],[4824,2217,94825],[4825,713,74412],[4826,3676,2122],[4828,1377,56147],[4829,1092,55844],[4830,4731,94794],[4831,2186,77528],[4832,639,34230],[4833,1263,37544],[4834,611,42309],[4835,779,48071],[4836,4732,77938],[4837,543,16588],[4838,4230,10228],[4839,2034,84285],[4840,531,147],[4841,3339,22941],[4842,3679,61463],[4843,25,35602],[4844,2300,54253],[4845,2725,11607],[4846,4636,81494],[4847,1016,19757],[4848,2478,80480],[4851,1576,47043],[4852,1216,4347],[4853,1403,55083],[4854,3844,79044],[4855,1583,50247],[4856,4780,76632],[4858,803,7034],[4859,2244,17262],[4860,3369,87245],[4861,2359,37927],[4862,2188,57550],[4863,3716,61628],[4864,883,6208],[4865,748,58641],[4866,2446,35579],[4867,4294,19134],[4868,2165,47254],[4869,4487,86939],[4870,200,54436],[4871,3992,26862],[4874,3503,389],[4875,2135,36172],[4876,4216,32802],[4877,4414,20265],[4878,2970,78655],[4879,2441,67982],[4880,4325,82202],[4881,4605,32399],[4882,4440,20060],[4883,51,76566],[4884,3654,34282],[4885,4502,50896],[4886,4050,63365],[4887,13,71783],[4888,2027,94033],[4889,345,13299],[4891,2631,23937],[4892,2101,11693],[4893,3070,30577],[4894,574,63789],[4895,1915,2410],[4896,1056,27361],[4897,3534,39732],[4898,515,95735],[4899,4062,72627],[4900,531,66987],[4901,4381,87830],[4902,851,2181],[4904,4294,98901],[4905,768,98467],[4906,3452,84798],[4907,3067,63384],[4908,3501,74516],[4910,1551,93555],[4912,1462,50598],[4913,2108,1946],[4914,4322,77859],[4915,1106,67460],[4916,1390,28943],[4917,88,57485],[4918,3254,43086],[4919,1302,52548],[4921,965,16060],[4922,2422,28910],[4923,4832,88005],[4924,1001,72133],[4925,499,57329],[4926,3787,37519],[4927,3560,32787],[4928,4871,3647],[4929,2312,80542],[4930,3421,67224],[4931,4583,9531],[4932,1772,33538],[4933,1139,53981],[4934,773,51190],[4935,354,91662],[4936,2471,2480],[4937,4924,49037],[4938,1949,33810],[4941,2310,14941],[4942,3019,96081],[4943,4115,93869],[4944,2011,44779],[4945,1581,86664],[4946,2921,82424],[4948,2905,83587],[4949,2213,30147],[4950,3593,70514],[4952,3028,29780],[4953,3237,10833],[4954,4074,8601],[4955,834,22383],[4956,959,7264],[4957,1385,2613],[4958,4140,56166],[4960,1446,6098],[4961,779,31827],[4962,1121,51094],[4963,1929,43],[4965,4938,62578],[4966,3406,91912],[4967,1830,71886],[4968,2488,71308],[4969,2557,68063],[4970,2184,55900],[4971,2316,38923],[4972,1645,8985],[4975,184,96201],[4976,1824,98544],[4977,1848,84683],[4978,3758,64277],[4979,3223,93658],[4980,1627,85933],[4981,2383,17030],[4982,601,81340],[4983,2932,41897],[4984,1040,94623],[4985,2096,210],[4986,3765,77764],[4987,4926,12438],[4988,536,3552],[4989,3904,98678],[4991,3543,80364],[4992,3549,86382],[4993,220,2875],[4994,2732,64195],[4995,1159,47116],[4996,3493,39077],[4997,975,76116],[4998,1300,66438],[4999,4497,95817],[5000,492,8423],[5001,3303,93179],[5002,4491,69078],[5003,4887,34527],[5004,3784,46453],[5005,4219,3531],[5006,2755,89601],[5007,2830,19574],[5008,1689,51612],[5009,2518,58863],[5011,2259,20021],[5012,3357,12291],[5013,34,13848],[5014,4870,61852],[5015,3907,55279],[5016,4505,19462],[5017,3608,11338],[5018,749,62385],[5019,620,69781],[5020,1148,21148],[5021,4392,91390],[5023,3398,1276],[5024,1804,87503],[5025,1845,57473],[5026,2737,75821],[5027,2039,43945],[5028,1438,639],[5029,1197,86897],[5030,2922,18293],[5031,4611,9313],[5032,4970,92991],[5033,2734,68332],[5034,2967,24101],[5035,3791,88000],[5036,1403,58753],[5037,4144,16031],[5039,3911,53694],[5040,4917,8454],[5041,4317,70494],[5042,601,96285],[5043,3303,66438],[5044,1664,46273],[5045,2571,4659],[5047,3247,43904],[5048,1084,27725],[5049,1312,8813],[5050,3394,46672],[5051,859,34244],[5052,2052,16107],[5054,1384,86318],[5056,1062,69011],[5057,1637,85042],[5058,3393,58092],[5059,3600,11436],[5063,4575,10201],[5064,4271,61004],[5065,2924,1048],[5066,2133,32828],[5067,307,19470],[5068,878,50976],[5069,800,8885],[5070,3999,6668],[5072,674,1011],[5073,2631,2642],[5074,1229,56272],[5075,109,38474],[5076,4891,15325],[5077,1262,10012],[5078,2689,93010],[5079,2586,46840],[5081,2521,4180],[5082,4228,71476],[5084,2563,2967],[5085,3797,75682],[5086,914,27881],[5087,3689,69639],[5088,1043,40659],[5089,261,76999],[5090,3561,37062],[5091,1513,37325],[5093,163,10455],[5094,4625,77024],[5095,3322,56191],[5096,1160,60551],[5097,4103,92332],[5098,4957,51648],[5099,1888,63794],[5100,1924,43202],[5101,517,28950],[5102,4231,29351],[5103,4139,39935],[5105,2945,93499],[5107,4305,30742],[5108,1056,23710],[5109,3170,27688],[5110,2406,43483],[5111,1977,6733],[5112,1184,16376],[5113,2534,86505],[5114,3426,52088],[5115,1034,73215],[5116,4750,68378],[5117,668,16247],[5118,2608,48586],[5119,2671,56837],[5120,3560,31900],[5121,210,61196],[5122,3288,94986],[5123,424,81422],[5124,1756,90676],[5125,4238,67258],[5126,409,9053],[5128,1235,89579],[5130,3534,1492],[5131,3140,54305],[5132,1045,10063],[5135,2344,48736],[5136,4385,63066],[5137,2803,9615],[5138,346,29755],[5139,1328,23138],[5140,1720,98085],[5141,2117,90414],[5142,4826,37171],[5143,3722,95272],[5144,2333,75609],[5145,3649,47629],[5146,1734,59651],[5147,945,873],[5148,1605,85659],[5149,1149,81390],[5150,3870,58076],[5151,311,9066],[5152,4793,86879],[5155,273,13981],[5156,564,89459],[5157,294,12985],[5158,4357,40643],[5160,550,59699],[5161,3719,82775],[5162,4637,88229],[5163,2046,50347],[5164,1145,33270],[5165,2635,58556],[5167,2266,17315],[5168,636,74418],[5169,4437,300],[5170,186,19149],[5171,2174,76749],[5172,4264,99628],[5173,3217,17101],[5174,5113,1905],[5175,3232,19172],[5177,758,8939],[5178,780,2599],[5179,4966,51436],[5180,1325,19357],[5181,2914,3567],[5182,2762,59557],[5183,4764,56602],[5184,503,3290],[5185,4402,88492],[5186,403,81808],[5187,4381,11678],[5188,5182,89364],[5189,4800,81793],[5190,2627,56079],[5191,2305,8057],[5192,1768,30821],[5193,813,98760],[5194,1562,29919],[5195,877,30488],[5196,2101,7896],[5197,3677,98542],[5198,3935,65162],[5199,4363,4111],[5200,2281,52968],[5201,4902,10581],[5202,4006,95046],[5203,4737,86021],[5204,2108,78381],[5205,370,89705],[5206,1592,9124],[5207,3274,31366],[5208,1206,80811],[5209,1125,22906],[5210,883,34491],[5213,1648,90210],[5214,1858,44675],[5215,3726,32517],[5216,4776,28098],[5217,2584,95623],[5220,4047,98012],[5222,297,80216],[5223,1080,38888],[5224,1878,71155],[5225,103,78673],[5226,4190,63156],[5227,1895,18185],[5228,3363,42839],[5230,2475,11270],[5231,5138,71286],[5232,4307,18214],[5233,5059,27668],[5234,4436,52267],[5235,3797,393],[5236,5139,4429],[5237,2723,15694],[5238,576,17098],[5239,3274,99132],[5241,3851,6048],[5242,4330,41329],[5243,835,8966],[5244,182,48182],[5245,3438,2465],[5247,3068,18112],[5248,2561,77483],[5249,5068,31133],[5251,2336,16916],[5252,64,91562],[5254,4038,81954],[5255,5097,40951],[5256,5080,90923],[5257,260,29175],[5258,2769,38065],[5259,2480,46830],[5260,3513,40251],[5262,2965,42511],[5263,4422,98610],[5264,3202,97558],[5265,3603,93923],[5266,4879,69456],[5267,276,39341],[5268,1250,45667],[5269,763,32782],[5270,4118,81897],[5271,3140,52878],[5272,1749,32846],[5273,4596,69642],[5275,2041,97030],[5276,2312,77324],[5277,2393,17798],[5279,5080,30687],[5280,3784,79480],[5281,4795,70982],[5282,4996,69148],[5283,871,68154],[5284,745,65751],[5285,2223,70521],[5286,3996,16350],[5287,5189,60883],[5288,824,1580],[5289,1566,93815],[5290,3705,26003],[5291,3950,75139],[5292,540,65509],[5293,1368,96697],[5294,4802,9913],[5295,2418,37395],[5296,1985,1944],[5298,3810,99890],[5299,3701,69599],[5300,149,93711],[5301,3855,92746],[5302,3007,70690],[5303,2357,48065],[5304,3380,87408],[5305,3686,77129],[5306,4237,67110],[5307,5166,54844],[5308,598,43711],[5309,3474,72185],[5311,156,91480],[5312,5011,16471],[5314,3658,78334],[5315,2597,8708],[5316,3589,89346],[5317,129,52403],[5319,1706,52151],[5320,2780,53519],[5321,2541,89795],[5322,3123,88123],[5323,5257,15282],[5324,5007,40471],[5325,1118,95041],[5326,3890,5288],[5327,3740,45794],[5328,1414,69513],[5329,1173,65430],[5330,3973,9862],[5331,5305,86519],[5332,4777,15225],[5333,1491,40879],[5334,167,68339],[5335,3533,99884],[5337,712,94302],[5338,2531,10873],[5339,1060,51313],[5340,1984,80000],[5341,3560,15752],[5342,820,50862],[5343,616,20895],[5344,3159,81312],[5345,4069,67186],[5346,509,99196],[5347,4575,48404],[5348,2717,70232],[5349,3429,25884],[5350,1672,94949],[5351,2425,4067],[5352,4306,48950],[5353,2983,36789],[5354,4209,82645],[5355,4359,17846],[5356,4954,23968],[5357,3323,53943],[5358,5180,57925],[5359,3860,66010],[5360,4012,86699],[5361,5188,46315],[5362,4436,76552],[5363,3395,81514],[5364,1536,87818],[5365,51,60624],[5366,3909,56168],[5367,4276,75392],[5368,1474,53994],[5369,843,6623],[5370,5244,50991],[5371,2737,49567],[5372,3092,36055],[5373,4153,24489],[5374,1344,64576],[5375,3192,7771],[5376,1484,44739],[5377,2028,98682],[5379,3629,46785],[5380,3057,59306],[5381,1852,91526],[5382,3423,34096],[5383,1329,89077],[5384,515,33033],[5385,1314,78861],[5386,1055,31295],[5387,3739,54083],[5388,4652,40326],[5389,2429,7338],[5390,4272,27584],[5391,3816,22160],[5392,4140,74488],[5393,1724,88419],[5394,291,61388],[5395,3047,38938],[5396,4885,86093],[5397,195,51687],[5398,4471,50515],[5399,2197,21657],[5400,233,97384],[5401,4413,99331],[5402,5383,35673],[5403,3442,99448],[5404,2903,80299],[5405,2991,4388],[5406,4295,82896],[5408,4576,56212],[5409,1040,37729],[5410,996,27933],[5411,2430,27134],[5414,2034,33353],[5415,2905,48982],[5416,1627,62576],[5418,2411,36658],[5420,1185,12843],[5421,3700,33316],[5422,1125,8203],[5423,800,51282],[5424,1887,92282],[5426,1938,60878],[5427,3020,70224],[5429,3881,80073],[5430,3604,67619],[5431,3885,65279],[5432,220,26829],[5433,1712,26490],[5434,4267,21338],[5435,1906,40476],[5437,3195,50822],[5438,5240,9231],[5439,2941,97757],[5440,4342,21041],[5441,5205,90875],[5442,2921,5388],[5443,3612,46953],[5444,1492,10657],[5445,2396,86430],[5446,5214,22243],[5447,3432,50648],[5449,3321,84088],[5450,2743,83021],[5451,2710,18317],[5452,1209,95857],[5454,2507,1141],[5455,4346,13965],[5456,2965,69213],[5458,3339,64777],[5460,3849,95743],[5461,3841,75506],[5462,4514,25499],[5463,2199,75710],[5464,32,33138],[5465,3353,82921],[5466,356,75152],[5467,917,37722],[5468,963,98529],[5470,1157,84875],[5471,4686,33415],[5472,203,92749],[5473,91,18844],[5474,3596,3273],[5475,4299,80595],[5476,4392,11367],[5477,4001,57951],[5478,138,51411],[5479,4021,47197],[5480,2019,27864],[5481,267,2336],[5482,2333,49910],[5483,2376,52503],[5484,2349,37756],[5485,240,5267],[5486,2074,43833],[5487,1343,86220],[5488,3416,64931],[5489,1005,81346],[5490,1199,63912],[5491,1943,80197],[5492,925,4827],[5493,2149,75437],[5494,1294,3842],[5495,4001,17295],[5496,3412,59786],[5497,3903,85126],[5498,1902,35913],[5499,1441,28638],[5500,4043,69003],[5501,4733,57288],[5502,4116,90986],[5504,3621,27498],[5505,29,83340],[5506,4324,40341],[5507,1324,23592],[5508,2174,47707],[5509,2301,96192],[5510,865,16290],[5511,952,27239],[5512,2200,27034],[5513,3024,94514],[5514,5445,55487],[5516,2563,48149],[5518,4416,35112],[5519,128,17205],[5520,313,36138],[5521,4374,78918],[5523,95,26777],[5524,3446,97035],[5525,1833,35072],[5526,1434,94731],[5527,514,3716],[5528,3574,47794],[5529,2945,44877],[5530,1626,96701],[5531,366,42377],[5532,1171,42366],[5535,307,23460],[5536,2769,18621],[5537,4981,78201],[5540,344,53213],[5541,3689,12245],[5542,2709,87987],[5543,560,46959],[5544,3699,95157],[5545,2474,79588],[5546,487,20313],[5547,4732,11054],[5551,5522,83055],[5552,1127,52773],[5553,1827,45973],[5554,189,34218],[5555,4570,3428],[5556,2132,18386],[5557,3051,4894],[5558,4154,84585],[5559,2626,38974],[5560,1143,3665],[5561,2826,90627],[5562,4821,51185],[5563,1870,39127],[5564,1646,17493],[5565,101,51413],[5566,4479,26491],[5567,3854,46793],[5568,136,9649],[5570,3804,23431],[5571,992,81507],[5572,4880,77656],[5573,4643,54554],[5574,1339,5546],[5575,3739,4621],[5576,3665,44340],[5577,2253,56995],[5578,3097,53677],[5579,481,17686],[5580,1844,57197],[5581,687,23824],[5582,4001,12075],[5583,1074,75557],[5584,5331,59846],[5586,401,83992],[5587,650,21887],[5588,2126,91454],[5589,205,11279],[5590,388,33902],[5591,4708,94009],[5592,4240,28716],[5593,1671,83058],[5594,4629,77236],[5595,660,97701],[5596,5582,91210],[5597,2147,34173],[5598,3937,30845],[5599,265,11017],[5600,896,30200],[5601,2040,83755],[5603,5059,15731],[5604,1048,31364],[5605,4621,58441],[5606,2533,7536],[5607,1538,40743],[5608,4476,32726],[5609,3762,90397],[5610,1646,61360],[5611,1177,68182],[5612,2559,59808],[5613,5487,29640],[5614,4259,38040],[5615,2343,6742],[5616,3311,34317],[5618,1809,64221],[5619,2992,78968],[5621,2063,1572],[5622,1841,30498],[5623,501,64835],[5624,846,73074],[5625,5231,97474],[5626,2742,92735],[5627,263,30920],[5631,808,81045],[5632,5193,56085],[5633,1810,24283],[5636,5234,3790],[5637,1138,53138],[5639,871,89393],[5641,3124,78086],[5642,3846,88306],[5643,3097,45623],[5644,5539,50727],[5645,5509,14027],[5646,579,76491],[5647,3493,65298],[5648,4908,61148],[5650,843,42752],[5651,3555,19907],[5653,5071,35474],[5654,4332,96444],[5656,5566,91465],[5657,3131,79999],[5659,52,24605],[5660,5175,84559],[5661,5010,71664],[5662,1943,14359],[5663,70,11172],[5664,2438,80303],[5667,2870,88471],[5668,2774,61734],[5670,458,28813],[5671,3889,7137],[5672,3306,40402],[5673,5299,5026],[5674,5555,94408],[5675,2617,24073],[5676,1766,86966],[5677,3764,57706],[5678,4963,81306],[5679,351,65725],[5680,1064,90122],[5681,3371,3656],[5682,2967,84542],[5683,1258,53823],[5685,1004,54878],[5686,3206,77901],[5687,2425,13957],[5688,153,64485],[5689,3230,67785],[5690,5472,32721],[5692,1810,22518],[5693,623,59998],[5694,142,90783],[5696,1508,98549],[5697,4659,64396],[5698,2965,4996],[5699,4810,86188],[5700,1194,54613],[5701,4682,29410],[5702,1446,14873],[5703,3901,42285],[5704,4340,26700],[5705,757,98599],[5706,4437,23410],[5707,2161,32506],[5709,5164,8902],[5710,19,65418],[5711,5000,76640],[5712,703,15666],[5713,3219,83122],[5714,1168,90816],[5715,3097,85317],[5716,396,25561],[5718,3069,95573],[5719,1871,6680],[5720,5682,39346],[5721,521,24898],[5722,741,40776],[5723,2952,29555],[5724,3194,42880],[5725,255,52559],[5726,1110,6815],[5727,1210,35076],[5728,4974,36678],[5729,326,24210],[5730,5631,13037],[5732,280,98203],[5733,4323,80344],[5734,3990,41955],[5735,3387,40076],[5737,4082,93643],[5738,3608,28946],[5740,997,39935],[5741,763,83885],[5742,325,81248],[5743,4344,80904],[5745,233,23064],[5746,1944,53903],[5748,3730,85391],[5750,3970,13045],[5751,4404,29945],[5752,4679,85136],[5753,2912,40179],[5754,2610,68669],[5755,2818,13235],[5756,2550,49621],[5757,1759,39328],[5758,3534,38359],[5759,3643,30819],[5760,4976,41323],[5762,1476,28015],[5763,4033,15471],[5764,3307,54381],[5765,4469,67451],[5766,4409,84259],[5767,4627,343],[5768,1219,51183],[5769,2,72001],[5770,171,56977],[5771,2837,53365],[5772,4755,53746],[5773,4726,18278],[5774,3796,10837],[5775,633,97611],[5776,4594,42588],[5777,2681,74120],[5778,2061,52034],[5779,3190,33344],[5780,4193,37755],[5781,1399,26188],[5782,4126,29488],[5783,1085,14150],[5784,350,83946],[5785,4656,708],[5786,5430,99934],[5787,549,54966],[5788,5735,79915],[5789,3812,71891],[5790,1925,33992],[5791,2341,2856],[5792,735,28090],[5793,4742,71140],[5794,860,29676],[5795,1333,44403],[5796,868,11337],[5797,3640,13188],[5799,5222,8470],[5800,520,37307],[5801,5690,96610],[5802,3984,82772],[5804,2452,64506],[5805,3074,68532],[5806,589,94334],[5807,3317,88808],[5808,531,95483],[5809,2548,2153],[5810,5362,23707],[5811,181,43927],[5812,2378,7337],[5814,249,33673],[5816,233,58766],[5817,634,96792],[5818,4449,63448],[5819,235,23437],[5820,3715,45088],[5821,174,94220],[5822,1119,86066],[5823,1117,298],[5824,506,75146],[5825,4023,34533],[5826,4579,9979],[5827,512,35868],[5828,207,42468],[5829,5500,60778],[5830,5184,87008],[5831,3232,11780],[5832,1646,29461],[5833,3994,70882],[5834,2260,99065],[5835,2461,35386],[5836,333,76436],[5837,5427,49711],[5838,3595,81321],[5839,4912,72740],[5840,3505,9161],[5841,3536,44279],[5842,1411,75351],[5843,3085,30779],[5844,5405,70651],[5845,2762,63206],[5847,2596,71089],[5848,2865,21880],[5849,3044,76616],[5851,4260,88935],[5852,5480,5743],[5853,2046,45748],[5854,298,67987],[5855,522,55165],[5856,2431,54108],[5857,1451,53806],[5858,1632,37933],[5860,2061,81981],[5861,5533,6628],[5862,1338,2008],[5863,5861,39949],[5864,2140,11697],[5865,2631,10015],[5866,4151,42222],[5867,4720,96228],[5868,4357,44298],[5869,3648,56979],[5870,1037,95099],[5872,3632,73283],[5873,5667,45903],[5874,3404,59737],[5875,888,88843],[5877,920,89520],[5878,3197,39470],[5879,4181,18513],[5880,1176,90536],[5881,4119,18867],[5882,5044,24236],[5883,333,60667],[5884,2974,78863],[5885,548,6322],[5886,4291,70292],[5887,20,19497],[5888,1255,60421],[5889,3969,88147],[5890,4363,18603],[5891,3099,80941],[5892,1198,86951],[5893,4523,85372],[5894,53,50625],[5895,4748,59881],[5897,4993,39748],[5898,5667,42746],[5899,5652,86801],[5900,1536,97270],[5901,3494,7487],[5902,5897,94828],[5903,298,19976],[5904,5601,40583],[5905,2052,55512],[5906,4272,45557],[5907,1177,49898],[5908,2939,65558],[5909,5020,12912],[5910,4127,81628],[5911,4211,20999],[5912,4092,46669],[5913,5426,92792],[5914,3808,78717],[5915,1813,35992],[5916,1250,75908],[5917,3998,27555],[5918,5019,99109],[5919,1072,33384],[5921,5174,72428],[5922,1401,55690],[5923,1761,18892],[5924,3074,44392],[5925,3169,41980],[5926,4945,8182],[5927,4242,95674],[5928,932,73584],[5929,5158,41380],[5930,625,40587],[5933,1101,32353],[5934,5741,39820],[5935,176,87408],[5936,3208,26687],[5937,4211,38509],[5938,1234,62255],[5940,933,33868],[5942,5934,37186],[5943,4733,72860],[5944,1007,53290],[5945,1901,63318],[5946,4076,44226],[5947,673,20061],[5948,1240,80484],[5949,814,28440],[5950,4227,33961],[5951,4847,11336],[5952,1158,59893],[5953,2227,84536],[5954,4484,48534],[5955,3052,90702],[5956,4302,69368],[5957,3845,24107],[5958,1915,119],[5959,1006,90281],[5960,3335,71318],[5961,1947,76490],[5962,1911,44391],[5963,2297,83460],[5964,2419,58041],[5965,3026,25007],[5966,4208,78781],[5967,2761,94340],[5968,2361,90260],[5969,5852,75170],[5970,3407,27814],[5971,3843,7005],[5972,4484,18010],[5973,5146,26400],[5974,5943,61756],[5975,2868,51679],[5976,2418,79317],[5977,5612,46915],[5978,3137,83092],[5979,3351,12625],[5980,3183,31684],[5981,1066,92692],[5982,5631,68029],[5983,3148,29366],[5984,3617,1166],[5985,931,57805],[5986,5141,73951],[5987,2425,71343],[5988,5444,58678],[5989,1872,55810],[5990,5442,82699],[5991,1814,88313],[5992,2595,89058],[5993,2712,18574],[5994,3075,8040],[5995,3441,25287],[5996,34,97704],[5997,1016,56177],[5998,4460,67223],[6001,4737,3509],[6002,5933,36297],[6003,5331,95528],[6005,651,83962],[6006,2451,57297],[6007,87,56048],[6008,2444,51709],[6009,1859,1116],[6011,3392,88333],[6012,1470,71423],[6013,1711,23893],[6014,689,60487],[6015,3427,48773],[6016,2516,48094],[6017,3431,95261],[6018,1319,31169],[6019,2197,65103],[6020,1922,16074],[6021,1068,21330],[6022,4343,88104],[6023,3814,92347],[6025,4710,8204],[6026,2087,19158],[6027,1705,76777],[6029,5228,55344],[6030,2310,19007],[6031,710,67668],[6032,4618,87322],[6033,29,48582],[6034,3584,19516],[6035,2236,9743],[6036,2829,65487],[6037,1722,75968],[6038,2681,17097],[6039,1583,41837],[6040,1921,37240],[6041,2235,72059],[6043,5935,24728],[6044,1414,96811],[6045,1226,2816],[6046,996,68217],[6047,3891,87894],[6048,5446,18616],[6049,4706,22966],[6050,1919,78661],[6052,5243,97996],[6054,97,6214],[6055,5767,67966],[6056,1359,46195],[6057,2838,49001],[6058,1747,33698],[6060,3351,22712],[6061,1495,37869],[6062,5562,37506],[6063,353,96142],[6064,3254,20783],[6065,2721,98498],[6066,5482,83378],[6067,2017,88298],[6068,2628,4914],[6069,1554,71139],[6070,3153,54385],[6072,1367,95332],[6073,1310,8502],[6074,3024,52789],[6075,632,6624],[6078,5135,51884],[6079,5543,13032],[6080,5804,2248],[6081,4054,88093],[6082,3668,66204],[6083,3286,43218],[6084,459,83800],[6085,3184,9654],[6086,5844,17743],[6087,2631,60445],[6088,5332,25507],[6089,4715,13601],[6090,383,94086],[6091,4409,75694],[6092,3846,34403],[6093,3247,16667],[6094,3071,15265],[6095,4226,98735],[6097,4486,34497],[6098,3874,6074],[6099,3905,1713],[6100,2703,25058],[6101,5578,59512],[6102,4498,79605],[6103,1803,8779],[6104,2952,36939],[6105,317,76937],[6106,3125,54407],[6108,3101,33831],[6109,1590,77973],[6110,4680,31716],[6111,1782,74011],[6112,4741,41976],[6113,5434,20655],[6114,3946,96552],[6115,2276,64863],[6116,1763,39728],[6118,1316,91435],[6119,1280,35965],[6120,2818,72362],[6121,1919,2581],[6122,6001,9525],[6124,5171,45844],[6125,4282,96547],[6126,4086,95553],[6127,248,77500],[6128,3098,40918],[6129,4452,37017],[6130,1389,33732],[6131,238,24824],[6132,4201,67584],[6133,4168,47388],[6134,3619,95376],[6135,5585,28885],[6136,1518,45941],[6138,5804,92655],[6139,3295,8561],[6140,6125,23894],[6141,1605,23578],[6142,974,63751],[6144,1602,74828],[6145,2423,70742],[6146,3568,97683],[6147,3385,26496],[6148,2503,24421],[6149,239,99430],[6150,3099,88843],[6151,3212,55202],[6152,2569,43298],[6153,4990,80765],[6154,5841,75807],[6155,3987,8138],[6156,734,64764],[6157,3920,74538],[6158,697,71615],[6159,194,69153],[6160,1562,53261],[6161,2244,58858],[6162,3527,14301],[6163,5130,31784],[6164,2578,14728],[6165,419,27470],[6166,4587,49816],[6167,4635,56798],[6168,712,22352],[6169,536,82625],[6171,2795,63729],[6172,4413,15896],[6173,2000,21465],[6174,6036,25834],[6175,4277,11519],[6176,3272,19166],[6177,891,26654],[6178,1914,67835],[6179,4793,53924],[6180,1276,38911],[6181,5454,35139],[6182,1808,78567],[6183,781,30236],[6184,2875,9006],[6186,1850,16334],[6187,418,17481],[6188,5769,94536],[6190,4829,28220],[6191,3713,31660],[6192,167,63128],[6193,4508,19991],[6194,5334,33391],[6195,4642,3011],[6196,4180,31404],[6197,1953,27010],[6198,4467,16118],[6199,4326,79603],[6200,6161,75847],[6201,1167,67015],[6202,1094,43316],[6204,1117,85546],[6205,4648,91562],[6206,4688,32048],[6207,1430,6393],[6208,5891,31574],[6210,3734,63291],[6211,4697,75175],[6212,5494,84854],[6213,495,9118],[6214,4971,19020],[6215,2802,84932],[6216,1975,63477],[6218,2311,21814],[6219,3709,33846],[6220,1067,36100],[6222,5660,97083],[6223,2351,11185],[6224,3020,66923],[6225,4120,20412],[6226,201,25430],[6227,5946,71326],[6228,864,98314],[6229,2137,12123],[6230,373,32915],[6231,3105,2776],[6232,4421,50472],[6233,3037,8665],[6234,4587,82380],[6235,914,15133],[6236,1251,50997],[6237,1185,7778],[6238,1521,97125],[6239,5200,43898],[6240,4533,94732],[6241,2328,25923],[6242,4440,39369],[6243,2621,38263],[6244,3555,19633],[6245,5750,58142],[6247,327,54606],[6248,5819,56585],[6249,232,87412],[6251,4439,72128],[6252,5372,531],[6253,3988,8947],[6254,4396,22375],[6255,4681,16114],[6256,3596,73187],[6257,2369,39321],[6258,4790,99247],[6259,5221,39559],[6260,4332,2709],[6261,3933,88516],[6262,5676,32527],[6263,5603,78526],[6264,1994,8820],[6266,3908,46152],[6267,4433,76625],[6268,1390,74548],[6269,3660,78738],[6270,2398,73278],[6271,5157,31563],[6272,1402,21770],[6273,1291,65797],[6275,5647,14311],[6276,4430,12075],[6277,364,12046],[6278,3084,66818],[6279,2806,78790],[6280,4863,73590],[6281,3777,53312],[6282,5725,12935],[6283,1492,65584],[6284,3880,73256],[6286,108,88922],[6287,6132,19025],[6288,1094,67741],[6289,2393,48859],[6290,5075,29589],[6291,2409,95972],[6292,3725,28858],[6293,848,69324],[6294,2043,97278],[6295,3068,62299],[6297,575,16745],[6298,5050,17973],[6299,5746,84827],[6300,5967,78011],[6301,2680,40755],[6302,224,10030],[6304,5055,64282],[6305,5491,84077],[6306,6116,76140],[6307,884,48789],[6308,4128,79244],[6309,5002,32004],[6310,5676,875],[6311,5191,6923],[6312,4083,46919],[6313,1205,54061],[6315,614,7585],[6316,4259,16026],[6317,1931,21506],[6318,3440,69142],[6319,5757,24912],[6320,647,27568],[6321,4308,66011],[6323,3751,24759],[6324,4695,46735],[6326,4966,10849],[6328,1754,68828],[6329,5799,16217],[6330,1336,78128],[6331,2433,84466],[6332,4667,48282],[6334,3278,84035],[6335,4722,99948],[6336,5500,82518],[6337,2632,47157],[6338,4836,46174],[6339,4496,95462],[6341,1697,65116],[6342,1657,42169],[6343,2737,19384],[6344,4191,30625],[6345,948,95436],[6346,4013,51356],[6347,1007,48686],[6348,1996,28921],[6349,4023,54196],[6350,966,45476],[6351,1966,260],[6352,4633,23120],[6353,375,31525],[6354,4441,45686],[6355,821,67940],[6356,1540,20998],[6357,4414,37928],[6358,4613,53352],[6359,5137,95437],[6360,165,85940],[6361,1439,10106],[6362,5,43272],[6363,1625,64110],[6364,3766,66574],[6365,1606,83921],[6368,5914,80390],[6369,3637,57284],[6370,2039,12473],[6371,4046,48493],[6372,1298,89219],[6373,12,87962],[6374,3377,84964],[6375,4162,99258],[6376,3361,13267],[6377,2505,94130],[6378,5622,35924],[6379,5156,11166],[6380,877,26070],[6381,642,79298],[6382,4446,63813],[6385,5782,68663],[6387,1795,90230],[6388,4243,92545],[6389,4217,30245],[6390,5034,15926],[6391,2464,9450],[6392,2695,60517],[6393,2338,38816],[6395,3754,20143],[6397,2686,47500],[6399,303,32263],[6400,48,53481],[6401,3649,16819],[6402,6344,50733],[6403,4023,44767],[6404,944,54571],[6406,3550,92271],[6407,5724,67350],[6408,727,27362],[6409,2664,75473],[6410,644,74872],[6411,3614,71276],[6413,521,40084],[6414,2159,96247],[6415,5752,27727],[6416,367,3598],[6417,3670,48241],[6418,5656,17697],[6419,4269,82108],[6420,1809,84308],[6421,2050,75318],[6422,5513,56153],[6423,3849,95471],[6424,3349,45711],[6425,3962,63047],[6426,3,66395],[6427,5803,63175],[6428,326,43690],[6429,484,56388],[6431,979,15591],[6433,2145,58961],[6434,1803,64463],[6435,3522,79658],[6436,3713,39883],[6437,219,9444],[6438,3992,63260],[6439,865,10003],[6440,5825,50251],[6441,5649,56704],[6442,732,21989],[6443,2364,75548],[6445,5525,24708],[6446,3971,3924],[6447,4511,97980],[6448,6269,68186],[6449,3239,91087],[6450,1911,55643],[6451,3366,75851],[6452,3605,73742],[6453,3927,35937],[6454,2295,96444],[6455,5070,84521],[6456,620,15705],[6457,4943,29230],[6458,4600,8123],[6459,6365,69351],[6460,3868,67428],[6461,2036,79106],[6462,2552,72044],[6463,1180,68234],[6464,6237,60319],[6465,5904,85884],[6466,2973,70639],[6467,2734,26367],[6468,4528,72210],[6469,2245,38571],[6470,3397,51518],[6471,4758,87981],[6472,5156,93227],[6473,1418,38023],[6474,114,7386],[6475,3388,55803],[6476,3136,94552],[6477,327,75950],[6478,65,5650],[6480,1857,40733],[6481,2072,71585],[6482,1308,58436],[6483,5067,35711],[6485,3459,56428],[6486,1166,32398],[6487,200,10391],[6488,455,27393],[6489,1453,39736],[6490,4953,81076],[6491,149,8724],[6492,5049,21696],[6494,3840,40606],[6495,4604,63133],[6496,1884,95767],[6497,2107,48665],[6499,6279,10066],[6500,5113,45707],[6501,1289,51276],[6502,408,14293],[6503,1040,45132],[6504,1812,71292],[6505,1837,14149],[6506,4963,6295],[6507,3107,9891],[6508,2540,30423],[6509,4560,52076],[6510,326,20299],[6511,2956,5742],[6512,3602,88322],[6514,96,31784],[6515,703,95823],[6516,5910,9105],[6518,1787,57192],[6519,1029,11874],[6522,2598,79082],[6523,229,75786],[6524,5130,22058],[6525,4835,53517],[6527,1718,67264],[6528,1113,54355],[6529,886,62078],[6531,2957,91136],[6532,1512,15765],[6533,995,23040],[6534,1046,17467],[6535,1850,63983],[6536,4186,44896],[6537,3090,64319],[6538,687,1567],[6539,2651,29253],[6540,5827,47561],[6541,2672,39592],[6543,4233,30992],[6544,2976,7567],[6546,249,88266],[6547,140,69078],[6548,3055,51534],[6549,2966,24830],[6550,6492,55585],[6551,5689,66320],[6552,2376,40216],[6553,4574,23517],[6554,6479,36400],[6555,1532,34617],[6557,1500,98144],[6558,1144,82766],[6560,6388,71026],[6561,4475,31471],[6562,3078,1666],[6563,5823,60712],[6564,3552,62499],[6565,257,68670],[6566,1687,92067],[6567,3448,84459],[6568,4660,55220],[6569,6019,1557],[6570,3600,96260],[6571,1360,67744],[6572,3854,99990],[6573,5766,21472],[6574,1807,96712],[6575,4863,96045],[6576,136,15910],[6577,1109,72514],[6578,4209,65769],[6579,4457,35276],[6580,5900,30447],[6581,4161,62389],[6582,299,87778],[6583,6158,6889],[6584,5037,64542],[6585,2880,759],[6587,5984,61737],[6588,3509,16591],[6589,4185,97431],[6590,967,97030],[6591,4858,22500],[6592,5486,6422],[6593,5255,68561],[6594,3803,85190],[6595,1483,94215],[6596,2337,19037],[6597,4495,51764],[6600,6225,73468],[6601,5881,70972],[6602,4015,85365],[6603,1782,96119],[6604,1120,32890],[6605,6413,70558],[6606,876,51275],[6607,643,41799],[6608,5037,57131],[6609,3787,76233],[6610,2512,98781],[6612,6027,10402],[6613,4332,24575],[6615,3993,43780],[6616,2130,73396],[6617,4783,84699],[6618,1494,76182],[6619,260,46726],[6621,3464,13638],[6622,1199,13963],[6624,3616,10730],[6625,4257,98919],[6626,1745,14494],[6627,4606,57473],[6628,4987,43727],[6629,4631,38697],[6630,3928,67187],[6632,4166,45008],[6633,2614,67188],[6634,720,76281],[6635,6409,37032],[6636,2671,70969],[6637,3294,49800],[6638,3027,21497],[6639,2267,52594],[6640,5504,45665],[6641,1045,62007],[6642,3789,22001],[6643,4489,88019],[6644,1756,55124],[6645,2468,36475],[6647,1121,89004],[6648,3582,54259],[6649,6031,31054],[6650,981,74471],[6652,2601,76352],[6654,6206,22436],[6655,1084,32591],[6656,2437,61692],[6658,6511,47343],[6659,4508,77584],[6662,2391,19377],[6663,1795,53534],[6664,3049,21650],[6665,1061,26238],[6666,372,86373],[6667,4374,90991],[6668,2011,79474],[6669,4577,57849],[6670,1505,39074],[6671,977,28638],[6672,5634,3352],[6673,4633,76226],[6674,346,13106],[6675,4682,79774],[6676,2558,53640],[6677,1947,95744],[6678,339,71860],[6680,3330,37380],[6681,1026,72781],[6682,4326,677],[6683,2169,59223],[6684,3877,44816],[6685,956,32267],[6686,2410,39266],[6687,6203,54267],[6688,6342,7002],[6689,2235,67449],[6690,625,39788],[6691,1021,31919],[6692,2511,66530],[6693,3871,39124],[6694,769,87351],[6695,3909,34554],[6696,569,1810],[6697,996,67427],[6698,6297,81091],[6699,1540,53479],[6700,5626,6233],[6701,1112,48100],[6702,227,5341],[6703,3052,16626],[6704,5599,91268],[6705,2161,13960],[6706,4205,48578],[6707,4303,6531],[6708,771,72510],[6709,898,2644],[6710,4745,52699],[6711,3798,60925],[6712,2631,88518],[6713,3980,13626],[6714,1560,40333],[6715,6235,80147],[6716,1408,99755],[6717,80,38663],[6718,4051,69335],[6719,5048,14459],[6720,498,30091],[6721,5049,65330],[6722,3857,39604],[6723,5317,90061],[6724,2786,25639],[6725,1709,51183],[6726,3229,54245],[6727,5699,41970],[6728,3636,35683],[6731,2458,39685],[6732,1670,93848],[6733,4874,49756],[6735,3824,83283],[6736,512,3353],[6738,4508,13094],[6739,6697,84542],[6740,3116,44429],[6741,3890,88230],[6742,3498,81728],[6743,1690,71900],[6744,3674,78901],[6745,1303,79316],[6746,1463,46090],[6747,1323,79763],[6748,4859,89495],[6749,6218,14349],[6750,2612,71919],[6751,743,37056],[6752,1419,33194],[6753,1895,29654],[6754,5197,87042],[6755,1214,26316],[6756,6743,54274],[6757,1197,94343],[6758,4727,54582],[6759,5281,48557],[6760,6084,90263],[6761,4121,45360],[6762,1808,46653],[6763,4697,94516],[6764,2112,3329],[6765,4121,38026],[6766,3692,20532],[6767,5274,78118],[6768,3512,81941],[6769,690,29666],[6770,5097,81144],[6771,1025,7070],[6772,2612,2643],[6773,1969,25503],[6774,3115,91838],[6776,4945,70448],[6779,750,88810],[6780,1766,34400],[6781,5888,3226],[6782,4036,14115],[6783,6069,55110],[6784,3627,75190],[6786,1882,90635],[6787,6474,1232],[6788,2553,65911],[6789,3139,6364],[6790,4287,10867],[6791,1663,69859],[6792,2218,1117],[6793,1149,27722],[6794,6357,29919],[6795,2944,40939],[6796,154,95644],[6797,4614,76653],[6798,3292,51745],[6799,3405,13782],[6800,4121,49218],[6801,6494,94631],[6802,4690,26868],[6803,4601,23647],[6804,5432,31702],[6807,6638,99188],[6808,5717,33093],[6809,3987,6352],[6810,5216,71564],[6811,148,85301],[6812,1565,35866],[6813,420,27390],[6814,4374,41476],[6815,2438,55112],[6817,1961,58739],[6818,6212,30980],[6819,3017,75091],[6820,1589,25223],[6821,6792,71034],[6822,5436,83968],[6823,147,73975],[6824,2256,19127],[6825,6639,56352],[6826,5349,32012],[6829,4721,61634],[6830,3584,99414],[6831,3124,70592],[6832,300,29430],[6833,4983,61678],[6834,3630,22955],[6835,5193,92565],[6836,6377,50961],[6837,470,53502],[6838,3690,78789],[6839,4072,63301],[6840,2716,65757],[6841,3251,6200],[6842,3938,51657],[6843,4849,20468],[6844,1800,81949],[6845,5384,8936],[6846,2012,68115],[6847,781,10448],[6848,3945,10834],[6850,5911,82141],[6851,661,18909],[6852,762,73681],[6853,854,83107],[6854,6601,77128],[6855,3153,965],[6856,1319,18038],[6857,3278,10213],[6858,1862,33617],[6859,4544,26380],[6860,2765,38997],[6861,4287,87717],[6862,3179,77443],[6863,4117,13696],[6864,552,98928],[6865,509,35295],[6866,1758,97501],[6868,702,53933],[6870,6630,96573],[6871,1511,46852],[6872,4030,12005],[6873,3752,70067],[6874,369,76703],[6875,4535,15291],[6877,6117,16403],[6878,650,19122],[6879,981,23035],[6880,2747,27962],[6881,6816,15190],[6882,165,56959],[6883,740,79318],[6884,4348,42288],[6885,2789,62477],[6886,5714,72554],[6887,3518,6554],[6888,229,13925],[6889,4106,60144],[6890,6617,32054],[6891,4441,38326],[6892,6010,32661],[6893,4536,66514],[6894,311,71942],[6895,459,14287],[6896,1532,35782],[6898,2725,1437],[6899,2127,65784],[6900,5263,29344],[6901,470,6256],[6902,3909,7587],[6903,4269,27110],[6904,3228,745],[6905,1421,10370],[6906,3942,83095],[6907,594,56417],[6908,106,76181],[6909,618,22798],[6911,4326,27948],[6912,2884,95230],[6913,1831,23658],[6914,4393,28817],[6916,2655,46683],[6917,1540,58973],[6918,503,37063],[6919,5972,54531],[6920,6909,24603],[6921,5869,92588],[6922,5414,46279],[6923,1962,10268],[6924,2695,6960],[6925,2054,79840],[6926,4365,79790],[6927,3440,86753],[6928,3054,4759],[6930,5641,42935],[6931,5593,31134],[6932,18,29305],[6933,2221,76459],[6935,370,84411],[6936,2249,57170],[6937,2488,66222],[6938,2357,47839],[6939,2088,3362],[6940,5190,93914],[6941,4711,84037],[6942,6281,68702],[6943,6706,73528],[6944,3393,33470],[6945,4709,3366],[6946,2214,63425],[6947,2411,60533],[6948,3429,43985],[6949,5399,23654],[6950,4771,92968],[6951,1905,9006],[6952,4396,46851],[6953,2389,30022],[6954,5048,64363],[6955,6782,15306],[6956,3627,64845],[6957,6918,59271],[6959,2767,38119],[6960,5020,29628],[6961,6255,49980],[6962,2172,1653],[6964,933,14509],[6965,4368,98661],[6966,4377,38391],[6968,4083,71561],[6969,695,86678],[6970,3447,44385],[6971,2591,46667],[6972,5952,13494],[6973,1145,36266],[6975,3445,5171],[6976,3534,12973],[6978,5873,27557],[6979,6321,59111],[6980,796,4426],[6982,1749,85280],[6983,1518,52688],[6984,3609,68385],[6985,663,61414],[6986,2729,94491],[6987,5271,63664],[6988,4932,96799],[6989,1913,30905],[6990,2270,886],[6991,464,97173],[6992,3967,63839],[6993,5902,85314],[6994,2104,79838],[6995,2131,54931],[6996,267,81735],[6998,5914,8149],[6999,6147,38483],[7000,365,21814],[7001,6799,98319],[7002,1948,14519],[7003,1192,20191],[7004,6832,94458],[7005,6376,16224],[7007,144,70551],[7008,6118,24548],[7009,6265,43463],[7010,3306,44568],[7011,5266,63542],[7012,4354,14715],[7013,3399,91319],[7014,2832,85469],[7015,5922,68580],[7018,4846,23952],[7019,169,58906],[7020,4662,96616],[7021,5403,69663],[7022,6577,83406],[7023,2358,97235],[7024,2952,51711],[7025,3473,18344],[7026,2158,52291],[7027,1018,62253],[7028,1429,38372],[7029,679,11258],[7030,4303,14515],[7031,6342,61828],[7032,5927,82973],[7033,2482,45251],[7034,3541,26967],[7035,6626,29750],[7036,5968,78974],[7038,1553,93744],[7040,2724,68980],[7042,4575,27124],[7043,6705,14069],[7044,2249,98524],[7045,244,9622],[7047,2622,65920],[7048,4110,77707],[7049,5613,48970],[7050,4233,93782],[7051,4903,80330],[7052,720,93268],[7053,3331,14305],[7055,803,4809],[7056,5492,59661],[7057,6260,76438],[7058,3062,37621],[7059,64,6264],[7060,5688,74517],[7061,4019,21481],[7062,91,32251],[7063,6715,39112],[7064,5747,48551],[7065,2201,87652],[7066,3224,7848],[7068,6236,58189],[7069,5236,70158],[7070,1896,89285],[7071,6271,94612],[7072,5848,27056],[7073,2084,69075],[7074,5488,24149],[7075,3528,53223],[7076,99,76922],[7077,6527,49875],[7078,6120,6072],[7079,578,8753],[7080,5822,2798],[7083,285,28902],[7086,185,44307],[7087,2075,41143],[7088,4285,12469],[7089,1713,40581],[7090,1096,14010],[7094,520,35227],[7095,3357,50016],[7096,790,20141],[7097,5121,40779],[7099,1623,41275],[7100,2610,44787],[7101,7013,52485],[7102,742,24166],[7104,1759,91290],[7105,5019,78012],[7106,3281,93594],[7107,6849,35987],[7108,6533,98050],[7109,4262,45879],[7110,4234,71845],[7111,4354,2323],[7112,2582,62696],[7113,1216,58878],[7115,926,34571],[7117,5600,7290],[7118,2970,95320],[7119,1198,95017],[7120,6602,70793],[7121,1132,9360],[7122,5841,30553],[7123,750,16481],[7124,1885,71597],[7125,368,12397],[7126,459,36876],[7127,1262,36273],[7128,676,26352],[7129,411,662],[7130,6288,23561],[7131,211,71512],[7132,3102,78956],[7133,6382,81898],[7134,3674,35828],[7136,1280,61240],[7137,445,18443],[7138,3211,52096],[7139,4252,19571],[7140,4567,58461],[7142,3379,7425],[7143,1973,87972],[7144,989,99684],[7145,1910,36918],[7148,3112,99457],[7149,3816,78299],[7150,2948,45584],[7151,6947,37118],[7152,6362,55903],[7153,1629,58749],[7154,5846,83641],[7155,6646,24005],[7156,5570,43100],[7157,6804,22524],[7159,247,75757],[7160,4999,649],[7161,2627,24208],[7162,5955,9038],[7163,4211,58669],[7164,5819,14253],[7165,6785,14745],[7166,2231,71036],[7167,937,8064],[7168,713,38225],[7169,3853,64534],[7170,5687,83117],[7171,3267,63689],[7172,3810,26488],[7174,7003,86115],[7175,6908,24672],[7176,5914,16445],[7177,324,74196],[7179,7031,2171],[7180,5216,77408],[7181,2085,39812],[7182,5388,60],[7185,4702,28854],[7186,6136,82039],[7187,863,59927],[7188,3825,67758],[7190,30,30982],[7191,2720,1309],[7192,6313,5048],[7193,1628,35663],[7195,2881,65594],[7196,4100,46054],[7197,3219,84230],[7199,6171,25456],[7201,3628,34670],[7203,352,64500],[7205,5968,73793],[7206,1852,23924],[7207,6180,17016],[7208,6658,76526],[7209,6173,91313],[7210,6588,35518],[7211,444,17566],[7212,1526,10648],[7213,6286,87551],[7215,4977,3904],[7216,5374,99146],[7217,6442,33889],[7218,4494,44627],[7219,4097,8968],[7220,6175,72543],[7221,4350,73088],[7222,5081,97813],[7223,4674,8759],[7224,5877,59916],[7225,3928,2736],[7226,4888,75724],[7227,4659,57027],[7228,2097,3823],[7229,5631,4253],[7230,4701,21050],[7233,7105,53185],[7234,3852,79065],[7235,5884,95064],[7236,869,28313],[7237,3267,3999],[7238,2164,86713],[7239,231,22062],[7240,2421,83363],[7241,197,21241],[7242,733,20986],[7243,483,63486],[7244,4487,71890],[7245,4774,56788],[7246,2767,67430],[7247,1361,37955],[7248,649,52613],[7249,5017,71705],[7250,6291,72345],[7251,882,65835],[7252,6417,91169],[7253,5217,53418],[7255,374,18001],[7256,1074,87384],[7257,1494,93830],[7258,3116,71535],[7259,2613,3430],[7260,3806,66452],[7261,4734,28905],[7264,4907,55590],[7265,6501,81273],[7266,5645,65019],[7267,7131,62268],[7268,4588,83004],[7270,1934,62830],[7271,2506,3662],[7272,6319,77256],[7273,1919,26905],[7274,754,39878],[7275,4730,95692],[7276,6127,97778],[7277,6453,37498],[7278,3721,72944],[7279,4825,56457],[7280,1876,34099],[7281,575,9231],[7282,3433,71877],[7284,3994,26975],[7285,5022,8318],[7286,885,17536],[7287,5444,18110],[7288,7000,5668],[7289,1484,33072],[7291,3210,66538],[7292,713,14140],[7293,3587,41426],[7294,6568,75264],[7295,4701,82058],[7296,5013,73693],[7297,3461,47878],[7298,1205,39730],[7299,4575,95839],[7300,535,58931],[7301,3175,82294],[7302,4586,12164],[7303,1943,72862],[7304,2654,14115],[7305,5634,75419],[7306,3358,75282],[7307,4913,50864],[7308,813,71356],[7309,3673,58678],[7310,4008,43938],[7311,2704,2891],[7312,1344,6637],[7313,6304,62801],[7314,3585,96358],[7315,2679,77749],[7316,6281,36263],[7317,1150,14201],[7318,4795,18766],[7319,3075,59191],[7320,7095,7567],[7321,981,79158],[7322,4140,50131],[7323,6440,16349],[7324,3354,94130],[7325,4458,58149],[7327,2680,39547],[7328,4830,51481],[7329,1773,11453],[7331,3428,87681],[7332,3301,76951],[7333,5653,92212],[7334,2880,55916],[7335,4056,16515],[7336,6137,24377],[7337,5191,68364],[7339,927,99978],[7340,3082,36794],[7342,5383,62929],[7343,554,22877],[7345,5567,70829],[7346,5204,51894],[7347,1537,35626],[7348,1752,6008],[7349,6017,6366],[7350,5365,42667],[7351,636,46814],[7352,5937,9331],[7353,5392,71827],[7354,5020,70539],[7355,5664,90889],[7356,3243,95683],[7357,4181,71185],[7358,2420,56592],[7359,5691,76614],[7360,6072,46872],[7361,7100,27918],[7362,1248,82949],[7363,3081,78495],[7364,3315,48113],[7365,2567,67023],[7366,6385,74426],[7368,1094,88345],[7369,4962,26740],[7370,6562,48051],[7371,5030,62083],[7372,639,46083],[7373,5043,24200],[7374,5018,91126],[7375,3488,9367],[7376,6834,7087],[7377,3361,27305],[7378,2509,13605],[7379,530,29459],[7380,1173,3619],[7381,1234,36954],[7382,2852,74969],[7383,6770,38588],[7384,1122,91817],[7386,3186,25809],[7389,5160,51409],[7390,4452,74478],[7391,5357,50790],[7394,2535,80278],[7395,2444,72140],[7396,372,83502],[7397,2197,58067],[7398,4851,27508],[7399,1202,40634],[7400,869,17633],[7401,6423,50320],[7402,3281,97008],[7403,5021,19693],[7404,2783,82171],[7406,3934,39446],[7407,1271,12071],[7408,4658,84562],[7409,787,56504],[7410,3484,45654],[7411,1852,73929],[7412,2906,30969],[7413,3177,7597],[7414,5884,44075],[7415,2251,19780],[7416,5571,85876],[7417,5077,73434],[7419,5060,36815],[7420,3516,65384],[7422,5670,37062],[7423,6721,57185],[7424,4665,52991],[7425,5635,51062],[7426,5682,66595],[7427,3143,24729],[7428,5910,9902],[7429,736,80449],[7430,7397,72213],[7431,6756,49734],[7432,2528,96368],[7433,2589,9502],[7434,1461,76460],[7435,2591,98659],[7436,1791,4334],[7437,1254,29009],[7438,6469,21811],[7439,4411,30189],[7440,6444,10768],[7441,2775,87299],[7442,1949,81004],[7443,4445,50449],[7444,5760,85132],[7445,2218,72797],[7446,574,26780],[7447,4229,18244],[7448,3796,80642],[7450,7274,94147],[7451,5856,57220],[7453,2748,42775],[7455,1357,82363],[7456,5827,80071],[7457,3542,53744],[7458,2202,1070],[7459,2335,39871],[7460,5051,11197],[7461,29,68817],[7462,5642,80666],[7463,3257,48081],[7464,5133,60903],[7465,4283,15551],[7466,2123,58229],[7467,819,60260],[7468,6171,88023],[7469,4671,69211],[7470,5140,78605],[7471,1232,67700],[7472,1955,43412],[7473,3629,59743],[7474,4419,43620],[7475,551,80029],[7476,3009,91971],[7477,5137,69945],[7478,1061,21303],[7479,6493,69920],[7480,5907,9795],[7481,1210,52572],[7482,1935,44453],[7483,4956,75307],[7484,559,2959],[7485,3710,47868],[7486,3274,64558],[7487,455,60232],[7488,6338,54016],[7489,4979,83419],[7490,2821,392],[7491,2603,42544],[7492,4892,16879],[7493,1450,55373],[7494,825,80997],[7495,4369,17390],[7496,1956,64984],[7497,5934,78955],[7498,6456,40908],[7499,5675,26644],[7500,5544,80752],[7501,2103,1693],[7502,4876,27931],[7505,3909,22280],[7506,5553,43058],[7507,2273,93845],[7508,2371,42568],[7509,1416,62908],[7510,983,96230],[7511,2995,81084],[7512,5367,96785],[7513,4134,70482],[7514,682,86528],[7515,4326,21879],[7517,110,39235],[7518,6003,16773],[7519,1345,79696],[7520,2503,58237],[7521,5746,67410],[7522,3635,1592],[7523,276,80350],[7525,4086,30983],[7526,2418,66574],[7527,7100,35167],[7528,3106,10625],[7529,6304,41896],[7530,4584,21475],[7531,585,83855],[7532,1894,13863],[7533,716,93222],[7534,5650,6443],[7535,1194,40844],[7536,7370,47510],[7537,5711,58427],[7539,1585,24384],[7540,4719,45585],[7541,6297,82570],[7542,909,23181],[7543,2125,17127],[7544,6924,83137],[7545,7430,92128],[7546,116,53442],[7547,4918,70783],[7548,2351,60175],[7549,3588,13453],[7550,1354,34650],[7551,3906,25162],[7552,3612,79528],[7553,5125,16833],[7554,6956,12748],[7555,1846,90022],[7556,187,62181],[7557,5050,45629],[7558,3601,42244],[7560,6775,473],[7562,5284,79736],[7563,6372,14160],[7564,996,644],[7565,463,52869],[7566,2333,10055],[7567,937,72678],[7568,4801,57526],[7569,6528,7799],[7570,950,64791],[7571,6951,37479],[7572,2837,21022],[7573,2206,82612],[7574,6245,55543],[7575,2380,87952],[7576,782,18904],[7578,2110,23105],[7579,6721,27841],[7580,4008,39601],[7581,473,84049],[7582,6046,67048],[7583,2674,77086],[7584,3454,42360],[7585,6294,92603],[7586,4622,30264],[7588,2141,43024],[7589,169,81049],[7590,4139,82845],[7592,6630,15960],[7594,4153,16405],[7596,7436,44572],[7597,3045,33897],[7598,7479,68192],[7599,1288,53175],[7600,1951,45909],[7601,1980,26915],[7603,1004,48892],[7604,6422,94035],[7605,6989,20674],[7606,2780,74536],[7607,3916,32290],[7608,3843,88753],[7609,2716,44258],[7610,4545,39739],[7611,4805,18767],[7612,3297,64831],[7613,2059,91099],[7615,6718,39897],[7616,2616,80284],[7617,1156,70078],[7618,652,2154],[7619,5902,69070],[7620,2279,25177],[7621,5325,66856],[7622,5436,42968],[7623,5059,13699],[7624,2722,58519],[7625,1577,60129],[7626,3102,78380],[7627,1643,83247],[7628,3394,60047],[7629,4540,203],[7630,2620,88249],[7631,1791,19083],[7632,5875,95921],[7633,3166,33735],[7634,1856,82394],[7635,7598,90185],[7636,4288,57184],[7638,7146,41490],[7639,1307,30863],[7640,1320,72327],[7641,4070,89072],[7642,2262,63624],[7643,1995,35853],[7644,7377,19615],[7645,6839,2078],[7646,5231,11180],[7647,1729,16069],[7648,7041,94359],[7649,7043,20461],[7650,3573,98154],[7651,3572,47925],[7652,7285,47354],[7653,1662,75266],[7655,1801,93592],[7656,4041,82014],[7657,7356,8395],[7658,795,29155],[7659,2548,73408],[7660,2297,97061],[7662,6715,96316],[7663,258,92940],[7664,2756,37967],[7665,4909,95206],[7666,3420,66564],[7667,6983,16762],[7668,746,38630],[7670,7634,67086],[7671,3075,40474],[7672,1507,25908],[7673,3445,66470],[7675,3006,37940],[7676,5709,54921],[7677,1189,52177],[7678,5377,59416],[7679,7246,36880],[7680,6899,65194],[7681,1698,87095],[7682,3471,66146],[7683,356,70105],[7684,1548,63250],[7685,6300,29288],[7686,5318,44248],[7687,513,88224],[7688,3289,86968],[7689,3203,25979],[7690,1112,67044],[7691,586,93079],[7692,494,70507],[7693,4754,76410],[7694,878,19955],[7695,1337,83033],[7696,2918,79173],[7697,4422,87776],[7699,1701,3874],[7700,577,14300],[7701,2295,64653],[7702,6443,8928],[7704,1632,28948],[7705,3561,32491],[7706,5085,56262],[7707,7545,4498],[7708,1002,19796],[7709,276,23476],[7710,4313,3548],[7711,7062,89334],[7712,7060,56196],[7713,5217,31108],[7714,4165,63524],[7715,3874,68647],[7716,5760,57946],[7717,4991,69147],[7718,1813,86155],[7719,4722,38612],[7720,5406,96297],[7721,3254,16273],[7722,1530,2407],[7724,7201,48508],[7725,266,10727],[7726,593,57911],[7727,4515,43881],[7728,2469,88088],[7729,2289,42831],[7730,6889,44214],[7731,6039,83123],[7732,5460,76160],[7733,4285,77467],[7734,1321,79801],[7735,4829,14336],[7737,3660,83472],[7738,2384,50168],[7739,7297,25668],[7740,3619,15617],[7741,5025,58818],[7742,283,75577],[7743,5998,26709],[7744,5647,35492],[7745,3813,43618],[7746,6040,11596],[7747,5892,71711],[7749,817,87342],[7750,5618,30414],[7751,2650,75591],[7752,2452,88823],[7753,2182,17067],[7754,5633,94752],[7755,7261,29225],[7756,5980,77447],[7757,6405,68435],[7758,19,12287],[7759,7017,20493],[7760,4846,28875],[7761,4838,56732],[7762,4441,59482],[7763,7326,62081],[7764,3521,17379],[7765,5236,92076],[7767,3892,62620],[7768,829,42567],[7770,1415,43626],[7771,5212,55161],[7772,151,2413],[7774,334,77026],[7775,7531,6344],[7776,1455,52030],[7777,1216,40200],[7778,5212,66789],[7780,1505,36182],[7781,6828,34281],[7782,6216,9532],[7783,4444,33031],[7784,3649,85426],[7786,7089,95167],[7787,2720,73133],[7788,1792,67250],[7789,5879,46402],[7790,2409,87646],[7792,4747,65303],[7793,3200,18540],[7794,1157,23873],[7795,6531,80140],[7796,650,19124],[7797,5769,60286],[7798,1215,14953],[7799,3146,63861],[7800,91,75843],[7801,4986,70026],[7802,4345,68943],[7803,5361,71601],[7804,7699,63426],[7805,2732,62013],[7806,4905,59170],[7807,7722,23722],[7808,1775,33194],[7810,6779,84535],[7811,4597,83502],[7812,1203,6514],[7813,5135,86768],[7814,3987,64751],[7816,6639,64578],[7818,2595,33734],[7819,4170,25552],[7820,2283,58005],[7821,1315,39803],[7822,2879,13336],[7824,6833,55718],[7825,4120,65896],[7826,6811,35936],[7827,2030,33876],[7828,442,44438],[7829,6057,90253],[7830,1028,31921],[7832,1816,39583],[7833,4855,91517],[7834,1370,95036],[7835,6694,4207],[7836,4532,63410],[7837,545,11765],[7838,1872,72142],[7839,4030,83862],[7840,4089,96815],[7842,2284,79938],[7843,2877,11349],[7844,5190,34587],[7845,2467,53948],[7846,5543,26779],[7847,5098,21390],[7848,5454,1223],[7849,5629,12728],[7850,7224,77079],[7851,1258,71783],[7852,6634,51337],[7853,7011,61575],[7854,3785,26753],[7855,6390,70877],[7856,3135,88881],[7857,1091,17516],[7858,3763,62760],[7859,2830,45349],[7860,1696,92670],[7861,5714,28867],[7862,2708,74009],[7863,6067,32060],[7865,6265,40888],[7866,2430,27159],[7867,2103,20449],[7868,713,80818],[7869,6266,66536],[7870,5928,94426],[7872,4653,98689],[7873,7559,20798],[7874,7703,49943],[7875,7173,4566],[7877,6111,61448],[7878,1783,18070],[7879,3004,52971],[7880,3312,26430],[7881,6727,34853],[7882,6439,87150],[7883,691,2654],[7886,4881,6706],[7887,7170,15335],[7888,2358,30730],[7889,5158,79023],[7890,3826,63516],[7891,2582,93328],[7893,2390,85931],[7894,6194,44865],[7896,2113,80035],[7897,7136,80195],[7898,1188,60623],[7899,2939,14128],[7900,6602,30411],[7901,4686,19851],[7903,3725,86107],[7904,904,15639],[7905,2834,40249],[7906,2686,49338],[7907,7005,88044],[7908,3080,77360],[7909,6575,44462],[7910,2526,29628],[7911,6882,73375],[7913,5160,90946],[7914,2455,93511],[7915,1279,92105],[7916,6783,7617],[7917,3629,68277],[7918,679,3856],[7919,1655,21094],[7920,6081,33350],[7921,7661,87463],[7923,791,87623],[7925,5923,75829],[7927,5557,83789],[7930,6827,82813],[7931,1995,50772],[7932,643,79487],[7934,3401,91049],[7935,2640,68686],[7936,2137,59998],[7937,3865,38253],[7938,110,44294],[7939,5219,1962],[7940,5177,27210],[7941,7522,53137],[7942,1180,78915],[7943,1346,73349],[7944,6285,16626],[7945,1542,28051],[7946,1915,40024],[7947,5819,57922],[7948,786,14264],[7949,3365,94909],[7950,2546,25910],[7951,6497,68501],[7952,6502,66762],[7953,1822,76697],[7955,6090,78093],[7956,4883,4979],[7957,5163,4421],[7958,504,79522],[7959,6578,39842],[7960,1692,34375],[7961,7848,37773],[7962,3973,94733],[7963,6947,88370],[7964,5050,85714],[7965,3184,77827],[7966,2117,27676],[7967,3741,59102],[7968,1373,66243],[7969,7786,98208],[7970,1957,98730],[7971,5851,34530],[7972,7238,82164],[7973,4003,29561],[7974,2652,9985],[7975,3977,45061],[7976,5739,11511],[7978,7218,47361],[7980,392,83240],[7981,5424,59054],[7982,1822,32560],[7984,2685,85579],[7985,4327,21348],[7986,2191,24142],[7988,4503,77468],[7989,3546,7442],[7990,3100,39126],[7991,933,34045],[7992,5039,81981],[7993,6491,83294],[7994,5375,78605],[7995,5569,61770],[7996,3639,43887],[7997,1964,66495],[7999,7368,23321],[8001,26,37015],[8002,2985,83723],[8003,246,31023],[8004,1795,69653],[8005,3119,4728],[8007,6693,8981],[8008,1368,59077],[8009,259,7497],[8010,2614,92087],[8011,4853,30401],[8012,1868,56307],[8013,5743,41096],[8014,2406,37161],[8015,3458,77668],[8017,6096,98489],[8018,4022,60600],[8019,7813,13046],[8020,6765,36040],[8021,6575,44515],[8022,6039,97023],[8023,5219,44061],[8025,4336,25573],[8026,3210,57070],[8027,2747,34455],[8028,5282,59785],[8030,116,96426],[8031,3556,72609],[8032,4774,14845],[8033,6470,65735],[8034,4152,51195],[8035,8012,55688],[8036,1704,57320],[8037,7252,64828],[8039,6946,21775],[8040,1420,60287],[8041,5164,35549],[8042,5166,76007],[8043,4844,94473],[8044,4288,42680],[8045,7285,14811],[8046,6402,53776],[8047,5537,95535],[8048,6351,74808],[8049,5542,15943],[8050,1101,7426],[8051,7188,33215],[8052,335,53642],[8053,3434,68357],[8054,7013,46254],[8055,4376,7295],[8056,1966,75883],[8057,4270,70656],[8058,4539,18053],[8059,5350,81478],[8060,2371,80175],[8061,5006,73718],[8062,3754,59414],[8063,3186,41075],[8064,6404,46720],[8065,5093,56392],[8066,2032,7100],[8067,2439,45812],[8068,3613,59786],[8069,3223,55117],[8070,1254,42205],[8071,4597,83427],[8072,7883,55328],[8073,3897,18621],[8075,2449,80083],[8076,1577,42168],[8077,1490,35313],[8078,1280,7833],[8079,5025,1230],[8080,7697,24993],[8081,7766,73707],[8082,6465,81409],[8083,2933,4909],[8084,5339,47693],[8085,1317,69412],[8087,7687,82487],[8088,1366,57881],[8089,6776,73146],[8090,3643,80818],[8091,5794,28694],[8092,1914,52915],[8093,733,77707],[8094,1823,21221],[8095,7701,35108],[8096,5496,16531],[8097,2965,43208],[8098,1771,93438],[8099,7638,31598],[8100,800,73263],[8101,277,87658],[8102,4057,92823],[8103,6957,19195],[8104,5191,51704],[8105,6297,5780],[8106,7687,77807],[8107,6535,70918],[8108,3935,23396],[8109,5712,9069],[8110,600,15920],[8111,3223,70215],[8112,7482,7074],[8113,2450,52568],[8114,1953,1511],[8115,2079,41627],[8116,5634,22694],[8117,8052,78012],[8118,5788,32531],[8119,7941,31406],[8121,4483,28774],[8122,7353,92410],[8123,556,23587],[8124,2931,49170],[8125,3585,47872],[8126,3700,12444],[8127,4383,42649],[8128,5249,8549],[8129,5662,99237],[8130,5019,37831],[8131,5131,17178],[8132,2304,31267],[8133,3525,11303],[8135,5865,51106],[8136,3535,92894],[8137,8056,97159],[8138,6830,14322],[8139,4747,58402],[8140,2027,93155],[8141,6368,85670],[8142,5210,87742],[8143,4323,89984],[8144,1945,41683],[8146,142,8752],[8148,537,24404],[8149,2286,21761],[8150,4896,7604],[8151,920,54057],[8152,7929,22855],[8153,3472,55723],[8154,2418,8061],[8155,6862,31979],[8156,5231,19480],[8157,5931,98556],[8158,2075,79282],[8159,650,4855],[8160,7166,14655],[8161,242,28671],[8162,2647,33827],[8163,2814,94900],[8164,5229,50047],[8165,5623,64514],[8166,412,40925],[8167,4580,7831],[8168,724,14078],[8169,4366,69586],[8170,7934,94792],[8172,828,3835],[8173,7506,26999],[8174,1196,34948],[8175,2848,42262],[8176,1938,31681],[8177,3607,14314],[8178,2372,33273],[8179,6643,38491],[8180,1381,93267],[8181,648,11986],[8182,27,82046],[8183,6124,72812],[8184,7345,42407],[8185,7489,27532],[8186,5035,11188],[8187,6288,39968],[8188,7602,23052],[8189,3826,44600],[8190,462,34709],[8191,2898,69025],[8192,4495,12533],[8193,5279,46135],[8194,3336,60171],[8195,6686,38620],[8196,4465,63477],[8197,1315,86333],[8198,3570,95041],[8200,5134,94844],[8201,4676,29207],[8202,364,18913],[8203,2971,31855],[8204,7952,2553],[8205,986,79858],[8206,2673,66053],[8208,1361,7343],[8209,8037,45157],[8211,6572,22719],[8212,387,83808],[8213,321,88549],[8215,7040,75103],[8216,8145,75916],[8217,1541,38218],[8218,7270,72050],[8219,2115,50888],[8220,3971,63508],[8221,5656,36831],[8222,6160,76820],[8223,7395,33569],[8224,7633,28066],[8225,1893,34650],[8226,1211,7852],[8227,1479,79168],[8228,4778,96107],[8229,867,84742],[8230,2543,69112],[8231,6413,36135],[8232,7051,77888],[8233,3502,65746],[8234,4708,40027],[8235,7329,92612],[8236,5891,32857],[8238,4909,33272],[8239,3222,31829],[8240,761,48542],[8241,8186,34072],[8242,4947,50549],[8243,4556,17168],[8244,2982,9537],[8245,2722,49080],[8246,738,28018],[8247,6552,22612],[8248,524,96031],[8249,1308,96053],[8250,3348,23142],[8251,4631,0],[8252,1131,34386],[8253,3752,60735],[8255,7742,9792],[8256,2824,59355],[8257,8001,25573],[8258,5734,35112],[8259,4752,80219],[8260,3431,63855],[8261,5420,26630],[8262,3248,73954],[8263,7860,12527],[8264,6958,98478],[8265,4605,33127],[8266,2174,17213],[8267,2306,16730],[8268,4972,79737],[8269,4312,62423],[8270,840,71573],[8271,4765,49527],[8272,5237,42101],[8273,2020,23414],[8274,2644,38037],[8276,6517,5993],[8277,1190,37882],[8278,2639,58519],[8279,3673,94724],[8280,1454,58009],[8281,6477,91941],[8282,5333,66168],[8283,8135,46233],[8284,7337,53832],[8285,5444,31038],[8287,7775,351],[8290,1185,53],[8291,3663,24471],[8292,8090,76854],[8293,4053,28914],[8294,6956,58233],[8295,4662,2547],[8296,3637,1158],[8297,7848,86276],[8298,6490,76649],[8299,155,37772],[8300,4706,51966],[8301,6242,55512],[8302,7135,82954],[8303,355,64280],[8304,351,5758],[8305,2131,44059],[8306,6499,97016],[8307,1501,78558],[8308,4858,24175],[8309,4787,43221],[8310,1476,45622],[8311,2176,37434],[8312,3716,82921],[8313,542,8829],[8314,92,61767],[8315,2076,40456],[8316,127,77458],[8317,7908,41990],[8318,5527,50976],[8319,3473,86482],[8320,4625,27461],[8322,2716,57168],[8323,5862,20890],[8324,5363,68997],[8325,5793,33551],[8326,5436,4722],[8327,3662,6065],[8328,2759,18911],[8329,4315,16277],[8330,3495,34667],[8331,1141,72571],[8332,5158,73360],[8333,5162,96257],[8334,3575,59560],[8335,6695,82693],[8336,6447,70014],[8337,3759,40928],[8338,1931,48645],[8339,3658,97351],[8340,1344,72077],[8341,353,66761],[8342,2980,93805],[8343,5129,28064],[8344,749,56576],[8345,4200,8894],[8346,6007,47683],[8347,3217,17622],[8348,2700,1424],[8349,4632,191],[8350,115,94951],[8351,1121,36763],[8352,2851,60064],[8353,7605,9744],[8354,7206,23277],[8355,740,7741],[8357,3453,83850],[8359,658,34478],[8360,3582,93160],[8361,8285,38181],[8362,2953,41402],[8364,3117,47800],[8365,5803,75642],[8366,7233,29052],[8367,2296,97154],[8368,8019,43966],[8369,6111,77869],[8370,1376,56888],[8371,1697,16439],[8372,4171,98711],[8374,2511,21350],[8375,3643,85461],[8376,6193,42432],[8377,4925,14756],[8378,3459,45729],[8379,818,55814],[8380,3383,54926],[8381,1088,79948],[8382,3746,89137],[8383,174,44018],[8384,5300,51277],[8385,1330,22521],[8386,6840,30664],[8387,4239,11192],[8388,20,75446],[8389,5054,82537],[8391,4585,52168],[8392,2948,75969],[8393,4187,11826],[8395,5308,76419],[8396,1652,1176],[8397,3211,23170],[8398,4625,31791],[8399,5346,41614],[8401,2320,31439],[8402,5495,71547],[8403,7340,11292],[8404,8013,74912],[8405,345,95392],[8406,7648,78389],[8407,5599,1703],[8408,6092,92701],[8410,3924,54725],[8411,5942,49602],[8412,3506,31547],[8414,2514,86912],[8415,5271,60371],[8417,4167,95505],[8418,199,63722],[8419,7703,84526],[8420,429,35127],[8421,6437,38010],[8422,8133,69081],[8423,6216,74716],[8424,4545,91578],[8425,7156,68351],[8426,6463,74779],[8427,6746,68783],[8428,3685,76819],[8429,8129,35524],[8430,3739,24709],[8431,4348,64538],[8432,5664,76273],[8433,5022,76374],[8434,5777,88935],[8435,1149,60247],[8436,2753,90699],[8437,805,52820],[8438,2606,91017],[8439,3969,90],[8440,8357,71482],[8441,5404,17625],[8443,6104,82902],[8444,1293,91559],[8445,3620,11107],[8446,2049,92997],[8447,3367,22638],[8448,4834,25974],[8449,981,54380],[8450,758,92366],[8451,1412,56730],[8453,7224,69480],[8455,5082,39265],[8456,5476,28434],[8457,5334,70936],[8458,2102,94066],[8459,5325,57043],[8460,3484,76807],[8461,6354,38679],[8462,1196,30707],[8464,7767,72804],[8465,1757,25016],[8466,3918,33554],[8467,5946,96151],[8468,5227,69924],[8469,1778,18199],[8470,877,29003],[8471,5749,58948],[8472,6074,87591],[8473,600,8921],[8474,3618,73528],[8475,639,63680],[8476,7520,24668],[8477,6398,9286],[8478,1641,97801],[8479,6644,14719],[8480,6909,5190],[8482,6824,89533],[8483,3072,80879],[8486,1948,92541],[8487,8125,58840],[8488,5098,58306],[8489,4,36004],[8490,234,30922],[8491,2550,51439],[8492,6532,48757],[8494,2085,1777],[8496,3671,86968],[8497,680,68654],[8498,3641,55704],[8500,3801,7283],[8501,3899,83316],[8502,4362,80553],[8503,1860,68646],[8504,3708,61500],[8505,4890,87295],[8506,5508,74837],[8507,1462,26938],[8508,6811,59182],[8509,2637,50534],[8510,7980,26018],[8511,4412,32585],[8512,2067,79682],[8513,8263,81416],[8514,3201,67555],[8515,895,41970],[8516,2074,78776],[8517,6614,59208],[8518,4095,34872],[8519,6424,20933],[8520,824,66663],[8521,2378,49570],[8522,3168,55046],[8523,3196,69634],[8524,4640,56251],[8525,4207,59730],[8526,7482,59268],[8527,685,11407],[8528,6835,16498],[8529,5154,17870],[8530,8233,15635],[8531,5109,18029],[8532,2428,77076],[8533,365,98899],[8534,6128,20114],[8535,6028,79454],[8536,2728,98100],[8537,4782,23659],[8539,2339,98445],[8540,695,43249],[8541,6230,70233],[8542,6677,31346],[8543,6603,6632],[8544,5074,25730],[8546,5302,39713],[8547,5837,81084],[8548,5737,76663],[8549,3973,55113],[8553,7621,72904],[8554,3093,19153],[8555,5989,58222],[8556,120,67440],[8557,951,69528],[8558,6301,39936],[8559,4598,6913],[8560,301,22599],[8561,6157,94983],[8562,8003,31396],[8563,8485,45222],[8564,2898,13825],[8565,1680,90154],[8566,264,60106],[8567,5813,12454],[8568,4515,30041],[8569,4125,45060],[8570,5154,16740],[8571,5738,12524],[8573,2612,45482],[8574,3954,46367],[8575,5034,91078],[8577,2466,58183],[8578,4569,41856],[8579,6414,54191],[8580,3619,11611],[8581,4858,48708],[8582,4242,35438],[8583,6826,86549],[8584,345,88850],[8586,4952,92197],[8587,4036,30018],[8588,3023,79889],[8589,2116,5345],[8590,3108,77405],[8592,5182,43488],[8593,2261,2415],[8594,5802,71864],[8595,2233,50720],[8596,5552,98655],[8597,7281,30479],[8599,7810,33814],[8600,4197,97185],[8601,1788,78567],[8602,3688,12576],[8604,4296,40686],[8605,8541,55353],[8606,6926,80440],[8607,2947,53141],[8608,4295,62898],[8609,6571,42592],[8610,6967,82244],[8611,2126,90530],[8612,8265,50372],[8613,4311,82987],[8614,8131,17614],[8615,1949,76526],[8616,5915,239],[8618,7722,64778],[8619,4886,11962],[8620,3081,53982],[8621,2553,72841],[8622,3351,23207],[8623,4649,38416],[8624,7819,18926],[8625,7240,42051],[8626,3953,18735],[8627,7003,70105],[8628,6653,46198],[8629,6947,16643],[8630,3610,62489],[8631,6546,40832],[8632,2147,18474],[8633,2698,76654],[8634,5044,75055],[8635,4757,29011],[8636,1171,15424],[8637,2185,82229],[8638,6284,52205],[8639,4536,41943],[8640,5799,52411],[8641,484,73697],[8642,6224,90157],[8643,6018,18012],[8644,5860,41088],[8645,2231,77275],[8646,4367,84692],[8647,4828,64075],[8648,2567,7239],[8649,2396,32315],[8650,2081,70910],[8651,6774,89116],[8652,6886,15609],[8653,3342,51834],[8654,5382,76728],[8655,6147,10304],[8656,8628,98333],[8657,4587,73039],[8660,992,11312],[8661,502,1138],[8662,4383,85708],[8663,2876,94706],[8665,7481,11062],[8667,2253,15466],[8668,3468,58895],[8669,3596,68820],[8670,3732,34885],[8671,7513,47657],[8672,1336,56378],[8674,126,63363],[8675,677,66672],[8676,5821,15635],[8677,1231,98827],[8678,7178,52395],[8679,241,93666],[8680,3346,5001],[8682,3889,89392],[8683,8643,86103],[8684,4625,3007],[8685,7358,65638],[8686,5327,79636],[8687,3975,75076],[8688,1500,73165],[8689,6647,36217],[8690,940,89764],[8691,764,63086],[8692,775,20685],[8693,5093,80222],[8694,7539,90371],[8695,3082,66391],[8696,2446,43855],[8698,3243,47102],[8699,2967,71641],[8701,3065,10953],[8702,7710,28658],[8703,6897,69104],[8704,399,68836],[8705,6597,70926],[8706,2425,97960],[8707,1169,3615],[8708,6611,10532],[8709,7182,42151],[8710,1405,67960],[8711,5248,93187],[8712,2874,3097],[8713,1916,31662],[8714,5822,24530],[8716,2908,13815],[8717,6814,96343],[8718,1079,26572],[8719,5446,77946],[8720,3581,10947],[8721,8227,14126],[8722,5960,94701],[8723,5326,68731],[8724,2218,56298],[8725,2847,68881],[8726,4963,32912],[8728,3183,28318],[8729,4776,27642],[8730,4233,1908],[8731,8616,14937],[8732,4772,24265],[8733,7230,36466],[8734,1240,95092],[8735,872,11269],[8736,2915,18901],[8737,2377,10712],[8739,2599,1881],[8740,155,41311],[8741,4677,37363],[8742,6513,11126],[8743,8162,4820],[8744,7805,95519],[8745,6120,63829],[8746,3188,16869],[8747,836,35390],[8748,1775,45973],[8749,522,5368],[8750,6322,9251],[8751,4471,55643],[8752,6626,50403],[8753,8024,95571],[8754,3323,59197],[8755,2291,45473],[8756,3114,52526],[8757,388,24992],[8758,5563,51978],[8759,7218,6357],[8760,5344,8340],[8761,2176,72374],[8762,4033,70258],[8763,1143,7313],[8764,2689,58343],[8765,2235,93233],[8767,5879,37843],[8768,2312,30671],[8769,5796,72930],[8770,7161,27728],[8771,7862,9074],[8773,3198,50889],[8774,4606,6197],[8775,4710,51354],[8776,7612,1382],[8777,1884,3629],[8778,3487,98048],[8779,6574,59667],[8780,117,55574],[8781,349,25483],[8782,1516,47162],[8783,1212,33464],[8784,8159,52538],[8785,3206,93851],[8787,7343,44539],[8788,2341,53199],[8789,1927,54602],[8790,5860,35869],[8792,4833,6643],[8793,3102,17884],[8794,3305,75371],[8795,2473,39923],[8796,6926,97007],[8797,8663,74094],[8798,5688,68226],[8799,657,30968],[8800,3960,36042],[8801,4282,75546],[8802,7330,51674],[8803,3465,61723],[8804,2766,78984],[8805,28,93252],[8806,1356,57665],[8807,1463,20965],[8808,6594,7253],[8809,4891,56972],[8810,1864,22797],[8811,3531,21268],[8812,7972,8378],[8813,6714,12076],[8814,4607,80558],[8815,1123,99224],[8816,3873,8858],[8817,1522,68735],[8818,8271,15922],[8819,7052,98483],[8820,1993,39530],[8821,4245,3484],[8822,6987,67920],[8823,7422,36592],[8824,459,97407],[8825,1212,88619],[8826,6335,90663],[8828,7684,78856],[8829,5996,52036],[8830,1740,38332],[8831,5691,43124],[8832,8796,18090],[8833,7388,81848],[8834,4232,49370],[8835,6507,63964],[8837,7758,50046],[8838,3626,57179],[8840,8294,35535],[8841,8142,68829],[8842,1715,54012],[8843,1545,90741],[8844,7464,62479],[8845,2408,54603],[8846,7495,85535],[8847,5670,31707],[8848,7064,14780],[8849,1504,40747],[8850,5337,66258],[8852,4880,29317],[8853,8001,89121],[8854,3191,43554],[8855,7005,23525],[8856,3472,44446],[8857,7367,41354],[8858,5262,31373],[8859,4149,23859],[8860,5233,81577],[8861,2702,57304],[8862,2874,39819],[8863,4882,45921],[8864,7892,20156],[8865,7378,48081],[8866,778,7971],[8867,768,10457],[8869,8694,16564],[8870,4345,37731],[8871,5498,59403],[8872,7972,32749],[8873,488,66404],[8874,5351,12021],[8875,6990,654],[8876,679,13028],[8877,1405,38148],[8878,6117,55783],[8879,927,21827],[8880,5052,47140],[8881,8595,60978],[8883,6037,29784],[8884,273,61342],[8885,5317,42025],[8886,7500,64816],[8887,8826,8009],[8888,2501,1823],[8889,6775,98308],[8890,5471,94791],[8891,2175,80125],[8892,4993,92106],[8893,4883,3632],[8894,3890,74275],[8895,5786,96363],[8896,3556,74007],[8897,5921,86409],[8898,1129,34168],[8899,1509,55597],[8900,7886,56100],[8901,3528,83208],[8902,4803,62742],[8903,4311,39080],[8904,2735,25155],[8905,5597,83242],[8908,4280,36020],[8909,8406,76794],[8910,7212,68853],[8911,3641,73928],[8912,711,96838],[8913,2887,10331],[8914,1020,68695],[8915,6289,90283],[8916,8317,8245],[8917,7007,41486],[8918,2517,55550],[8919,2733,19880],[8920,955,56141],[8921,7804,72407],[8922,6215,14335],[8923,2876,28993],[8924,4871,6810],[8925,7872,78252],[8926,3508,6326],[8927,6671,23073],[8928,6000,78719],[8929,6478,21246],[8932,7867,68246],[8934,3085,76721],[8936,78,74425],[8937,4302,34112],[8938,2369,68137],[8939,5627,32332],[8940,8112,30275],[8941,8282,78304],[8942,1159,71943],[8943,7950,20831],[8944,5931,70022],[8945,4223,96370],[8946,4606,56292],[8947,7832,46946],[8949,1666,89730],[8950,4937,2034],[8951,7971,74633],[8952,6679,26937],[8954,4416,99953],[8955,651,63448],[8957,7586,7881],[8958,3987,96922],[8959,1987,73928],[8960,2535,36843],[8961,8142,70421],[8962,2739,49951],[8963,5200,53361],[8964,973,23623],[8965,7307,48286],[8966,3796,31330],[8967,6207,16064],[8968,1749,30755],[8969,6284,88760],[8971,6282,86071],[8972,3496,50073],[8974,4777,13394],[8975,2695,13124],[8976,5897,3316],[8977,4090,16583],[8978,8038,7813],[8979,730,64208],[8980,1507,52073],[8981,8961,65723],[8982,1330,10078],[8983,7646,73115],[8984,2182,51418],[8985,5055,61395],[8986,7881,73439],[8987,2308,17289],[8988,8706,28060],[8989,3389,63310],[8990,3730,82619],[8991,8640,18979],[8992,2600,14022],[8994,3316,67639],[8995,7009,95678],[8996,6846,86882],[8997,6265,95094],[8998,8517,64030],[8999,5199,26245],[9000,1335,9539],[9001,4999,17893],[9002,5050,7625],[9003,5476,97931],[9005,3374,79589],[9006,2765,39577],[9007,5606,15065],[9008,906,96362],[9009,368,81866],[9010,7702,19065],[9011,8974,7426],[9012,4041,50525],[9013,3720,74518],[9014,4003,63249],[9015,4735,58168],[9016,8758,77756],[9017,4101,12846],[9018,7653,48953],[9019,2249,18285],[9020,3316,60513],[9021,1681,17139],[9022,2235,3711],[9023,3046,90671],[9024,3200,69148],[9025,3144,72620],[9026,5414,73875],[9027,222,42338],[9028,3933,88288],[9029,1754,50144],[9030,1233,89457],[9031,7274,31868],[9032,5676,26015],[9033,5391,78456],[9034,5781,64513],[9035,3636,22084],[9036,7770,31400],[9037,5734,3814],[9038,1398,61659],[9042,4381,94229],[9043,2045,51601],[9044,5382,25453],[9045,8624,97791],[9046,5224,58575],[9047,1002,16645],[9048,1692,54745],[9049,2152,14656],[9050,3060,86103],[9051,1240,14398],[9052,7672,67288],[9053,4581,15801],[9054,6973,48142],[9055,8945,96910],[9057,4010,34915],[9059,6719,3180],[9060,2525,30206],[9061,5112,2881],[9062,8275,69275],[9063,7908,73344],[9064,2823,80839],[9065,764,70322],[9066,1555,2699],[9067,1907,78876],[9068,2066,6151],[9069,4809,64238],[9070,812,50554],[9071,1527,64835],[9072,6992,88836],[9073,1670,25001],[9074,209,38889],[9075,7192,51402],[9076,7196,10629],[9077,5124,59597],[9079,7322,4040],[9080,3443,22354],[9081,2634,27435],[9082,1881,39696],[9083,202,6872],[9084,477,31989],[9085,8481,28984],[9086,7042,92756],[9087,195,744],[9088,7620,76454],[9089,2392,7890],[9091,1119,15607],[9092,6981,9850],[9094,5215,32594],[9095,1074,3118],[9096,7249,28472],[9098,4650,26511],[9099,3265,47039],[9100,450,75248],[9101,2342,38500],[9102,8191,14578],[9103,7244,32606],[9104,9054,73665],[9105,7867,11334],[9106,6049,31933],[9108,7264,8453],[9109,5419,79202],[9110,2247,73849],[9111,1712,47799],[9112,4973,74199],[9113,2778,11911],[9114,5480,21953],[9115,7169,85822],[9116,6591,13433],[9117,6945,52835],[9118,1198,91433],[9119,384,26513],[9120,7746,30766],[9121,5539,65278],[9123,8721,21336],[9124,6763,73973],[9125,641,95534],[9126,4168,93766],[9127,483,93449],[9128,8848,70645],[9129,7715,73059],[9131,3643,84988],[9132,8483,71128],[9133,7312,39522],[9134,2074,91223],[9135,4015,23532],[9136,740,49875],[9137,2694,62946],[9138,418,65143],[9139,4563,34368],[9140,740,60904],[9141,8735,62590],[9142,1024,8307],[9143,188,24386],[9144,2888,98820],[9145,7001,21435],[9146,2086,68471],[9147,2882,6330],[9148,1865,19934],[9149,5957,27799],[9150,4694,89318],[9151,2117,10707],[9152,5457,29735],[9153,3802,98992],[9154,3492,66486],[9155,6834,40962],[9156,7898,3402],[9157,2033,24791],[9158,123,62159],[9159,8813,13384],[9160,2536,65450],[9162,4598,58819],[9163,8849,33218],[9164,3241,19869],[9165,6035,22874],[9167,259,94153],[9168,5544,30344],[9169,1597,47005],[9170,4030,30303],[9172,776,79584],[9173,8814,10348],[9174,1640,94645],[9175,8045,46570],[9176,6716,16780],[9177,352,3040],[9178,8147,40541],[9179,4936,90576],[9180,3642,29764],[9181,6890,68472],[9182,2878,67971],[9183,4424,97066],[9184,4095,49041],[9187,8253,55669],[9188,7039,60885],[9189,1406,51041],[9190,5112,56522],[9191,1578,85588],[9192,4807,83239],[9193,2198,96683],[9195,5843,72426],[9196,6536,39117],[9197,4596,14801],[9200,1049,90826],[9201,5577,30225],[9202,211,62119],[9203,5515,67810],[9204,3536,78483],[9206,4207,30513],[9207,6381,65498],[9208,1898,58485],[9209,2704,86155],[9210,5961,98522],[9211,2088,86934],[9212,2533,2999],[9214,5970,32671],[9215,4002,42964],[9216,2320,13547],[9218,5471,35131],[9219,5958,74226],[9220,9119,17222],[9221,3269,47873],[9222,800,82346],[9223,6269,88567],[9224,2019,13083],[9225,4098,35542],[9226,979,7173],[9227,7474,56892],[9228,7960,5076],[9229,522,81192],[9230,63,23609],[9231,4270,22224],[9232,1958,66108],[9233,4054,72565],[9234,9005,14973],[9236,5296,42810],[9237,4404,53291],[9238,7467,58989],[9239,146,57759],[9240,8212,51014],[9241,8827,57998],[9242,3011,76259],[9243,4927,93879],[9244,7379,5567],[9245,6318,5242],[9246,2653,11495],[9247,5122,41924],[9248,7335,34885],[9249,2254,82453],[9250,2886,24099],[9252,2562,4684],[9253,3293,27734],[9254,7892,50169],[9255,6465,62731],[9256,3767,77724],[9257,1302,25992],[9258,5737,76978],[9259,594,25138],[9261,7986,90326],[9262,1127,40528],[9263,5687,91698],[9264,4657,35772],[9265,8846,54800],[9266,6123,71314],[9267,2804,73195],[9269,4976,97144],[9270,7345,79389],[9271,8249,32937],[9272,3506,36476],[9273,5268,50894],[9274,7714,79520],[9275,2841,19475],[9276,4003,54606],[9277,5325,86317],[9278,39,9882],[9279,6025,67865],[9280,3597,83048],[9281,6168,70030],[9282,1483,4292],[9283,4732,29062],[9284,528,58682],[9287,6095,89213],[9288,1759,42334],[9289,6683,86167],[9291,2740,6634],[9292,9252,92962],[9293,6518,86049],[9294,9108,37049],[9295,1573,67874],[9296,4312,5540],[9297,2125,42993],[9298,464,17462],[9299,7373,22665],[9300,9103,80821],[9301,3976,86743],[9302,7308,4829],[9303,5464,16993],[9305,4641,87690],[9306,6839,27252],[9307,3267,90434],[9308,2183,147],[9309,6787,19717],[9310,5361,54135],[9311,3691,85895],[9312,8553,91743],[9313,3869,10638],[9314,9262,23596],[9315,5859,47548],[9316,93,38082],[9317,3317,70763],[9318,6533,62186],[9319,5826,48109],[9320,7518,96274],[9321,8976,86150],[9322,4902,29358],[9323,6515,52858],[9324,6229,81661],[9325,8062,24814],[9326,6242,16550],[9327,5877,78379],[9328,3682,95452],[9329,1954,47890],[9331,94,22963],[9332,8508,49513],[9333,3725,43671],[9334,2628,65098],[9335,7238,50143],[9336,5943,6635],[9337,4549,82911],[9338,5424,38141],[9340,6003,49180],[9341,6926,5250],[9342,4763,68196],[9343,1721,503],[9344,8960,35945],[9345,2864,43077],[9346,7111,71728],[9347,5589,61755],[9348,7164,49122],[9349,5013,67958],[9350,5943,12798],[9351,6829,3039],[9352,3320,39006],[9353,7078,4688],[9354,6214,1377],[9355,3538,69525],[9356,3663,6572],[9357,2789,88453],[9358,4587,87803],[9359,9038,78917],[9360,1944,39363],[9361,5067,91808],[9362,3000,37165],[9363,3209,72305],[9364,7308,72603],[9365,2556,48968],[9366,4209,94424],[9367,7009,23833],[9368,649,78727],[9369,2504,80883],[9370,6502,67131],[9371,1182,308],[9372,8096,49521],[9373,2354,72847],[9374,7925,8570],[9375,5240,6274],[9376,5309,61034],[9377,7810,53624],[9378,7697,7954],[9379,1469,41284],[9380,6322,84267],[9381,3877,4327],[9382,5618,81957],[9383,5443,86557],[9384,2080,4568],[9385,6042,99053],[9386,1046,32127],[9387,8842,61765],[9388,2621,22519],[9389,8068,63603],[9391,9204,48777],[9392,2766,45332],[9393,2549,33520],[9395,8572,24995],[9396,7764,8961],[9398,4661,92126],[9399,9051,89092],[9400,7888,22928],[9401,6869,53320],[9402,5417,50925],[9404,1493,4258],[9405,3512,40519],[9406,3410,98089],[9407,7016,47653],[9408,1397,38079],[9409,8244,74077],[9410,3237,97013],[9411,1710,97956],[9412,8782,68407],[9413,9208,16280],[9414,797,14229],[9415,2318,29682],[9416,4579,22153],[9417,9363,93276],[9418,5683,95470],[9419,6625,62070],[9420,935,22101],[9421,8535,70312],[9423,3156,24344],[9424,3634,11931],[9425,7935,15720],[9426,8851,89891],[9427,4099,41800],[9428,5775,19894],[9429,1159,5122],[9430,8912,48100],[9431,5051,48135],[9432,7595,53295],[9433,8194,57768],[9434,9155,30110],[9435,5024,30766],[9436,1300,81897],[9437,4263,45091],[9438,3794,49228],[9439,8320,5497],[9441,6047,53009],[9442,8836,36254],[9443,2524,32234],[9444,8796,36167],[9445,224,9651],[9446,7087,93994],[9447,4334,36365],[9448,5225,12915],[9449,1702,10861],[9450,8489,58384],[9451,6256,49915],[9452,45,91203],[9453,7576,63444],[9454,3816,95745],[9455,7210,96605],[9456,6024,6192],[9457,7137,40896],[9458,1307,36280],[9459,8870,39154],[9460,1872,61382],[9461,2375,40443],[9462,1989,307],[9463,9288,46360],[9464,2387,59409],[9465,7304,70897],[9466,3389,26710],[9467,6624,49206],[9468,1471,45941],[9469,5461,1839],[9470,6626,84483],[9471,8124,8936],[9472,7792,9939],[9473,2353,23162],[9474,5379,20690],[9475,6291,39012],[9476,4344,73802],[9477,6644,61765],[9478,3025,10738],[9479,3019,65641],[9480,4884,94951],[9481,4771,46819],[9482,7308,57784],[9483,2112,17088],[9485,3683,20112],[9486,9213,72679],[9487,7876,65321],[9488,8002,90485],[9489,6720,72782],[9490,6728,14082],[9491,7742,98592],[9493,9368,98895],[9494,440,91805],[9495,286,33220],[9496,3904,28392],[9499,2899,81172],[9500,7955,58828],[9501,357,28801],[9502,2897,75533],[9503,7683,65245],[9504,443,21606],[9505,8289,41789],[9506,5586,53518],[9507,3298,8173],[9508,2636,28508],[9509,1530,21390],[9510,3495,24316],[9511,7635,53406],[9512,316,27638],[9513,3076,64504],[9515,4944,99127],[9516,6506,72195],[9517,4027,43850],[9520,5559,60185],[9521,1347,25368],[9522,1278,14947],[9523,7150,8104],[9524,2500,53511],[9525,5285,6328],[9526,2385,39086],[9527,112,34393],[9528,3835,9338],[9531,6671,77909],[9532,2215,19857],[9533,6186,68633],[9534,5053,19204],[9535,6367,76777],[9536,14,85453],[9537,7988,54193],[9538,2175,25616],[9539,1833,23053],[9540,6983,38663],[9541,7976,41659],[9542,6368,2244],[9543,5634,44019],[9544,434,96093],[9545,8201,41822],[9546,1991,2957],[9548,4293,43187],[9549,7064,9827],[9550,8905,40921],[9551,854,73968],[9552,6948,30288],[9553,5841,63745],[9554,6998,77607],[9555,3646,31517],[9556,8781,31040],[9557,2966,87041],[9558,140,1123],[9559,3997,17800],[9560,7985,7673],[9561,5571,20314],[9562,4629,52645],[9563,6143,75014],[9565,4131,89268],[9566,1210,89700],[9567,7046,38315],[9568,5074,93149],[9569,1262,20099],[9570,9145,77811],[9572,7015,85079],[9573,310,748],[9574,4319,52565],[9575,7223,95464],[9576,9242,85934],[9577,4337,73858],[9578,1687,70969],[9579,4373,69130],[9580,3568,81627],[9581,1144,28982],[9583,2099,97995],[9584,1103,21011],[9585,1375,42431],[9587,903,55866],[9588,8406,57310],[9590,3504,50877],[9591,5006,69867],[9592,1714,16187],[9593,8413,88641],[9594,7647,90210],[9596,1277,97477],[9597,5086,40135],[9598,8313,46653],[9599,2340,21795],[9600,6769,23782],[9602,1052,64068],[9603,1245,37787],[9604,7330,71576],[9605,3823,72943],[9606,5121,5949],[9607,1171,57005],[9608,4580,22846],[9609,4914,36388],[9610,814,5953],[9611,6883,81416],[9612,8917,53904],[9613,932,60034],[9614,1177,81212],[9615,4355,43346],[9616,8417,71363],[9617,4310,10020],[9618,9156,91320],[9619,3473,54043],[9620,4911,88095],[9621,670,22283],[9622,5530,45756],[9623,7431,97833],[9624,8315,49930],[9625,2102,62372],[9626,583,23719],[9627,1324,70821],[9628,9068,19941],[9630,909,38136],[9631,6963,7402],[9632,6424,33705],[9633,5269,41946],[9634,7209,31212],[9635,4441,53436],[9636,3077,53145],[9637,1301,15889],[9638,1293,12191],[9639,5351,71405],[9640,4150,94816],[9641,3600,38682],[9642,558,84492],[9644,8819,63998],[9645,871,96724],[9647,3383,29253],[9648,4372,96817],[9649,6804,45496],[9650,1239,4771],[9652,4451,13989],[9653,5468,61656],[9655,2085,30396],[9656,2947,99026],[9658,1481,55967],[9659,8902,6479],[9660,5461,60189],[9661,2575,85595],[9662,3435,4702],[9663,5464,60671],[9664,2437,67035],[9665,2034,4770],[9666,7500,32901],[9667,5652,94682],[9668,7936,54064],[9669,7948,6759],[9670,3537,45858],[9671,9250,22725],[9672,8480,20951],[9674,8067,26509],[9675,2338,80800],[9676,281,15873],[9677,8302,65086],[9678,6514,13531],[9679,3390,74533],[9680,832,68886],[9681,8082,78535],[9682,816,65297],[9683,3591,5965],[9684,6115,29395],[9685,425,54647],[9686,6405,4655],[9687,6022,13226],[9688,9097,38989],[9689,396,47386],[9690,6358,77775],[9691,5174,79973],[9693,2583,67007],[9694,5432,82067],[9695,5760,78952],[9696,3118,34363],[9697,6958,76681],[9698,8477,7468],[9699,9124,96142],[9700,4625,64001],[9701,7625,14385],[9702,9425,75566],[9703,6397,404],[9704,8691,26139],[9705,352,81304],[9706,8225,90452],[9707,2806,68106],[9708,4611,95054],[9709,4067,77347],[9710,6512,94771],[9711,9011,79430],[9713,9258,75803],[9714,3316,65652],[9715,5303,69183],[9716,1790,8429],[9717,1890,79908],[9718,8443,15628],[9720,578,95704],[9721,8684,5644],[9722,7856,76710],[9723,2600,48148],[9724,9581,12498],[9725,5557,77763],[9726,7766,11898],[9727,3239,31195],[9728,644,52716],[9729,7555,66604],[9731,8938,41166],[9732,7956,42295],[9733,8020,27196],[9734,5554,95987],[9735,7202,249],[9736,9002,85860],[9737,3313,69873],[9738,2464,25383],[9739,8191,81054],[9741,3352,35549],[9745,6232,66026],[9746,1174,13337],[9747,7525,12886],[9748,5720,18721],[9749,2380,76992],[9750,1369,25932],[9752,8167,1372],[9754,6160,973],[9755,532,2756],[9757,2918,50414],[9758,3507,45152],[9759,1845,65995],[9760,3276,4161],[9761,6473,11600],[9763,6147,72905],[9765,6373,27766],[9766,1597,53688],[9767,4061,94033],[9768,6805,70887],[9770,7731,50339],[9771,1454,5333],[9772,5207,68604],[9774,8758,65791],[9775,2102,4418],[9776,4672,86318],[9777,559,5455],[9778,4077,46402],[9779,7654,61085],[9780,3491,83132],[9781,4931,65038],[9782,763,54214],[9783,1186,29710],[9784,344,14168],[9785,2530,81215],[9786,225,32024],[9787,5813,75197],[9788,9696,1271],[9789,5827,75636],[9790,5173,89004],[9791,8646,75996],[9792,7747,9885],[9793,9025,22849],[9794,2783,33459],[9796,2387,48594],[9797,345,14237],[9798,5500,15600],[9800,4955,28643],[9801,6671,21740],[9803,4998,16459],[9804,569,47676],[9806,494,73832],[9807,8143,91371],[9808,2918,63747],[9809,5360,18926],[9810,7414,10773],[9812,7002,93241],[9813,8703,26932],[9814,1179,29752],[9815,166,24802],[9816,5283,99131],[9817,2701,38890],[9818,4593,53015],[9820,1559,27266],[9822,6694,40296],[9823,5361,82568],[9824,2789,10804],[9825,8164,93824],[9826,9538,91940],[9827,8235,86375],[9828,7678,77716],[9829,5520,94195],[9830,1440,60277],[9831,4067,94746],[9832,4696,91785],[9833,3153,70737],[9834,3747,21233],[9835,7808,17456],[9836,934,58265],[9838,6794,44307],[9839,7014,36014],[9840,5222,20789],[9841,5965,65465],[9843,8841,10398],[9844,3487,91391],[9845,6481,14506],[9846,9717,85260],[9847,7394,26063],[9848,601,90541],[9849,6522,48008],[9850,8528,87125],[9851,9803,37208],[9853,9151,98332],[9854,6949,37592],[9855,7881,40384],[9856,6096,31131],[9858,6709,4394],[9859,2691,12032],[9860,8982,77023],[9862,2267,79255],[9863,2935,24262],[9864,6861,60520],[9865,3139,11125],[9866,5343,1829],[9867,835,96678],[9868,1912,16569],[9869,6651,31918],[9870,2831,72745],[9872,4967,78451],[9873,5250,45512],[9874,9772,62004],[9875,4704,8889],[9876,226,96109],[9877,6346,29828],[9878,1774,62270],[9879,829,16855],[9880,2136,44796],[9881,553,8791],[9882,6244,53133],[9884,3857,40249],[9885,7485,74693],[9886,922,10504],[9888,4610,78215],[9889,4968,95032],[9890,6402,71039],[9891,9445,56398],[9892,1616,63030],[9893,8301,23145],[9894,9099,40183],[9895,4923,46560],[9896,7342,38820],[9897,9778,83347],[9898,8043,30733],[9899,8802,48601],[9900,7350,63683],[9901,1489,68808],[9902,3351,29370],[9904,7064,5299],[9905,7224,2978],[9906,2879,7503],[9907,1968,37280],[9908,8991,55752],[9909,2200,50837],[9910,4405,56514],[9911,8093,73047],[9912,9724,19983],[9914,5377,42505],[9915,7449,52984],[9916,2571,91118],[9917,4019,42295],[9918,4808,19653],[9919,8001,56342],[9920,618,21640],[9921,8099,20170],[9922,8360,94082],[9924,485,62940],[9925,7408,33863],[9926,4546,9793],[9927,5849,70538],[9928,2625,71237],[9929,2361,30473],[9930,3344,27940],[9932,7543,93304],[9933,6066,45906],[9934,8482,75246],[9936,2485,23874],[9937,2533,53754],[9940,3180,74371],[9941,8159,84904],[9942,7519,44239],[9943,4798,44997],[9944,2136,20154],[9945,4068,93891],[9946,6298,91514],[9947,2331,6737],[9948,323,8274],[9949,8291,94590],[9950,427,59934],[9951,2792,901],[9952,9171,59643],[9953,4718,99168],[9954,9737,53076],[9955,4800,72935],[9956,5875,31166],[9957,3787,78114],[9958,416,25694],[9959,9228,44264],[9960,9123,63347],[9961,1067,30085],[9962,1570,20956],[9963,3933,40659],[9964,4597,96598],[9965,3765,13652],[9966,3071,52538],[9967,2061,21144],[9968,316,31620],[9969,4563,47436],[9970,380,95750],[9971,8735,24913],[9973,6169,41728],[9974,8327,94077],[9975,6129,41032],[9976,1467,95722],[9977,202,19633],[9978,1014,35297],[9979,240,27156],[9980,1367,40809],[9981,581,90598],[9982,4748,85735],[9983,1152,90998],[9984,7127,81725],[9985,3334,39841],[9986,3974,25449],[9987,4230,92359],[9988,4916,37387],[9989,1205,60560],[9990,5881,24515],[9991,8645,27564],[9992,7205,22024],[9993,4638,65069],[9994,1345,1464],[9995,512,16077],[9996,3862,57333],[9997,1765,81955],[9998,5452,97355],[9999,3956,66964],[10000,8761,55151]]

s = Solution()    
res = s.minCostToSupplyWater(n, wells_list, pipe_list)
print(res)


##########################################################################
stack: []
well_status= [9, 11, 12, 16, 18, 19, 22, 27, 34, 42, 46, 48, 52, 53, 59, 61, 66, 80, 84, 94, 97, 98, 111, 112, 117, 119, 122, 123, 128, 130, 131, 134, 140, 147, 150, 151, 153, 157, 161, 164, 165, 169, 171, 172, 173, 179, 180, 184, 191, 193, 194, 195, 198, 199, 203, 204, 208, 211, 213, 216, 217, 218, 220, 221, 222, 224, 230, 237, 239, 248, 250, 251, 254, 255, 256, 260, 264, 266, 269, 274, 275, 277, 279, 280, 284, 287, 290, 293, 295, 297, 299, 304, 314, 317, 321, 322, 325, 329, 330, 333, 334, 337, 339, 340, 346, 348, 349, 350, 351, 353, 356, 357, 362, 364, 366, 368, 369, 373, 378, 379, 381, 383, 384, 385, 386, 387, 388, 393, 394, 397, 400, 408, 409, 411, 412, 415, 417, 418, 428, 433, 439, 441, 442, 443, 450, 452, 453, 455, 459, 460, 465, 466, 468, 471, 472, 474, 475, 476, 477, 480, 482, 483, 487, 489, 491, 500, 501, 502, 505, 506, 507, 508, 510, 514, 515, 517, 518, 521, 523, 524, 526, 529, 530, 534, 543,

In [ ]:
323010112